## 1. Environment Setup and Dependencies

In [ ]:
!pip install pydantic openai pandas python-dotenv nest_asyncio pyyaml

In [ ]:
import os
import json
import re
import yaml
import asyncio
import logging
import pandas as pd
import nest_asyncio
from typing import List, Dict, Any, Optional, Tuple
from pydantic import BaseModel, Field
from datetime import datetime
from enum import Enum

# Apply nest_asyncio to allow async loops in notebook
nest_asyncio.apply()

# Configure Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("Evaluation")

In [ ]:
# --- CONFIGURATION ---
# Replace with your actual API Key
GOOGLE_API_KEY = "YOUR_API_KEY_HERE" 
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
GEMINI_MODEL = "gemini-1.5-flash"

SYSTEM_PROMPT_VI = """
Bạn là trợ lý ảo AI chuyên nghiệp của cửa hàng JanSport Việt Nam.
Nhiệm vụ của bạn là hỗ trợ khách hàng tìm kiếm sản phẩm, kiểm tra đơn hàng và giải đáp thắc mắc.
Luôn trả lời bằng tiếng Việt, giọng điệu thân thiện, nhiệt tình và chuyên nghiệp.
"""

# Mock Context Config (from context_config.yaml)
CONTEXT_CONFIG = {
    "nodes": [
        {
            "id": "guest_root",
            "label": "Guest Root",
            "children": [
                {"label": "Tìm sản phẩm", "value": "action:product_search"},
                {"label": "Tra cứu đơn hàng", "value": "action:order_track"}
            ]
        },
        {
            "id": "customer_root",
            "label": "Customer Root",
            "children": [
                {"label": "Đơn hàng của tôi", "value": "action:order_list"},
                {"label": "Giỏ hàng", "value": "action:view_cart"}
            ]
        }
    ]
}

In [ ]:
# --- DATA MODELS ---
# Combined from app/models/agent_types.py and app/models/api_models.py

class SessionContext(BaseModel):
    last_messages: List[Dict[str, Any]] = Field(default_factory=list)
    cart_id: Optional[str] = None
    last_product_ids: List[str] = Field(default_factory=list)

class ProcessedInput(BaseModel):
    session_id: str
    customer_id: Optional[str] = None
    text: str
    cleaned_text: str
    language: str = "vi"  # "vi" | "en"
    user_type: str = "guest"  # "guest" | "customer"
    tag: Optional[str] = None
    session_ctx: SessionContext = Field(default_factory=SessionContext)

class IntentResult(BaseModel):
    intent: str
    sub_intent: Optional[str] = None
    confidence: float = 0.0
    entities: Dict[str, Any] = Field(default_factory=dict)

class ActionPlan(BaseModel):
    tools: List[str] = Field(default_factory=list)
    required_data: List[str] = Field(default_factory=list)
    next_step: Optional[str] = None
    escalate: bool = False

class ToolResults(BaseModel):
    ok: bool
    data: Any = None
    errors: List[str] = Field(default_factory=list)
    timings_ms: Dict[str, int] = Field(default_factory=dict)

# From api_models.py
class ChatActionType(str, Enum):
    NAVIGATE = "navigate"
    API_CALL = "api_call"
    FORM = "form"
    SHOW_PRODUCTS = "show_products"
    REQUEST_ORDER_INFO = "request_order_info"
    GUIDE_TO_CART = "guide_to_cart"
    NONE = "none"

class ChatAction(BaseModel):
    type: ChatActionType = ChatActionType.NONE
    payload: Dict[str, Any] = Field(default_factory=dict)

class QuickReply(BaseModel):
    label: str
    value: str
    metadata: Optional[Dict[str, Any]] = None

class ProductVariantInfo(BaseModel):
    id: str
    title: str
    price: Optional[str] = None
    currency_code: Optional[str] = None

class ProductInfo(BaseModel):
    id: str
    title: str
    handle: str
    thumbnail: Optional[str] = None
    price: Optional[str] = None
    currency_code: Optional[str] = None
    variants: List[ProductVariantInfo] = Field(default_factory=list)
    
class ChatRequest(BaseModel):
    message: str
    session_id: str
    customer_id: Optional[str] = None
    tag: Optional[str] = None
    language: Optional[str] = None
    metadata: Dict[str, Any] = Field(default_factory=dict)

class ChatResponse(BaseModel):
    response: str
    session_id: str
    products: List[ProductInfo] = Field(default_factory=list)
    quick_replies: List[QuickReply] = Field(default_factory=list)
    metadata: Dict[str, Any] = Field(default_factory=dict)

In [ ]:
# --- RESPONSE TEMPLATES (app/agents/response_templates.py) ---
# Copied exactly from source

RESPONSE_TEMPLATES = {
    # Greetings
    "greeting": [
        "Xin chào! Tôi là trợ lý ảo của shop. Tôi có thể giúp gì cho bạn hôm nay? 😊",
        "Chào bạn! Rất vui được hỗ trợ bạn. Bạn cần tìm sản phẩm gì không?",
        "Hello! Chào mừng bạn đến với shop. Tôi có thể tư vấn gì cho bạn?"
    ],
    
    # Product Search Success
    "product_found": [
        "Mình tìm thấy {count} sản phẩm phù hợp với '{query}' ạ:",
        "Đây là {count} sản phẩm mình tìm được cho bạn về '{query}':",
        "Shop có {count} sản phẩm '{query}' như sau:"
    ],
    
    # Product Not Found
    "product_not_found": [
        "Rất tiếc, shop không tìm thấy sản phẩm '{query}' bạn cần 😔\nBạn có thể thử tìm kiếm sản phẩm tương tự không?",
        "Hiện tại shop chưa có sản phẩm '{query}' ạ. Bạn muốn xem sản phẩm khác không?",
        "Mình không tìm thấy '{query}' trong kho. Bạn có thể mô tả chi tiết hơn hoặc xem sản phẩm tương tự?"
    ],
    
    # Product Recommend
    "product_recommend": [
        "🌟 Đây là một số sản phẩm hot mà bạn có thể thích:",
        "Mình gợi ý cho bạn vài sản phẩm đang bán chạy nha:",
        "Đây là top sản phẩm được yêu thích nhất của shop:"
    ],
    
    # Cart Actions
    "cart_added": [
        "Đã thêm {product} vào giỏ hàng! Bạn muốn tiếp tục mua sắm hay thanh toán luôn?",
        "OK! Đã cho {product} vào giỏ rồi nhé. Mua thêm gì nữa không?"
    ],
    
    "cart_view_empty": [
        "Giỏ hàng của bạn đang trống. Tìm sản phẩm ngay nhé! 🛍️",
        "Bạn chưa có sản phẩm nào trong giỏ. Cùng khám phá shop nhé!"
    ],
    
    "cart_view_has_items": [
        "Giỏ hàng của bạn có {count} sản phẩm:",
        "Bạn đang có {count} món trong giỏ:"
    ],
    
    # Order Tracking
    "order_found": [
        "Đơn hàng #{order_id} của bạn:\nTrạng thái: {status}\nDự kiến giao: {delivery_date}",
        "Mình check đơn #{order_id} rồi:\n📦 {status}\n🚚 Giao dự kiến: {delivery_date}"
    ],
    
    "order_not_found": [
        "Không tìm thấy đơn hàng #{order_id}. Bạn kiểm tra lại mã đơn giúp mình nhé!",
        "Mình không thấy đơn #{order_id} trong hệ thống. Bạn có chắc mã đúng không?"
    ],
    
    # FAQ
    "faq_shipping": [
        "🚚 Thông tin vận chuyển:\n- Nội thành Hà Nội/HCM: 2-3 ngày, phí 30.000₫\n- Ngoại thành: 3-5 ngày, phí 50.000₫\n- Miễn phí ship cho đơn hàng trên 500.000₫\n- Hỗ trợ COD toàn quốc",
        "Shop ship nhanh trong 2-3 ngày nội thành nha! Phí 30k, FREE SHIP cho đơn trên 500k. Bạn có thể thanh toán COD khi nhận hàng ạ!",
        "Vận chuyển:\n• Giao hàng toàn quốc\n• Nội thành: 2-3 ngày (30k)\n• Ngoại thành: 3-5 ngày (50k)\n• FREE SHIP đơn > 500k\n• Hỗ trợ COD"
    ],
    
    "faq_return": [
        "🔄 Chính sách đổi trả:\n- Đổi trả miễn phí trong 7 ngày\n- Sản phẩm chưa sử dụng, còn nguyên tem mác\n- Miễn phí đổi size/màu (nếu còn hàng)\n- Hoàn tiền trong 3-5 ngày làm việc",
        "Bạn được đổi trả miễn phí trong 7 ngày nếu sản phẩm còn nguyên tem mác nha! Đổi size/màu miễn phí luôn ạ."
    ],
    
    "faq_payment": [
        "💳 Các hình thức thanh toán:\n- COD (tiền mặt khi nhận hàng)\n- Chuyển khoản\n- Thẻ tín dụng/ghi nợ\n- Ví điện tử (Momo, ZaloPay)",
        "Bạn có thể thanh toán bằng COD, chuyển khoản, thẻ, hoặc ví Momo/ZaloPay nha!"
    ],
    
    "faq_promo": [
        "🎁 Khuyến mãi hiện tại:\n- Giảm 10% đơn đầu tiên\n- Free ship đơn >500k\n- Tích điểm đổi quà",
        "Bạn mới có ưu đãi giảm 10% đơn đầu + freeship đơn >500k đó! 🎉"
    ],
    
    # Thank you & Goodbye
    "thank_you": [
        "Không có gì! Rất vui được hỗ trợ bạn. Cần gì cứ gọi mình nhé! 😊",
        "Hehe, có gì đâu! Cần gì cứ nhắn mình nha!",
        "Luôn sẵn sàng giúp bạn! Mua sắm vui vẻ nhé! 🛍️"
    ],
    
    "goodbye": [
        "Tạm biệt! Hẹn gặp lại bạn sớm. Chúc bạn một ngày tốt lành! 👋",
        "Bye bye! Nhớ ghé shop thường xuyên nha! 😊",
        "Hẹn gặp lại! Mua sắm vui vẻ nhé! 🛍️"
    ],
    
    # Errors
    "error_generic": [
        "Xin lỗi, có lỗi xảy ra. Bạn có thể thử lại không?",
        "Ối, có gì đó không ổn. Mình thử lại nhé!"
    ],
    
    "error_no_products": [
        "Shop đang cập nhật sản phẩm. Bạn quay lại sau nhé!",
        "Hiện tại sản phẩm đang hết hàng. Mình sẽ thông báo khi có hàng!"
    ],
    
    # Staff/Manager
    "staff_success": [
        "Đã xử lý xong! Kết quả:",
        "Done! Thông tin bạn cần:"
    ],
    
    "manager_report": [
        "📊 Báo cáo {report_type}:\n{data}",
        "Đây là thống kê {report_type} bạn yêu cầu:\n{data}"
    ],
    
    # Clarification needed
    "need_clarification": [
        "Mình chưa hiểu rõ lắm. Bạn có thể nói rõ hơn được không?",
        "Hmm, bạn muốn tìm gì nhỉ? Mô tả chi tiết giúp mình nha!",
        "Bạn cần tìm sản phẩm gì? Cho mình biết thêm thông tin nhé!"
    ],
    
    "ask_product_details": [
        "Bạn muốn tìm loại nào? (màu, size, giá...)",
        "Cho mình biết thêm chi tiết: màu sắc, kích thước, giá cả?"
    ],
    
    "ask_order_id": [
        "Bạn cho mình mã đơn hàng để check nhé! (VD: #1234 hoặc ORD-5678)",
        "Mã đơn hàng của bạn là gì ạ?"
    ]
}

def get_response_template(intent: str, context: Optional[dict] = None) -> str:
    """Get appropriate response template based on intent and context"""
    import random
    
    context = context or {}
    templates = RESPONSE_TEMPLATES.get(intent, RESPONSE_TEMPLATES["need_clarification"])
    
    # Select random template for variety
    template = random.choice(templates)
    
    # Format with context if available
    try:
        if context:
            template = template.format(**context)
    except KeyError:
        pass  # Use template as-is if formatting fails
    
    return template

In [ ]:
# --- MOCKS FOR EXTERNAL DEPENDENCIES ---

# Mock Tools to simulate Medusa/Redis calls
class MockTools:
    @staticmethod
    async def search_products(query, limit=5):
        return ToolResults(ok=True, data=[
            {"id": "p_1", "title": f"JanSport SuperBreak - {query}", "handle": "superbreak", "price": "500000", "currency_code": "vnd"},
            {"id": "p_2", "title": f"JanSport Big Student - {query}", "handle": "big-student", "price": "800000", "currency_code": "vnd"}
        ])
    
    @staticmethod
    async def get_product_details(product_id):
        return ToolResults(ok=True, data={"id": product_id, "title": "JanSport Detail", "description": "Great bag", "price": "500000", "currency_code": "vnd"})

    @staticmethod
    async def recommend_products(limit=5):
        return ToolResults(ok=True, data=[{"id": "rec_1", "title": "Trending Bag", "handle": "trending", "price": "600000", "currency_code": "vnd"}])

    @staticmethod
    async def view_cart(cart_id):
        return ToolResults(ok=True, data={"id": cart_id, "items": []})

    @staticmethod
    async def add_to_cart(cart_id, variant_id, qty):
        return ToolResults(ok=True, data={"cart_id": cart_id, "added": True})
        
    @staticmethod
    async def add_to_cart_smart(cart_id, query, qty):
        return ToolResults(ok=True, data={"cart_id": cart_id, "added": True, "product": query})

    @staticmethod
    async def lookup_order(order_id):
        if order_id == "123":
            return ToolResults(ok=True, data={"id": "123", "status": "shipped", "delivery_date": "2023-12-25"})
        return ToolResults(ok=False, errors=["Order not found"])
    
    @staticmethod
    async def list_orders(customer_id):
        return ToolResults(ok=True, data=[{"id": "123", "status": "shipped"}])

    # Staff/Manager Mocks
    @staticmethod
    async def check_stock(query): return ToolResults(ok=True, data={"product": query, "stock": 10})
    @staticmethod
    async def check_price(query): return ToolResults(ok=True, data={"product": query, "price": 500000})
    @staticmethod
    async def lookup_customer(query): return ToolResults(ok=True, data={"id": "cust_1", "name": "Test User"})
    @staticmethod
    async def get_customer_order_history(cid): return ToolResults(ok=True, data=[])
    @staticmethod
    async def create_draft_order(cid, items): return ToolResults(ok=True, data={"id": "draft_1"})
    @staticmethod
    async def update_order_status(oid, status): return ToolResults(ok=True, data={"id": oid, "status": status})
    @staticmethod
    async def print_shipping_label(oid): return ToolResults(ok=True, data={"url": "http://label.pdf"})
    @staticmethod
    async def get_sales_report(): return ToolResults(ok=True, data={"total_sales": 10000000})
    @staticmethod
    async def get_top_products(): return ToolResults(ok=True, data=[{"title": "Bag A", "sales": 100}])
    @staticmethod
    async def get_customer_analytics(): return ToolResults(ok=True, data={"new_users": 50})

# Mock Specialized Agents (to avoid importing their complex logic)
class MockSpecializedAgent:
    def __init__(self, name): self.name = name
    def can_handle(self, intent): return False 
    async def process(self, processed, intent): 
        # Simple pass-through for testing
        return ActionPlan(tools=[], next_step=f"mock_{self.name}"), None

# Define tool objects for Executor to use
cart_tools = MockTools
product_tools = MockTools
product_recommend_tool = MockTools
order_tools = MockTools
staff_tools = MockTools
report_tools = MockTools
system_tools = MockTools
customer_tools = MockTools

In [ ]:
# --- AGENT 1: INPUT PROCESSOR (app/agents/input_processor.py) ---
# from typing import Optional, Dict, Any, List
# import re
# import asyncpg
# from app.models import ChatRequest
# from app.models.agent_types import ProcessedInput, SessionContext

def _normalize_text(text: str) -> str:
    # Trim, collapse spaces; keep diacritics (for VI) and punctuation minimally
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text


def _detect_language_heuristic(text: str) -> str:
    t = text.lower()
    # Very lightweight heuristic: VI keywords/diacritics vs common EN markers
    if any(ch in t for ch in "ăâêôơưđáàảãạấầẩẫậắằẳẵặéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ"):
        return "vi"
    if any(word in t for word in [" the ", " and ", " price ", " order ", " product ", " hello "]):
        return "en"
    # Fallback: prefer VI as primary market
    return "vi"


async def _fetch_session_context(pool: Any, session_id: str) -> SessionContext:
    # MOCK IMPLEMENTATION
    ctx = SessionContext()
    # Simulate some context if needed
    return ctx


class InputProcessor:
    async def run(self, req: ChatRequest, pool: Any = None) -> ProcessedInput:
        print(f"[InputProcessor] Processing request: session={req.session_id}, msg='{req.message}'")
        cleaned = _normalize_text(req.message)
        lang = req.language or _detect_language_heuristic(cleaned)
        
        # Determine user_type:
        user_type = "guest"
        if req.metadata and "user_type" in req.metadata:
            user_type = req.metadata["user_type"]
        elif req.customer_id:
            user_type = "customer"
            
        session_ctx = await _fetch_session_context(pool, req.session_id)
        print(f"[InputProcessor] Context loaded: last_intent={session_ctx.last_messages[0]['intent'] if session_ctx.last_messages else 'None'}")

        return ProcessedInput(
            session_id=req.session_id,
            customer_id=req.customer_id,
            text=req.message,
            cleaned_text=cleaned,
            language=lang,
            user_type=user_type,
            tag=req.tag,
            session_ctx=session_ctx,
        )

In [ ]:
# --- AGENT 2: INTENT CLASSIFIER (app/agents/intent_classifier.py) ---
# from typing import Dict, List
# import re
# from app.models.agent_types import ProcessedInput, IntentResult

VI_KEYWORDS: Dict[str, List[str]] = {
    "GREETING": ["xin chào", "chào", "alo", "chào bạn", "hello", "hi", "hey", "chào buổi", "chào admin", "chào shop"],
    "PRODUCT.SEARCH": [
        "tìm", "tìm kiếm", "có không", "còn không", "có bán", "muốn mua", "giá", "sản phẩm", 
        "mua", "đặt", "kiếm", "tương tự", "giống", "cần", "shop có", "bên này có", "có hàng",
        "còn hàng", "hết hàng chưa", "còn size", "có màu", "loại", "dòng", "model"
    ],
    "PRODUCT.DETAIL": [
        "chi tiết", "thông số", "xem kỹ", "thông tin", "cụ thể", "mô tả", "spec", 
        "đặc điểm", "tính năng", "chất liệu", "kích thước", "size", "màu sắc", "xuất xứ"
    ],
    "PRODUCT.RECOMMEND": [
        "gợi ý cho tôi", "gợi ý cho mình", "gợi ý vài", "nên mua gì", "mua gì", 
        "hot trend", "bán chạy", "phổ biến", "đề xuất", "recommend me",
        "trending", "mới nhất", "best seller", "top sản phẩm", "sản phẩm tốt", "sản phẩm hay", "đáng mua nhất",
        "xem sản phẩm nào", "có sản phẩm nào", "sản phẩm nào tốt"
    ],
    "ORDER.TRACK": [
        "tra cứu", "kiểm tra", "đơn hàng", "ở đâu", "tình trạng", "tracking", "giao hàng",
        "ship", "vận chuyển", "đã giao chưa", "khi nào nhận", "bao giờ về", "đến chưa"
    ],
    "ORDER.CANCEL": ["hủy đơn", "không mua nữa", "huỷ đơn", "cancel", "không muốn", "đổi ý", "không lấy"],
    "ORDER.RETURN": [
        "đổi trả", "hoàn tiền", "bảo hành", "trả hàng", "return", "refund", "warranty",
        "đổi hàng", "lỗi", "hư", "không vừa", "sai", "không đúng"
    ],
    "CART.VIEW": ["xem giỏ", "giỏ hàng", "trong giỏ", "cart", "giỏ", "đã chọn", "đã thêm"],
    "CART.ADD": [
        "thêm vào giỏ", "bỏ vào giỏ", "mua cái này", "lấy cái này", "đặt cái này", 
        "lấy cho", "thêm 1", "đặt 1", "đặt hàng", "mua ngay", "chốt", "order",
        "cho vào giỏ", "bỏ giỏ", "add", "thêm", "bỏ", "cho tôi", "cho em",
        "mình lấy", "mình mua", "lấy luôn", "mua luôn"
    ],
    "CART.REMOVE": ["xóa khỏi giỏ", "bỏ ra", "xóa sản phẩm", "bỏ sản phẩm", "remove", "không lấy", "bỏ"],
    "CART.UPDATE": ["đổi số lượng", "thay đổi", "sửa giỏ", "update cart", "tăng", "giảm", "đổi size", "đổi màu"],
    "CHECKOUT": ["thanh toán", "checkout", "đặt luôn", "mua luôn", "pay", "trả tiền", "hoàn tất"],
    "ACCOUNT.LOGIN": ["đăng nhập", "login", "sign in", "vào tài khoản"],
    "ACCOUNT.REGISTER": ["đăng ký", "tạo tài khoản", "sign up", "register", "mở tài khoản"],
    "FAQ.SHIPPING": [
        "phí ship", "vận chuyển", "bao lâu", "giao hàng", "ship", "shipping",
        "delivery", "mất bao lâu", "khi nào nhận", "ship cod", "ship nhanh"
    ],
    "FAQ.PAYMENT": [
        "thanh toán", "chuyển khoản", "cod", "trả tiền", "payment", 
        "hình thức thanh toán", "trả góp", "ví", "momo", "zalopay", "banking"
    ],
    "FAQ.PROMO": ["khuyến mãi", "giảm giá", "sale", "voucher", "mã giảm", "coupon", "promotion", "ưu đãi"],
    "FAQ.RETURN": [
        "đổi trả", "chính sách đổi", "return policy", "hoàn tiền", "refund",
        "bảo hành", "warranty", "trả hàng", "đổi hàng", "chính sách hoàn"
    ],
    "SUPPORT.ESCALATE": [
        "nhân viên", "người thật", "không phải bot", "tư vấn", "hỗ trợ", 
        "admin", "staff", "human", "real person", "agent"
    ],
    "THANK": ["cảm ơn", "thanks", "thank you", "cám ơn", "ơn"],
    "GOODBYE": ["tạm biệt", "bye", "goodbye", "hẹn gặp lại", "đi đây"],
    # Staff / Manager Keywords
    "STAFF.CHECK_STOCK": ["check kho", "tồn kho", "kiểm kho", "số lượng tồn", "inventory", "stock"],
    "STAFF.CUSTOMER_LOOKUP": ["tìm khách", "check info khách", "thông tin khách", "lookup customer"],
    "STAFF.ORDER_HISTORY": ["lịch sử mua", "đơn cũ của khách", "khách mua gì", "order history"],
    "STAFF.CREATE_ORDER": ["tạo đơn giúp", "lên đơn", "tạo đơn", "create order", "draft order"],
    "MANAGER.REPORT_SALES": ["doanh thu", "báo cáo bán hàng", "doanh số", "sales report", "revenue"],
    "MANAGER.REPORT_CHATBOT": ["hiệu quả bot", "bot chat bao nhiêu", "thống kê bot", "chatbot stats"],
    "MANAGER.CONFIG_UPDATE": ["tắt bot", "bật bot", "chỉnh prompt", "cấu hình", "config"],
}

EN_KEYWORDS: Dict[str, List[str]] = {
    "GREETING": ["hello", "hi", "hey"],
    "PRODUCT.SEARCH": ["search", "find", "looking for", "do you have", "want to buy", "price", "product", "buy", "order"],
    "PRODUCT.DETAIL": ["detail", "specs", "info", "more info", "specification"],
    "PRODUCT.RECOMMEND": ["recommend", "suggest", "hot trend", "best seller", "popular"],
    "ORDER.TRACK": ["track", "check order", "order status", "where is", "shipping"],
    "ORDER.CANCEL": ["cancel order", "cancel"],
    "ORDER.RETURN": ["return", "refund", "warranty"],
    "CART.VIEW": ["view cart", "my cart", "shopping cart"],
    "CART.ADD": ["add to cart", "buy this", "get this", "order this"],
    "CART.REMOVE": ["remove", "delete item", "remove from cart"],
    "ACCOUNT.LOGIN": ["login", "sign in"],
    "ACCOUNT.REGISTER": ["register", "sign up"],
    "FAQ.SHIPPING": ["shipping cost", "delivery time", "shipping fee"],
    "FAQ.PAYMENT": ["payment", "cod", "bank transfer"],
    "SUPPORT.ESCALATE": ["human", "staff", "real person", "agent"],
    # Staff / Manager Keywords
    "STAFF.CHECK_STOCK": ["check stock", "inventory", "stock level"],
    "STAFF.CUSTOMER_LOOKUP": ["lookup customer", "find customer", "user info"],
    "STAFF.ORDER_HISTORY": ["customer order history", "order history"],
    "STAFF.CREATE_ORDER": ["create order", "draft order"],
    "MANAGER.REPORT_SALES": ["sales report", "revenue", "sales stats"],
    "MANAGER.REPORT_CHATBOT": ["chatbot stats", "bot performance"],
    "MANAGER.CONFIG_UPDATE": ["disable bot", "enable bot", "update config"],
}


def _match_score(text: str, phrases: List[str]) -> int:
    t = f" {text.lower()} "
    score = 0
    for p in phrases:
        if f" {p} " in t:
            score += 2 if len(p.split()) > 1 else 1
    return score


def _extract_product_query(cleaned_text: str) -> str:
    s = cleaned_text
    
    # Check if this is a context reference (not a search query)
    context_patterns = [
        r"\b(cái|sản phẩm|sp)\s+(này|kia|đó|ấy|vừa|trước)",
        r"\b(đầu tiên|thứ nhất|thứ hai|cuối cùng)",
        r"\bnó\b",
        r"\bchi tiết\b.*(?!\w+)",  # "chi tiết" without specific product name
    ]
    for pattern in context_patterns:
        if re.search(pattern, s.lower()):
            return ""  # Return empty to signal this needs context
    
    s = re.sub(r"[?,.]", " ", s) # Remove punctuation
    # remove common Vietnamese/English query anchors
    remove = [
        "tìm", "tìm kiếm", "có không", "còn không", "có bán", "muốn mua",
        "search", "find", "looking for", "do you have", "want to buy",
        "sản phẩm", "product", "mua", "đặt", "kiếm", "cho tôi", "cho anh", "cho em", "cho mình",
        "tương tự", "similar",
        "tôi", "anh", "em", "bạn", "mình", "cái", "chiếc", "là", "của",
        "giá", "bao nhiêu", "price", "cost", "how much", "tiền",
        "thông tin", "xem", "về", "info", "about",
        # Staff keywords
        "check kho", "tồn kho", "kiểm kho", "số lượng tồn",
        "check stock", "inventory", "stock level",
        "tìm khách", "check info khách", "thông tin khách", "khách hàng", "khách",
        "lookup customer", "find customer", "user info",
        "nhân viên", "quản lý", "staff", "manager"
    ]
    low = s.lower()
    for r in remove:
        low = low.replace(r, " ")
    low = re.sub(r"\s+", " ", low).strip()
    return low


class IntentClassifier:
    async def run(self, processed: ProcessedInput) -> IntentResult:
        print(f"[IntentClassifier] Classifying: '{processed.cleaned_text}'")
        text = processed.cleaned_text
        lang = processed.language or "vi"

        # Choose keyword table
        table = VI_KEYWORDS if lang == "vi" else EN_KEYWORDS

        # Compute scores
        scores: Dict[str, int] = {}
        for key, phrases in table.items():
            scores[key] = _match_score(text, phrases)
        
        # Boost PRODUCT.RECOMMEND score if recommend phrases are detected
        # This prevents "gợi ý sản phẩm" from being classified as product_inquiry
        recommend_boost_phrases = ["gợi ý cho", "đề xuất cho", "recommend me", "gợi ý vài", "gợi ý một", "nên mua gì"]
        if any(phrase in text.lower() for phrase in recommend_boost_phrases):
            if "PRODUCT.RECOMMEND" in scores:
                scores["PRODUCT.RECOMMEND"] += 5  # Strong boost to override PRODUCT.SEARCH
        
        # Boost PRODUCT.DETAIL for context reference patterns (e.g., "sản phẩm đầu tiên", "cái này")
        context_patterns = [
            r'\b(cái|sản phẩm|sp)\s+(này|kia|đó|ấy|vừa|trước)',
            r'\b(đầu tiên|thứ nhất|thứ hai|thứ ba|cuối cùng)',
            r'\bnó\b',
        ]
        for pattern in context_patterns:
            if re.search(pattern, text.lower()):
                if "PRODUCT.DETAIL" in scores:
                    scores["PRODUCT.DETAIL"] += 10  # Very strong boost to override PRODUCT.SEARCH
                break

        # Get top 3 intents for multi-intent detection
        sorted_scores = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
        top_key = sorted_scores[0][0] if sorted_scores else "UNKNOWN"
        top_score = sorted_scores[0][1] if sorted_scores else 0
        
        # Detect secondary intent (e.g., "tìm balo giá rẻ" = PRODUCT.SEARCH + PRODUCT.PRICE)
        secondary_intent = None
        if len(sorted_scores) > 1 and sorted_scores[1][1] >= 2:
            secondary_intent = sorted_scores[1][0]
        
        print(f"[IntentClassifier] Top: {top_key}({top_score}), Secondary: {secondary_intent}")

        # Map to system intents used by backend
        intent_map = {
            "GREETING": "general",
            "PRODUCT.SEARCH": "product_inquiry",
            "PRODUCT.DETAIL": "product_detail",
            "PRODUCT.RECOMMEND": "product_recommend",
            "ORDER.TRACK": "order_tracking",
            "ORDER.CANCEL": "order_cancel",
            "ORDER.RETURN": "order_return",
            "CART.VIEW": "cart_view",
            "CART.ADD": "cart_add",
            "CART.REMOVE": "cart_remove",
            "CART.UPDATE": "cart_update",
            "CHECKOUT": "checkout",
            "ACCOUNT.LOGIN": "account_login",
            "ACCOUNT.REGISTER": "account_register",
            "FAQ.SHIPPING": "faq_shipping",
            "FAQ.PAYMENT": "faq_payment",
            "FAQ.PROMO": "faq_promo",
            "FAQ.RETURN": "faq_return",
            "SUPPORT.ESCALATE": "human_escalation",
            "THANK": "thank_you",
            "GOODBYE": "goodbye",
            "STAFF.CHECK_STOCK": "staff_check_stock",
            "STAFF.CUSTOMER_LOOKUP": "staff_customer_lookup",
            "STAFF.ORDER_HISTORY": "staff_order_history",
            "STAFF.CREATE_ORDER": "staff_create_order",
            "MANAGER.REPORT_SALES": "manager_report_sales",
            "MANAGER.REPORT_CHATBOT": "manager_report_chatbot",
            "MANAGER.CONFIG_UPDATE": "manager_config_update",
        }

        mapped = intent_map.get(top_key, "general")
        sub_intent = intent_map.get(secondary_intent) if secondary_intent else None

        # Enhanced confidence scoring
        if top_score >= 4:
            conf = 0.95
        elif top_score >= 3:
            conf = 0.85
        elif top_score >= 2:
            conf = 0.70
        elif top_score >= 1:
            conf = 0.50
        else:
            conf = 0.30

        entities = {}
        
        # Enhanced entity extraction
        if mapped == "product_inquiry" or mapped == "cart_add" or mapped == "product_detail":
            q = _extract_product_query(text)
            # For product_detail, empty query means context reference
            if mapped == "product_detail" and q == "":
                entities["product_query"] = ""  # Signal to use context
            elif q and len(q) >= 2:
                entities["product_query"] = q
            
            # Extract quantity
            qty_match = re.search(r'(\d+)\s*(cái|chiếc|con|bộ|món|sản phẩm|sp)', text.lower())
            if qty_match:
                entities["quantity"] = int(qty_match.group(1))
            else:
                entities["quantity"] = 1
                
        elif mapped == "order_tracking":
            # Enhanced order ID extraction
            # Patterns: #1234, order_1234, ORD-1234, đơn 1234
            match = re.search(r'(order[_\s-]?[\w\d]+|#\d+|ord[_\s-]?\d+|đơn[\s]?\d+|\d{4,})', text.lower())
            if match:
                order_id = match.group(0).replace("#", "").replace("đơn", "").replace("order", "").replace("_", "").replace("-", "").strip()
                entities["order_id"] = order_id
                
        elif mapped == "staff_check_stock":
            q = _extract_product_query(text)
            if q:
                entities["product_query"] = q
                
        elif mapped == "staff_customer_lookup":
            # Extract customer info (email, phone, name)
            email_match = re.search(r'[\w\.-]+@[\w\.-]+\.\w+', text)
            phone_match = re.search(r'(\+?84|0)\d{9,10}', text)
            
            if email_match:
                entities["customer_email"] = email_match.group(0)
            elif phone_match:
                entities["customer_phone"] = phone_match.group(0)
            else:
                # Fallback to query extraction
                q = _extract_product_query(text)
                if q:
                    entities["customer_query"] = q
        
        print(f"[IntentClassifier] Result: intent={mapped}, sub={sub_intent}, entities={entities}")
        return IntentResult(
            intent=mapped,
            sub_intent=sub_intent,
            confidence=conf,
            entities=entities,
        )

In [ ]:
# --- AGENT 3: ORCHESTRATOR (app/agents/orchestrator.py) ---
# from typing import Optional, Tuple, Dict
# from app.models.agent_types import ActionPlan, ToolResults, ProcessedInput, IntentResult
# from app.agents.specialized.sales_agent import SalesAgent
# from app.agents.specialized.order_agent import OrderAgent
# from app.agents.specialized.staff_agent import StaffAgent
# from app.agents.specialized.manager_agent import ManagerAgent

class Orchestrator:
    def __init__(self):
        self.agents = {
            "sales": MockSpecializedAgent("sales"),
            "order": MockSpecializedAgent("order"),
            "staff": MockSpecializedAgent("staff"),
            "manager": MockSpecializedAgent("manager")
        }

    async def run(
        self,
        processed: ProcessedInput,
        intent: IntentResult,
    ) -> Tuple[ActionPlan, Optional[ToolResults]]:
        """
        Hybrid Orchestrator:
        1. Executable Tag -> Execute Tool directly
        2. Scope Tag -> Route to Specialized Agent (Scoped NLU)
        3. No Tag -> Global NLU -> Route to Agent
        """
        print(f"[Orchestrator] Routing... Tag={processed.tag}, Intent={intent.intent}")
        
        # 1. Executable Tag (e.g., "action:view_cart")
        if processed.tag and processed.tag.startswith("action:"):
            action = processed.tag.split(":", 1)[1]
            print(f"[Orchestrator] Direct action: {action}")
            
            if action == "view_cart":
                return ActionPlan(tools=["cart.view"], next_step="show_cart"), None
            elif action == "order_list":
                return ActionPlan(tools=["order.list"], next_step="show_orders"), None
            elif action == "order_track":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "reorder":
                return ActionPlan(tools=["order.reorder"], next_step="confirm_reorder"), None
            elif action == "logout":
                return ActionPlan(tools=["auth.logout"], next_step="confirm_logout"), None
            elif action == "chat_human":
                return ActionPlan(tools=["system.escalate"], next_step="escalate_chat"), None
            elif action == "my_reviews":
                return ActionPlan(tools=["product.reviews"], next_step="show_reviews"), None
            elif action == "check_stock":
                return ActionPlan(tools=[], next_step="ask_product_name"), None # Need product name
            elif action == "check_price":
                return ActionPlan(tools=[], next_step="ask_product_name"), None # Need product name
            elif action == "staff_lookup_order":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "staff_customer_lookup":
                return ActionPlan(tools=[], next_step="ask_customer_info"), None
            elif action == "update_order_status":
                return ActionPlan(tools=[], next_step="ask_order_id_status"), None
            elif action == "print_shipping_label":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "sales_report":
                return ActionPlan(tools=["manager.report_sales"], next_step="show_report"), None
            elif action == "top_products":
                return ActionPlan(tools=["manager.top_products"], next_step="show_top_products"), None
            elif action == "customer_analytics":
                return ActionPlan(tools=["manager.customer_analytics"], next_step="show_analytics"), None
            elif action.startswith("cart.add_item"):
                try:
                    params = {}
                    if "?" in action:
                        query = action.split("?", 1)[1]
                        for pair in query.split("&"):
                            if "=" in pair:
                                k, v = pair.split("=", 1)
                                params[k] = v
                    
                    intent.entities["product_id"] = params.get("product_id")
                    intent.entities["variant_id"] = params.get("variant_id")
                    intent.entities["quantity"] = 1
                    
                    return ActionPlan(tools=["cart.add"], next_step="confirm_add_to_cart"), None
                except Exception as e:
                    print(f"Error parsing cart.add_item tag: {e}")
                    return ActionPlan(tools=[], next_step="error"), None
            
            # Add more direct actions here
        
        # 2. Scope Tag (e.g., "scope:order")
        if processed.tag and processed.tag.startswith("scope:"):
            scope = processed.tag.split(":", 1)[1]
            agent = self.agents.get(scope)
            if agent:
                print(f"[Orchestrator] Scoped routing to: {scope}")
                # Agent performs its own scoped NLU if intent is generic/unknown or we want to re-evaluate
                # For now, we pass None as intent to force agent to re-classify within scope
                return await agent.process(processed, intent=None)
        
        # 3. Implicit Routing (Global NLU)
        # We use the intent passed from the global classifier to decide which agent to call
        target_agent = self._route_by_intent(intent.intent)
        if target_agent:
            print(f"[Orchestrator] Implicit routing to: {target_agent.name}")
            return await target_agent.process(processed, intent)
            
        # Default fallback
        print("[Orchestrator] No agent found, fallback.")
        return ActionPlan(tools=[], next_step=None), None

    def _route_by_intent(self, intent_name: str):
        """Route intent to appropriate specialized agent"""
        for agent in self.agents.values():
            if agent.can_handle(intent_name):
                return agent
        return self.agents["sales"] # Default to sales if unknown
    
    def get_agent_suggestions(self, intent: str) -> list:
        """Get suggested actions based on intent - for better UX"""
        suggestions = {
            "product_inquiry": [
                {"label": "Xem giỏ hàng", "action": "action:view_cart"},
                {"label": "Sản phẩm gợi ý", "action": "action:my_recommendations"},
            ],
            "cart_view": [
                {"label": "Thanh toán ngay", "action": "action:checkout"},
                {"label": "Tiếp tục mua sắm", "action": "action:continue_shopping"},
            ],
            "order_tracking": [
                {"label": "Đơn hàng của tôi", "action": "action:order_list"},
                {"label": "Liên hệ hỗ trợ", "action": "action:chat_human"},
            ],
            "general": [
                {"label": "Tìm sản phẩm", "action": "action:product_search"},
                {"label": "Xem giỏ hàng", "action": "action:view_cart"},
                {"label": "Kiểm tra đơn hàng", "action": "action:order_list"},
            ],
        }
        return suggestions.get(intent, suggestions["general"])

In [ ]:
# --- AGENT 4: EXECUTOR (app/agents/executor.py) ---
# import logging
# from typing import Optional, Dict, Any
# from app.models.agent_types import ProcessedInput, IntentResult, ActionPlan, ToolResults
# from app.tools import ...

def _merge_tool_result(results: ToolResults, tool_result: Any):
    if tool_result is None:
        return
    if isinstance(tool_result, ToolResults):
        results.ok = results.ok and tool_result.ok
        if tool_result.errors:
            results.errors.extend(tool_result.errors)
        if tool_result.timings_ms:
            results.timings_ms.update(tool_result.timings_ms)
        results.data = tool_result.data
    else:
        results.data = tool_result

class Executor:
    async def run(
        self, 
        processed: ProcessedInput, 
        intent: IntentResult, 
        plan: ActionPlan
    ) -> Optional[ToolResults]:
        if not plan.tools:
            return None

        results = ToolResults(ok=True)
        
        for tool_name in plan.tools:
            try:
                logger.info(f"[Executor] Running tool: {tool_name}")
                
                # Cart Tools
                if tool_name == "cart.view":
                    if processed.session_ctx.cart_id:
                        res = await cart_tools.view_cart(processed.session_ctx.cart_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing cart_id")
                elif tool_name == "cart.add":
                    # Extract product and quantity from entities
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    qty = intent.entities.get("quantity", 1)
                    variant_id = intent.entities.get("variant_id")
                    
                    if processed.session_ctx.cart_id:
                        if variant_id:
                            # If we have explicit variant_id (e.g. from UI action), use it directly
                            res = await cart_tools.add_to_cart(processed.session_ctx.cart_id, variant_id, qty)
                        else:
                            # Otherwise try smart add (search -> add)
                            res = await cart_tools.add_to_cart_smart(processed.session_ctx.cart_id, query, qty)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing cart_id")
                
                # Product Tools
                elif tool_name == "product.search":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await product_tools.search_products(query)
                    _merge_tool_result(results, res)
                elif tool_name == "product.recommend":
                    # The tool currently only accepts limit, not customer_id
                    res = await product_recommend_tool.recommend_products(limit=5)
                    _merge_tool_result(results, res)
                elif tool_name == "product.detail":
                    # Try to get product_id from entities first
                    product_id = intent.entities.get("product_id")
                    
                    # If no product_id in entities, try to get from context (last products shown)
                    if not product_id and processed.session_ctx.last_product_ids:
                        product_id = processed.session_ctx.last_product_ids[0]
                        logger.info(f"[Executor] Using context product_id: {product_id}")
                    
                    if product_id:
                        res = await product_tools.get_product_details(product_id)
                        _merge_tool_result(results, res)
                    else:
                        # Fallback: try to search if we have a query
                        # Extract product name from text (remove "chi tiết", "xem", etc)
                        import re
                        query = intent.entities.get("product_query")
                        if not query:
                            # Try to extract product name from cleaned text
                            text_lower = processed.cleaned_text.lower()
                            # Remove common phrases (order matters - remove longer phrases first)
                            for phrase in ["cho tôi xem chi tiết", "cho tôi xem", "xem chi tiết", "chi tiết của", "chi tiết", "xem", "tôi muốn", "muốn"]:
                                text_lower = text_lower.replace(phrase, "")
                            query = text_lower.strip()
                        
                        if query and len(query) >= 2:
                            logger.info(f"[Executor] Searching for product: '{query}'")
                            res = await product_tools.search_products(query, limit=1)
                            _merge_tool_result(results, res)
                            # Store search query in entities for response_generator
                            intent.entities["product_query"] = query
                        else:
                            results.ok = False
                            results.errors.append("Missing product_id or query")
                            # Still store a placeholder for template
                            intent.entities["product_query"] = "sản phẩm bạn cần"

                # Order Tools
                elif tool_name == "order.track":
                    order_id = intent.entities.get("order_id")
                    email = intent.entities.get("email") # Might need to ask user
                    if order_id:
                        res = await order_tools.lookup_order(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "order.list":
                    if processed.customer_id:
                        res = await order_tools.list_orders(processed.customer_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "order.reorder":
                    # Assuming we reorder the last order or specific order
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await order_tools.reorder(order_id)
                        _merge_tool_result(results, res)
                    else:
                        # Maybe list orders to pick?
                        results.ok = False
                        results.errors.append("Missing order_id")

                # Staff Tools
                elif tool_name == "staff.check_stock":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await staff_tools.check_stock(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.check_price":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await staff_tools.check_price(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.customer_lookup":
                    query = intent.entities.get("customer_query", processed.cleaned_text)
                    res = await staff_tools.lookup_customer(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.order_history":
                    customer_id = intent.entities.get("customer_id")
                    if customer_id:
                        res = await staff_tools.get_customer_order_history(customer_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "staff.create_order":
                    if processed.customer_id:
                        res = await staff_tools.create_draft_order(processed.customer_id, [])
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "staff.lookup_order":
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await staff_tools.lookup_order(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "staff.update_order":
                    order_id = intent.entities.get("order_id")
                    status = intent.entities.get("status", "processing")
                    if order_id:
                        res = await staff_tools.update_order_status(order_id, status)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "staff.print_label":
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await staff_tools.print_shipping_label(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                
                # Manager Tools
                elif tool_name == "manager.report_sales":
                    res = await report_tools.get_sales_report()
                    _merge_tool_result(results, res)
                elif tool_name == "manager.top_products":
                    res = await report_tools.get_top_products()
                    _merge_tool_result(results, res)
                elif tool_name == "manager.customer_analytics":
                    res = await report_tools.get_customer_analytics()
                    _merge_tool_result(results, res)

                # System/Auth Tools
                elif tool_name == "system.escalate":
                    # Just a marker, maybe log it
                    results.data = {"status": "escalated", "message": "Connecting to human agent..."}
                elif tool_name == "auth.logout":
                    # In a real app, this might invalidate session
                    results.data = {"status": "logged_out", "message": "You have been logged out."}
                
                # Product Tools (Extra)
                elif tool_name == "product.reviews":
                    # Mock
                    results.data = {"reviews": []}

                else:
                    logger.warning(f"[Executor] Unknown tool: {tool_name}")
                    results.errors.append(f"Unknown tool: {tool_name}")

            except Exception as e:
                logger.error(f"[Executor] Error running {tool_name}: {e}")
                results.ok = False
                results.errors.append(str(e))
        
        return results

In [ ]:
# --- AGENT 5: RESPONSE GENERATOR (app/agents/response_generator.py) ---
# import os
# import yaml
# import json
# import logging
# from typing import List, Dict, Any, Optional
# from openai import AsyncOpenAI
# from openai.types.chat import ChatCompletionSystemMessageParam, ChatCompletionUserMessageParam
# from app.config import GOOGLE_API_KEY, GEMINI_BASE_URL, GEMINI_MODEL, SYSTEM_PROMPT_VI
# from app.models.api_models import ChatResponse, QuickReply, ProductInfo
# from app.models.agent_types import ProcessedInput, IntentResult, ActionPlan, ToolResults
# from app.agents.response_templates import get_response_template

class ResponseGenerator:
    def __init__(self):
        if GOOGLE_API_KEY == "YOUR_API_KEY_HERE":
            self.client = None
        else:
            from openai import AsyncOpenAI
            self.client = AsyncOpenAI(api_key=GOOGLE_API_KEY, base_url=GEMINI_BASE_URL)
        self.context_config = CONTEXT_CONFIG

    def _load_context_config(self):
        return CONTEXT_CONFIG

    async def run(
        self, 
        processed: ProcessedInput, 
        intent: IntentResult, 
        plan: ActionPlan, 
        tool_res: Optional[ToolResults]
    ) -> ChatResponse:
        
        # 1. Generate Text Response
        response_text = await self._generate_llm_response(processed, intent, tool_res)
        
        # 2. Generate Quick Replies (Options)
        quick_replies = self._generate_quick_replies(processed)
        
        # 3. Format Products if available
        products = []
        if tool_res and tool_res.ok and isinstance(tool_res.data, list):
            for item in tool_res.data:
                # Try to map to ProductInfo
                try:
                    # If item is already ProductInfo, append directly
                    if isinstance(item, ProductInfo):
                        products.append(item)
                    # Handle if item is dict
                    elif isinstance(item, dict):
                        # Filter keys that match ProductInfo
                        valid_keys = ProductInfo.model_fields.keys()
                        filtered = {k: v for k, v in item.items() if k in valid_keys}
                        # Ensure required fields
                        if "id" in filtered and "title" in filtered and "handle" in filtered:
                            products.append(ProductInfo(**filtered))
                    # Handle if item is object
                    elif hasattr(item, "id") and hasattr(item, "title"):
                        # Manually map variants if they exist in the object but weren't automatically mapped
                        variants = getattr(item, "variants", [])
                        # If variants are already ProductVariantInfo objects, use them.
                        # If they are dicts, convert them.
                        mapped_variants = []
                        for v in variants:
                            if isinstance(v, dict):
                                mapped_variants.append(v) # Pydantic will handle dict -> model
                            else:
                                mapped_variants.append(v)

                        products.append(ProductInfo(
                            id=item.id,
                            title=item.title,
                            handle=item.handle,
                            thumbnail=getattr(item, "thumbnail", None),
                            price=getattr(item, "price", None),
                            currency_code=getattr(item, "currency_code", None),
                            variants=mapped_variants
                        ))
                except Exception as e:
                    logger.warning(f"Failed to map product item: {e}")
        
        # Add product-specific quick replies if products found
        if products:
            # Add "Buy Now" button first
            quick_replies.insert(0, QuickReply(label="Mua ngay", value=f"Đặt hàng {products[0].title}"))
            # Add "View Details" button second
            quick_replies.insert(1, QuickReply(label="Xem chi tiết", value=f"Chi tiết {products[0].title}"))

        return ChatResponse(
            response=response_text,
            session_id=processed.session_id,
            products=products,
            quick_replies=quick_replies,
            metadata={"intent": intent.intent}
        )

    async def _generate_llm_response(self, processed, intent, tool_res) -> str:
        # Try template-based response first for common cases
        template_context = {}
        
        print(f"[ResponseGen] intent={intent.intent}, tool_res={'exists' if tool_res else 'None'}, tool_res.ok={tool_res.ok if tool_res else 'N/A'}, data_len={len(tool_res.data) if tool_res and tool_res.ok else 0}")
        
        # PRIORITY 1: Handle FAQ intents with templates (avoid LLM hallucination)
        if intent.intent in ["faq_shipping", "faq_payment", "faq_return", "faq_promo"]:
            print(f"[ResponseGen] Using FAQ template for {intent.intent}")
            return get_response_template(intent.intent)
        
        # PRIORITY 2: Handle thank_you/goodbye
        if intent.intent == "thank_you":
            return get_response_template("thank_you")
        elif intent.intent == "goodbye":
            return get_response_template("goodbye")
        
        if tool_res:
            if tool_res.ok and isinstance(tool_res.data, list):
                template_context['count'] = len(tool_res.data)
                if len(tool_res.data) > 0:
                    first_item = tool_res.data[0]
                    if isinstance(first_item, dict):
                        template_context['product'] = first_item.get('title', '')
                    elif hasattr(first_item, 'title'):
                        template_context['product'] = first_item.title
            else:
                # tool_res.ok = False means tool failed/no results
                template_context['count'] = 0
        
        # PRIORITY 3: Handle product intents with templates (even without tool_res)
        if intent.intent == "general" and (not tool_res or not tool_res.ok):
            print("[ResponseGen] Using greeting template")
            return get_response_template("greeting")
        elif intent.intent in ["product_inquiry", "product_detail"]:
            # Get product query from entities (may be set by executor)
            query = intent.entities.get("product_query")
            if query:
                template_context['query'] = query
            else:
                # Fallback: extract from processed text
                template_context['query'] = "sản phẩm bạn cần"
            
            if template_context.get('count', 0) > 0:
                print(f"[ResponseGen] Using product_found template, count={template_context['count']}")
                return get_response_template("product_found", template_context)
            else:
                # Product not found - use template to avoid LLM hallucination
                print(f"[ResponseGen] Using product_not_found template, query={template_context.get('query')}")
                return get_response_template("product_not_found", template_context)
        elif intent.intent == "product_recommend":
            if tool_res.ok and template_context.get('count', 0) > 0:
                return get_response_template("product_recommend")
            else:
                return "Xin lỗi, shop đang cập nhật sản phẩm. Bạn quay lại sau nhé!"
        elif intent.intent == "cart_add":
            if tool_res.ok:
                return get_response_template("cart_added", template_context)
            else:
                return "Xin lỗi, tôi cần biết sản phẩm nào bạn muốn thêm vào giỏ hàng."
        
        # PRIORITY 4: Fallback to LLM only for complex cases
        # But prevent hallucination by checking tool_res.ok
        if tool_res and not tool_res.ok:
            # Tool failed - don't let LLM make up data
            print(f"[ResponseGen] Tool failed, using error template")
            if intent.intent in ["product_inquiry", "product_detail"]:
                return get_response_template("product_not_found", template_context)
            return "Xin lỗi, tôi không tìm thấy thông tin bạn cần. Bạn có thể hỏi lại được không?"
        
        # Construct prompt for LLM (only when tool succeeded or no tool needed)
        print(f"[ResponseGen] Falling through to LLM generation")
        context_str = ""
        if tool_res:
            if tool_res.ok:
                # Truncate data if too large
                data_str = json.dumps(tool_res.data, ensure_ascii=False, default=str)
                if len(data_str) > 3000: # Increased limit for better context
                    data_str = data_str[:3000] + "...(truncated)"
                context_str = f"Tool Result: {data_str}"
            else:
                context_str = f"Tool Error: {tool_res.errors}"
        
        # Add currency hint if available in tool result (e.g. from product search)
        currency_hint = ""
        if tool_res and tool_res.ok and isinstance(tool_res.data, list) and len(tool_res.data) > 0:
            first_item = tool_res.data[0]
            if isinstance(first_item, dict) and "currency_code" in first_item:
                 currency_hint = f"\nLưu ý: Đơn vị tiền tệ đang sử dụng là {first_item.get('currency_code', 'N/A').upper()}."
            elif hasattr(first_item, "currency_code"):
                 currency_hint = f"\nLưu ý: Đơn vị tiền tệ đang sử dụng là {getattr(first_item, 'currency_code', 'N/A').upper()}."

        messages = [
            {"role": "system", "content": SYSTEM_PROMPT_VI + currency_hint},
            {"role": "user", "content": f"User Input: {processed.text}\nIntent: {intent.intent}\nContext: {context_str}"}
        ]
        
        if self.client:
            try:
                completion = await self.client.chat.completions.create(
                    model=GEMINI_MODEL,
                    messages=messages,
                    temperature=0.7
                )
                return completion.choices[0].message.content or ""
            except Exception as e:
                logger.error(f"LLM Error: {e}")
                return get_response_template("error_generic")
        else:
            return f"[MOCK LLM RESPONSE] Based on {intent.intent} and data: {tool_res.data if tool_res else 'None'}"

    def _generate_quick_replies(self, processed) -> List[QuickReply]:
        nodes = self.context_config.get("nodes", [])
        
        # Helper to find node by ID recursively
        def find_node(node_list, target_id):
            for node in node_list:
                if node.get("id") == target_id:
                    return node
                if "children" in node:
                    found = find_node(node["children"], target_id)
                    if found: return found
            return None

        # Determine current context node
        current_node = None
        if processed.tag:
            # If tag is like "action:view_cart", we might not find a node if it's a leaf.
            # But if tag is a group ID, we might.
            # Let's try to find by ID first (assuming tag might be ID)
            current_node = find_node(nodes, processed.tag)
        
        # If no specific node found, default to user root
        if not current_node:
            user_root_id = f"{processed.user_type}_root"
            current_node = find_node(nodes, user_root_id)

        if not current_node:
            return []

        options = []
        # Return children of the current node
        for child in current_node.get("children", []):
            # Determine value: prefer value, then tag, then id
            val = child.get("value") or child.get("tag") or child.get("id")
            options.append(QuickReply(
                label=child["label"],
                value=val,
                metadata={"type": child.get("type")}
            ))
            
        return options

In [ ]:
# --- DATA LOADING ---
# Upload 'multi_agent_test_cases_full.csv' to Colab files
try:
    df = pd.read_csv('multi_agent_test_cases_full.csv')
    print(f"Loaded {len(df)} test cases.")
except FileNotFoundError:
    print("File not found. Creating sample data...")
    data = {
        "id": [1, 2, 3],
        "user_input": ["Xin chào shop", "Tìm balo JanSport màu đen", "Kiểm tra đơn hàng #123"],
        "expected_intent_classifier_intent": ["general", "product_inquiry", "order_tracking"],
        "expected_executor_tools_called": ["", "product.search", "order.track"]
    }
    df = pd.DataFrame(data)
    print("Created sample data.")

df.head()

In [ ]:
# --- EVALUATION LOOP ---

async def evaluate_row(row):
    # Initialize Agents
    input_processor = InputProcessor()
    intent_classifier = IntentClassifier()
    orchestrator = Orchestrator()
    executor = Executor()
    response_generator = ResponseGenerator()

    # 1. Input
    req = ChatRequest(session_id=f"sess_{row['id']}", message=row['user_input'])
    processed = await input_processor.run(req)
    
    # 2. Intent
    intent_res = await intent_classifier.run(processed)
    
    # 3. Orchestrator
    plan, _ = await orchestrator.run(processed, intent_res)
    
    # 4. Executor
    tool_res = await executor.run(processed, intent_res, plan)
    
    # 5. Response
    response = await response_generator.run(processed, intent_res, plan, tool_res)
    
    # --- CHECKS ---
    results = {
        "id": row['id'],
        "input": row['user_input'],
        "intent_actual": intent_res.intent,
        "intent_expected": row.get('expected_intent_classifier_intent'),
        "intent_match": intent_res.intent == row.get('expected_intent_classifier_intent'),
        
        "tools_actual": ",".join(plan.tools),
        "tools_expected": row.get('expected_executor_tools_called', ""),
        # Simple check: is expected tool in actual tools?
        "tools_match": str(row.get('expected_executor_tools_called', "")) in ",".join(plan.tools) if pd.notna(row.get('expected_executor_tools_called')) else True,
        
        "response_preview": response.response[:50] + "..."
    }
    return results

# Run Evaluation
results_list = []
print("Starting evaluation...")
for index, row in df.iterrows():
    res = await evaluate_row(row)
    results_list.append(res)
    print(f"Row {row['id']}: Intent Match={res['intent_match']}, Tools Match={res['tools_match']}")

results_df = pd.DataFrame(results_list)

In [ ]:
# --- SUMMARY REPORT ---
print("\n--- EVALUATION SUMMARY ---")
total = len(results_df)
intent_acc = results_df['intent_match'].mean() * 100
tools_acc = results_df['tools_match'].mean() * 100

print(f"Total Test Cases: {total}")
print(f"Intent Accuracy: {intent_acc:.2f}%")
print(f"Tool Selection Accuracy: {tools_acc:.2f}%")

print("\n--- FAILED CASES ---")
failed = results_df[~results_df['intent_match'] | ~results_df['tools_match']]
if not failed.empty:
    display(failed[['id', 'input', 'intent_actual', 'intent_expected', 'tools_actual', 'tools_expected']])
else:
    print("All cases passed!")

In [ ]:
!pip install pydantic openai pandas python-dotenv nest_asyncio pyyaml

In [ ]:
import os
import json
import re
import yaml
import asyncio
import logging
import pandas as pd
import nest_asyncio
from typing import List, Dict, Any, Optional, Tuple
from pydantic import BaseModel, Field
from datetime import datetime
from enum import Enum

# Apply nest_asyncio to allow async loops in notebook
nest_asyncio.apply()

# Configure Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("Evaluation")

In [ ]:
# --- CONFIGURATION ---
# Replace with your actual API Key
GOOGLE_API_KEY = "YOUR_API_KEY_HERE" 
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
GEMINI_MODEL = "gemini-1.5-flash"

SYSTEM_PROMPT_VI = """
Bạn là trợ lý ảo AI chuyên nghiệp của cửa hàng JanSport Việt Nam.
Nhiệm vụ của bạn là hỗ trợ khách hàng tìm kiếm sản phẩm, kiểm tra đơn hàng và giải đáp thắc mắc.
Luôn trả lời bằng tiếng Việt, giọng điệu thân thiện, nhiệt tình và chuyên nghiệp.
"""

# Mock Context Config (from context_config.yaml)
CONTEXT_CONFIG = {
    "nodes": [
        {
            "id": "guest_root",
            "label": "Guest Root",
            "children": [
                {"label": "Tìm sản phẩm", "value": "action:product_search"},
                {"label": "Tra cứu đơn hàng", "value": "action:order_track"}
            ]
        },
        {
            "id": "customer_root",
            "label": "Customer Root",
            "children": [
                {"label": "Đơn hàng của tôi", "value": "action:order_list"},
                {"label": "Giỏ hàng", "value": "action:view_cart"}
            ]
        }
    ]
}

In [ ]:
# --- DATA MODELS ---
# Combined from app/models/agent_types.py and app/models/api_models.py

class SessionContext(BaseModel):
    last_messages: List[Dict[str, Any]] = Field(default_factory=list)
    cart_id: Optional[str] = None
    last_product_ids: List[str] = Field(default_factory=list)

class ProcessedInput(BaseModel):
    session_id: str
    customer_id: Optional[str] = None
    text: str
    cleaned_text: str
    language: str = "vi"  # "vi" | "en"
    user_type: str = "guest"  # "guest" | "customer"
    tag: Optional[str] = None
    session_ctx: SessionContext = Field(default_factory=SessionContext)

class IntentResult(BaseModel):
    intent: str
    sub_intent: Optional[str] = None
    confidence: float = 0.0
    entities: Dict[str, Any] = Field(default_factory=dict)

class ActionPlan(BaseModel):
    tools: List[str] = Field(default_factory=list)
    required_data: List[str] = Field(default_factory=list)
    next_step: Optional[str] = None
    escalate: bool = False

class ToolResults(BaseModel):
    ok: bool
    data: Any = None
    errors: List[str] = Field(default_factory=list)
    timings_ms: Dict[str, int] = Field(default_factory=dict)

# From api_models.py
class ChatActionType(str, Enum):
    NAVIGATE = "navigate"
    API_CALL = "api_call"
    FORM = "form"
    SHOW_PRODUCTS = "show_products"
    REQUEST_ORDER_INFO = "request_order_info"
    GUIDE_TO_CART = "guide_to_cart"
    NONE = "none"

class ChatAction(BaseModel):
    type: ChatActionType = ChatActionType.NONE
    payload: Dict[str, Any] = Field(default_factory=dict)

class QuickReply(BaseModel):
    label: str
    value: str
    metadata: Optional[Dict[str, Any]] = None

class ProductVariantInfo(BaseModel):
    id: str
    title: str
    price: Optional[str] = None
    currency_code: Optional[str] = None

class ProductInfo(BaseModel):
    id: str
    title: str
    handle: str
    thumbnail: Optional[str] = None
    price: Optional[str] = None
    currency_code: Optional[str] = None
    variants: List[ProductVariantInfo] = Field(default_factory=list)
    
class ChatRequest(BaseModel):
    message: str
    session_id: str
    customer_id: Optional[str] = None
    tag: Optional[str] = None
    language: Optional[str] = None
    metadata: Dict[str, Any] = Field(default_factory=dict)

class ChatResponse(BaseModel):
    response: str
    session_id: str
    products: List[ProductInfo] = Field(default_factory=list)
    quick_replies: List[QuickReply] = Field(default_factory=list)
    metadata: Dict[str, Any] = Field(default_factory=dict)

In [ ]:
# --- RESPONSE TEMPLATES (app/agents/response_templates.py) ---
# Copied exactly from source

RESPONSE_TEMPLATES = {
    # Greetings
    "greeting": [
        "Xin chào! Tôi là trợ lý ảo của shop. Tôi có thể giúp gì cho bạn hôm nay? 😊",
        "Chào bạn! Rất vui được hỗ trợ bạn. Bạn cần tìm sản phẩm gì không?",
        "Hello! Chào mừng bạn đến với shop. Tôi có thể tư vấn gì cho bạn?"
    ],
    
    # Product Search Success
    "product_found": [
        "Mình tìm thấy {count} sản phẩm phù hợp với '{query}' ạ:",
        "Đây là {count} sản phẩm mình tìm được cho bạn về '{query}':",
        "Shop có {count} sản phẩm '{query}' như sau:"
    ],
    
    # Product Not Found
    "product_not_found": [
        "Rất tiếc, shop không tìm thấy sản phẩm '{query}' bạn cần 😔\nBạn có thể thử tìm kiếm sản phẩm tương tự không?",
        "Hiện tại shop chưa có sản phẩm '{query}' ạ. Bạn muốn xem sản phẩm khác không?",
        "Mình không tìm thấy '{query}' trong kho. Bạn có thể mô tả chi tiết hơn hoặc xem sản phẩm tương tự?"
    ],
    
    # Product Recommend
    "product_recommend": [
        "🌟 Đây là một số sản phẩm hot mà bạn có thể thích:",
        "Mình gợi ý cho bạn vài sản phẩm đang bán chạy nha:",
        "Đây là top sản phẩm được yêu thích nhất của shop:"
    ],
    
    # Cart Actions
    "cart_added": [
        "Đã thêm {product} vào giỏ hàng! Bạn muốn tiếp tục mua sắm hay thanh toán luôn?",
        "OK! Đã cho {product} vào giỏ rồi nhé. Mua thêm gì nữa không?"
    ],
    
    "cart_view_empty": [
        "Giỏ hàng của bạn đang trống. Tìm sản phẩm ngay nhé! 🛍️",
        "Bạn chưa có sản phẩm nào trong giỏ. Cùng khám phá shop nhé!"
    ],
    
    "cart_view_has_items": [
        "Giỏ hàng của bạn có {count} sản phẩm:",
        "Bạn đang có {count} món trong giỏ:"
    ],
    
    # Order Tracking
    "order_found": [
        "Đơn hàng #{order_id} của bạn:\nTrạng thái: {status}\nDự kiến giao: {delivery_date}",
        "Mình check đơn #{order_id} rồi:\n📦 {status}\n🚚 Giao dự kiến: {delivery_date}"
    ],
    
    "order_not_found": [
        "Không tìm thấy đơn hàng #{order_id}. Bạn kiểm tra lại mã đơn giúp mình nhé!",
        "Mình không thấy đơn #{order_id} trong hệ thống. Bạn có chắc mã đúng không?"
    ],
    
    # FAQ
    "faq_shipping": [
        "🚚 Thông tin vận chuyển:\n- Nội thành Hà Nội/HCM: 2-3 ngày, phí 30.000₫\n- Ngoại thành: 3-5 ngày, phí 50.000₫\n- Miễn phí ship cho đơn hàng trên 500.000₫\n- Hỗ trợ COD toàn quốc",
        "Shop ship nhanh trong 2-3 ngày nội thành nha! Phí 30k, FREE SHIP cho đơn trên 500k. Bạn có thể thanh toán COD khi nhận hàng ạ!",
        "Vận chuyển:\n• Giao hàng toàn quốc\n• Nội thành: 2-3 ngày (30k)\n• Ngoại thành: 3-5 ngày (50k)\n• FREE SHIP đơn > 500k\n• Hỗ trợ COD"
    ],
    
    "faq_return": [
        "🔄 Chính sách đổi trả:\n- Đổi trả miễn phí trong 7 ngày\n- Sản phẩm chưa sử dụng, còn nguyên tem mác\n- Miễn phí đổi size/màu (nếu còn hàng)\n- Hoàn tiền trong 3-5 ngày làm việc",
        "Bạn được đổi trả miễn phí trong 7 ngày nếu sản phẩm còn nguyên tem mác nha! Đổi size/màu miễn phí luôn ạ."
    ],
    
    "faq_payment": [
        "💳 Các hình thức thanh toán:\n- COD (tiền mặt khi nhận hàng)\n- Chuyển khoản\n- Thẻ tín dụng/ghi nợ\n- Ví điện tử (Momo, ZaloPay)",
        "Bạn có thể thanh toán bằng COD, chuyển khoản, thẻ, hoặc ví Momo/ZaloPay nha!"
    ],
    
    "faq_promo": [
        "🎁 Khuyến mãi hiện tại:\n- Giảm 10% đơn đầu tiên\n- Free ship đơn >500k\n- Tích điểm đổi quà",
        "Bạn mới có ưu đãi giảm 10% đơn đầu + freeship đơn >500k đó! 🎉"
    ],
    
    # Thank you & Goodbye
    "thank_you": [
        "Không có gì! Rất vui được hỗ trợ bạn. Cần gì cứ gọi mình nhé! 😊",
        "Hehe, có gì đâu! Cần gì cứ nhắn mình nha!",
        "Luôn sẵn sàng giúp bạn! Mua sắm vui vẻ nhé! 🛍️"
    ],
    
    "goodbye": [
        "Tạm biệt! Hẹn gặp lại bạn sớm. Chúc bạn một ngày tốt lành! 👋",
        "Bye bye! Nhớ ghé shop thường xuyên nha! 😊",
        "Hẹn gặp lại! Mua sắm vui vẻ nhé! 🛍️"
    ],
    
    # Errors
    "error_generic": [
        "Xin lỗi, có lỗi xảy ra. Bạn có thể thử lại không?",
        "Ối, có gì đó không ổn. Mình thử lại nhé!"
    ],
    
    "error_no_products": [
        "Shop đang cập nhật sản phẩm. Bạn quay lại sau nhé!",
        "Hiện tại sản phẩm đang hết hàng. Mình sẽ thông báo khi có hàng!"
    ],
    
    # Staff/Manager
    "staff_success": [
        "Đã xử lý xong! Kết quả:",
        "Done! Thông tin bạn cần:"
    ],
    
    "manager_report": [
        "📊 Báo cáo {report_type}:\n{data}",
        "Đây là thống kê {report_type} bạn yêu cầu:\n{data}"
    ],
    
    # Clarification needed
    "need_clarification": [
        "Mình chưa hiểu rõ lắm. Bạn có thể nói rõ hơn được không?",
        "Hmm, bạn muốn tìm gì nhỉ? Mô tả chi tiết giúp mình nha!",
        "Bạn cần tìm sản phẩm gì? Cho mình biết thêm thông tin nhé!"
    ],
    
    "ask_product_details": [
        "Bạn muốn tìm loại nào? (màu, size, giá...)",
        "Cho mình biết thêm chi tiết: màu sắc, kích thước, giá cả?"
    ],
    
    "ask_order_id": [
        "Bạn cho mình mã đơn hàng để check nhé! (VD: #1234 hoặc ORD-5678)",
        "Mã đơn hàng của bạn là gì ạ?"
    ]
}

def get_response_template(intent: str, context: Optional[dict] = None) -> str:
    """Get appropriate response template based on intent and context"""
    import random
    
    context = context or {}
    templates = RESPONSE_TEMPLATES.get(intent, RESPONSE_TEMPLATES["need_clarification"])
    
    # Select random template for variety
    template = random.choice(templates)
    
    # Format with context if available
    try:
        if context:
            template = template.format(**context)
    except KeyError:
        pass  # Use template as-is if formatting fails
    
    return template

In [ ]:
# --- MOCKS FOR EXTERNAL DEPENDENCIES ---

# Mock Tools to simulate Medusa/Redis calls
class MockTools:
    @staticmethod
    async def search_products(query, limit=5):
        return ToolResults(ok=True, data=[
            {"id": "p_1", "title": f"JanSport SuperBreak - {query}", "handle": "superbreak", "price": "500000", "currency_code": "vnd"},
            {"id": "p_2", "title": f"JanSport Big Student - {query}", "handle": "big-student", "price": "800000", "currency_code": "vnd"}
        ])
    
    @staticmethod
    async def get_product_details(product_id):
        return ToolResults(ok=True, data={"id": product_id, "title": "JanSport Detail", "description": "Great bag", "price": "500000", "currency_code": "vnd"})

    @staticmethod
    async def recommend_products(limit=5):
        return ToolResults(ok=True, data=[{"id": "rec_1", "title": "Trending Bag", "handle": "trending", "price": "600000", "currency_code": "vnd"}])

    @staticmethod
    async def view_cart(cart_id):
        return ToolResults(ok=True, data={"id": cart_id, "items": []})

    @staticmethod
    async def add_to_cart(cart_id, variant_id, qty):
        return ToolResults(ok=True, data={"cart_id": cart_id, "added": True})
        
    @staticmethod
    async def add_to_cart_smart(cart_id, query, qty):
        return ToolResults(ok=True, data={"cart_id": cart_id, "added": True, "product": query})

    @staticmethod
    async def lookup_order(order_id):
        if order_id == "123":
            return ToolResults(ok=True, data={"id": "123", "status": "shipped", "delivery_date": "2023-12-25"})
        return ToolResults(ok=False, errors=["Order not found"])
    
    @staticmethod
    async def list_orders(customer_id):
        return ToolResults(ok=True, data=[{"id": "123", "status": "shipped"}])

    # Staff/Manager Mocks
    @staticmethod
    async def check_stock(query): return ToolResults(ok=True, data={"product": query, "stock": 10})
    @staticmethod
    async def check_price(query): return ToolResults(ok=True, data={"product": query, "price": 500000})
    @staticmethod
    async def lookup_customer(query): return ToolResults(ok=True, data={"id": "cust_1", "name": "Test User"})
    @staticmethod
    async def get_customer_order_history(cid): return ToolResults(ok=True, data=[])
    @staticmethod
    async def create_draft_order(cid, items): return ToolResults(ok=True, data={"id": "draft_1"})
    @staticmethod
    async def update_order_status(oid, status): return ToolResults(ok=True, data={"id": oid, "status": status})
    @staticmethod
    async def print_shipping_label(oid): return ToolResults(ok=True, data={"url": "http://label.pdf"})
    @staticmethod
    async def get_sales_report(): return ToolResults(ok=True, data={"total_sales": 10000000})
    @staticmethod
    async def get_top_products(): return ToolResults(ok=True, data=[{"title": "Bag A", "sales": 100}])
    @staticmethod
    async def get_customer_analytics(): return ToolResults(ok=True, data={"new_users": 50})

# Mock Specialized Agents (to avoid importing their complex logic)
class MockSpecializedAgent:
    def __init__(self, name): self.name = name
    def can_handle(self, intent): return False 
    async def process(self, processed, intent): 
        # Simple pass-through for testing
        return ActionPlan(tools=[], next_step=f"mock_{self.name}"), None

# Define tool objects for Executor to use
cart_tools = MockTools
product_tools = MockTools
product_recommend_tool = MockTools
order_tools = MockTools
staff_tools = MockTools
report_tools = MockTools
system_tools = MockTools
customer_tools = MockTools

In [ ]:
# --- AGENT 1: INPUT PROCESSOR (app/agents/input_processor.py) ---
# from typing import Optional, Dict, Any, List
# import re
# import asyncpg
# from app.models import ChatRequest
# from app.models.agent_types import ProcessedInput, SessionContext

def _normalize_text(text: str) -> str:
    # Trim, collapse spaces; keep diacritics (for VI) and punctuation minimally
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text


def _detect_language_heuristic(text: str) -> str:
    t = text.lower()
    # Very lightweight heuristic: VI keywords/diacritics vs common EN markers
    if any(ch in t for ch in "ăâêôơưđáàảãạấầẩẫậắằẳẵặéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ"):
        return "vi"
    if any(word in t for word in [" the ", " and ", " price ", " order ", " product ", " hello "]):
        return "en"
    # Fallback: prefer VI as primary market
    return "vi"


async def _fetch_session_context(pool: Any, session_id: str) -> SessionContext:
    # MOCK IMPLEMENTATION
    ctx = SessionContext()
    # Simulate some context if needed
    return ctx


class InputProcessor:
    async def run(self, req: ChatRequest, pool: Any = None) -> ProcessedInput:
        print(f"[InputProcessor] Processing request: session={req.session_id}, msg='{req.message}'")
        cleaned = _normalize_text(req.message)
        lang = req.language or _detect_language_heuristic(cleaned)
        
        # Determine user_type:
        user_type = "guest"
        if req.metadata and "user_type" in req.metadata:
            user_type = req.metadata["user_type"]
        elif req.customer_id:
            user_type = "customer"
            
        session_ctx = await _fetch_session_context(pool, req.session_id)
        print(f"[InputProcessor] Context loaded: last_intent={session_ctx.last_messages[0]['intent'] if session_ctx.last_messages else 'None'}")

        return ProcessedInput(
            session_id=req.session_id,
            customer_id=req.customer_id,
            text=req.message,
            cleaned_text=cleaned,
            language=lang,
            user_type=user_type,
            tag=req.tag,
            session_ctx=session_ctx,
        )

In [ ]:
# --- AGENT 2: INTENT CLASSIFIER (app/agents/intent_classifier.py) ---
# from typing import Dict, List
# import re
# from app.models.agent_types import ProcessedInput, IntentResult

VI_KEYWORDS: Dict[str, List[str]] = {
    "GREETING": ["xin chào", "chào", "alo", "chào bạn", "hello", "hi", "hey", "chào buổi", "chào admin", "chào shop"],
    "PRODUCT.SEARCH": [
        "tìm", "tìm kiếm", "có không", "còn không", "có bán", "muốn mua", "giá", "sản phẩm", 
        "mua", "đặt", "kiếm", "tương tự", "giống", "cần", "shop có", "bên này có", "có hàng",
        "còn hàng", "hết hàng chưa", "còn size", "có màu", "loại", "dòng", "model"
    ],
    "PRODUCT.DETAIL": [
        "chi tiết", "thông số", "xem kỹ", "thông tin", "cụ thể", "mô tả", "spec", 
        "đặc điểm", "tính năng", "chất liệu", "kích thước", "size", "màu sắc", "xuất xứ"
    ],
    "PRODUCT.RECOMMEND": [
        "gợi ý cho tôi", "gợi ý cho mình", "gợi ý vài", "nên mua gì", "mua gì", 
        "hot trend", "bán chạy", "phổ biến", "đề xuất", "recommend me",
        "trending", "mới nhất", "best seller", "top sản phẩm", "sản phẩm tốt", "sản phẩm hay", "đáng mua nhất",
        "xem sản phẩm nào", "có sản phẩm nào", "sản phẩm nào tốt"
    ],
    "ORDER.TRACK": [
        "tra cứu", "kiểm tra", "đơn hàng", "ở đâu", "tình trạng", "tracking", "giao hàng",
        "ship", "vận chuyển", "đã giao chưa", "khi nào nhận", "bao giờ về", "đến chưa"
    ],
    "ORDER.CANCEL": ["hủy đơn", "không mua nữa", "huỷ đơn", "cancel", "không muốn", "đổi ý", "không lấy"],
    "ORDER.RETURN": [
        "đổi trả", "hoàn tiền", "bảo hành", "trả hàng", "return", "refund", "warranty",
        "đổi hàng", "lỗi", "hư", "không vừa", "sai", "không đúng"
    ],
    "CART.VIEW": ["xem giỏ", "giỏ hàng", "trong giỏ", "cart", "giỏ", "đã chọn", "đã thêm"],
    "CART.ADD": [
        "thêm vào giỏ", "bỏ vào giỏ", "mua cái này", "lấy cái này", "đặt cái này", 
        "lấy cho", "thêm 1", "đặt 1", "đặt hàng", "mua ngay", "chốt", "order",
        "cho vào giỏ", "bỏ giỏ", "add", "thêm", "bỏ", "cho tôi", "cho em",
        "mình lấy", "mình mua", "lấy luôn", "mua luôn"
    ],
    "CART.REMOVE": ["xóa khỏi giỏ", "bỏ ra", "xóa sản phẩm", "bỏ sản phẩm", "remove", "không lấy", "bỏ"],
    "CART.UPDATE": ["đổi số lượng", "thay đổi", "sửa giỏ", "update cart", "tăng", "giảm", "đổi size", "đổi màu"],
    "CHECKOUT": ["thanh toán", "checkout", "đặt luôn", "mua luôn", "pay", "trả tiền", "hoàn tất"],
    "ACCOUNT.LOGIN": ["đăng nhập", "login", "sign in", "vào tài khoản"],
    "ACCOUNT.REGISTER": ["đăng ký", "tạo tài khoản", "sign up", "register", "mở tài khoản"],
    "FAQ.SHIPPING": [
        "phí ship", "vận chuyển", "bao lâu", "giao hàng", "ship", "shipping",
        "delivery", "mất bao lâu", "khi nào nhận", "ship cod", "ship nhanh"
    ],
    "FAQ.PAYMENT": [
        "thanh toán", "chuyển khoản", "cod", "trả tiền", "payment", 
        "hình thức thanh toán", "trả góp", "ví", "momo", "zalopay", "banking"
    ],
    "FAQ.PROMO": ["khuyến mãi", "giảm giá", "sale", "voucher", "mã giảm", "coupon", "promotion", "ưu đãi"],
    "FAQ.RETURN": [
        "đổi trả", "chính sách đổi", "return policy", "hoàn tiền", "refund",
        "bảo hành", "warranty", "trả hàng", "đổi hàng", "chính sách hoàn"
    ],
    "SUPPORT.ESCALATE": [
        "nhân viên", "người thật", "không phải bot", "tư vấn", "hỗ trợ", 
        "admin", "staff", "human", "real person", "agent"
    ],
    "THANK": ["cảm ơn", "thanks", "thank you", "cám ơn", "ơn"],
    "GOODBYE": ["tạm biệt", "bye", "goodbye", "hẹn gặp lại", "đi đây"],
    # Staff / Manager Keywords
    "STAFF.CHECK_STOCK": ["check kho", "tồn kho", "kiểm kho", "số lượng tồn", "inventory", "stock"],
    "STAFF.CUSTOMER_LOOKUP": ["tìm khách", "check info khách", "thông tin khách", "lookup customer"],
    "STAFF.ORDER_HISTORY": ["lịch sử mua", "đơn cũ của khách", "khách mua gì", "order history"],
    "STAFF.CREATE_ORDER": ["tạo đơn giúp", "lên đơn", "tạo đơn", "create order", "draft order"],
    "MANAGER.REPORT_SALES": ["doanh thu", "báo cáo bán hàng", "doanh số", "sales report", "revenue"],
    "MANAGER.REPORT_CHATBOT": ["hiệu quả bot", "bot chat bao nhiêu", "thống kê bot", "chatbot stats"],
    "MANAGER.CONFIG_UPDATE": ["tắt bot", "bật bot", "chỉnh prompt", "cấu hình", "config"],
}

EN_KEYWORDS: Dict[str, List[str]] = {
    "GREETING": ["hello", "hi", "hey"],
    "PRODUCT.SEARCH": ["search", "find", "looking for", "do you have", "want to buy", "price", "product", "buy", "order"],
    "PRODUCT.DETAIL": ["detail", "specs", "info", "more info", "specification"],
    "PRODUCT.RECOMMEND": ["recommend", "suggest", "hot trend", "best seller", "popular"],
    "ORDER.TRACK": ["track", "check order", "order status", "where is", "shipping"],
    "ORDER.CANCEL": ["cancel order", "cancel"],
    "ORDER.RETURN": ["return", "refund", "warranty"],
    "CART.VIEW": ["view cart", "my cart", "shopping cart"],
    "CART.ADD": ["add to cart", "buy this", "get this", "order this"],
    "CART.REMOVE": ["remove", "delete item", "remove from cart"],
    "ACCOUNT.LOGIN": ["login", "sign in"],
    "ACCOUNT.REGISTER": ["register", "sign up"],
    "FAQ.SHIPPING": ["shipping cost", "delivery time", "shipping fee"],
    "FAQ.PAYMENT": ["payment", "cod", "bank transfer"],
    "SUPPORT.ESCALATE": ["human", "staff", "real person", "agent"],
    # Staff / Manager Keywords
    "STAFF.CHECK_STOCK": ["check stock", "inventory", "stock level"],
    "STAFF.CUSTOMER_LOOKUP": ["lookup customer", "find customer", "user info"],
    "STAFF.ORDER_HISTORY": ["customer order history", "order history"],
    "STAFF.CREATE_ORDER": ["create order", "draft order"],
    "MANAGER.REPORT_SALES": ["sales report", "revenue", "sales stats"],
    "MANAGER.REPORT_CHATBOT": ["chatbot stats", "bot performance"],
    "MANAGER.CONFIG_UPDATE": ["disable bot", "enable bot", "update config"],
}


def _match_score(text: str, phrases: List[str]) -> int:
    t = f" {text.lower()} "
    score = 0
    for p in phrases:
        if f" {p} " in t:
            score += 2 if len(p.split()) > 1 else 1
    return score


def _extract_product_query(cleaned_text: str) -> str:
    s = cleaned_text
    
    # Check if this is a context reference (not a search query)
    context_patterns = [
        r"\b(cái|sản phẩm|sp)\s+(này|kia|đó|ấy|vừa|trước)",
        r"\b(đầu tiên|thứ nhất|thứ hai|cuối cùng)",
        r"\bnó\b",
        r"\bchi tiết\b.*(?!\w+)",  # "chi tiết" without specific product name
    ]
    for pattern in context_patterns:
        if re.search(pattern, s.lower()):
            return ""  # Return empty to signal this needs context
    
    s = re.sub(r"[?,.]", " ", s) # Remove punctuation
    # remove common Vietnamese/English query anchors
    remove = [
        "tìm", "tìm kiếm", "có không", "còn không", "có bán", "muốn mua",
        "search", "find", "looking for", "do you have", "want to buy",
        "sản phẩm", "product", "mua", "đặt", "kiếm", "cho tôi", "cho anh", "cho em", "cho mình",
        "tương tự", "similar",
        "tôi", "anh", "em", "bạn", "mình", "cái", "chiếc", "là", "của",
        "giá", "bao nhiêu", "price", "cost", "how much", "tiền",
        "thông tin", "xem", "về", "info", "about",
        # Staff keywords
        "check kho", "tồn kho", "kiểm kho", "số lượng tồn",
        "check stock", "inventory", "stock level",
        "tìm khách", "check info khách", "thông tin khách", "khách hàng", "khách",
        "lookup customer", "find customer", "user info",
        "nhân viên", "quản lý", "staff", "manager"
    ]
    low = s.lower()
    for r in remove:
        low = low.replace(r, " ")
    low = re.sub(r"\s+", " ", low).strip()
    return low


class IntentClassifier:
    async def run(self, processed: ProcessedInput) -> IntentResult:
        print(f"[IntentClassifier] Classifying: '{processed.cleaned_text}'")
        text = processed.cleaned_text
        lang = processed.language or "vi"

        # Choose keyword table
        table = VI_KEYWORDS if lang == "vi" else EN_KEYWORDS

        # Compute scores
        scores: Dict[str, int] = {}
        for key, phrases in table.items():
            scores[key] = _match_score(text, phrases)
        
        # Boost PRODUCT.RECOMMEND score if recommend phrases are detected
        # This prevents "gợi ý sản phẩm" from being classified as product_inquiry
        recommend_boost_phrases = ["gợi ý cho", "đề xuất cho", "recommend me", "gợi ý vài", "gợi ý một", "nên mua gì"]
        if any(phrase in text.lower() for phrase in recommend_boost_phrases):
            if "PRODUCT.RECOMMEND" in scores:
                scores["PRODUCT.RECOMMEND"] += 5  # Strong boost to override PRODUCT.SEARCH
        
        # Boost PRODUCT.DETAIL for context reference patterns (e.g., "sản phẩm đầu tiên", "cái này")
        context_patterns = [
            r'\b(cái|sản phẩm|sp)\s+(này|kia|đó|ấy|vừa|trước)',
            r'\b(đầu tiên|thứ nhất|thứ hai|thứ ba|cuối cùng)',
            r'\bnó\b',
        ]
        for pattern in context_patterns:
            if re.search(pattern, text.lower()):
                if "PRODUCT.DETAIL" in scores:
                    scores["PRODUCT.DETAIL"] += 10  # Very strong boost to override PRODUCT.SEARCH
                break

        # Get top 3 intents for multi-intent detection
        sorted_scores = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
        top_key = sorted_scores[0][0] if sorted_scores else "UNKNOWN"
        top_score = sorted_scores[0][1] if sorted_scores else 0
        
        # Detect secondary intent (e.g., "tìm balo giá rẻ" = PRODUCT.SEARCH + PRODUCT.PRICE)
        secondary_intent = None
        if len(sorted_scores) > 1 and sorted_scores[1][1] >= 2:
            secondary_intent = sorted_scores[1][0]
        
        print(f"[IntentClassifier] Top: {top_key}({top_score}), Secondary: {secondary_intent}")

        # Map to system intents used by backend
        intent_map = {
            "GREETING": "general",
            "PRODUCT.SEARCH": "product_inquiry",
            "PRODUCT.DETAIL": "product_detail",
            "PRODUCT.RECOMMEND": "product_recommend",
            "ORDER.TRACK": "order_tracking",
            "ORDER.CANCEL": "order_cancel",
            "ORDER.RETURN": "order_return",
            "CART.VIEW": "cart_view",
            "CART.ADD": "cart_add",
            "CART.REMOVE": "cart_remove",
            "CART.UPDATE": "cart_update",
            "CHECKOUT": "checkout",
            "ACCOUNT.LOGIN": "account_login",
            "ACCOUNT.REGISTER": "account_register",
            "FAQ.SHIPPING": "faq_shipping",
            "FAQ.PAYMENT": "faq_payment",
            "FAQ.PROMO": "faq_promo",
            "FAQ.RETURN": "faq_return",
            "SUPPORT.ESCALATE": "human_escalation",
            "THANK": "thank_you",
            "GOODBYE": "goodbye",
            "STAFF.CHECK_STOCK": "staff_check_stock",
            "STAFF.CUSTOMER_LOOKUP": "staff_customer_lookup",
            "STAFF.ORDER_HISTORY": "staff_order_history",
            "STAFF.CREATE_ORDER": "staff_create_order",
            "MANAGER.REPORT_SALES": "manager_report_sales",
            "MANAGER.REPORT_CHATBOT": "manager_report_chatbot",
            "MANAGER.CONFIG_UPDATE": "manager_config_update",
        }

        mapped = intent_map.get(top_key, "general")
        sub_intent = intent_map.get(secondary_intent) if secondary_intent else None

        # Enhanced confidence scoring
        if top_score >= 4:
            conf = 0.95
        elif top_score >= 3:
            conf = 0.85
        elif top_score >= 2:
            conf = 0.70
        elif top_score >= 1:
            conf = 0.50
        else:
            conf = 0.30

        entities = {}
        
        # Enhanced entity extraction
        if mapped == "product_inquiry" or mapped == "cart_add" or mapped == "product_detail":
            q = _extract_product_query(text)
            # For product_detail, empty query means context reference
            if mapped == "product_detail" and q == "":
                entities["product_query"] = ""  # Signal to use context
            elif q and len(q) >= 2:
                entities["product_query"] = q
            
            # Extract quantity
            qty_match = re.search(r'(\d+)\s*(cái|chiếc|con|bộ|món|sản phẩm|sp)', text.lower())
            if qty_match:
                entities["quantity"] = int(qty_match.group(1))
            else:
                entities["quantity"] = 1
                
        elif mapped == "order_tracking":
            # Enhanced order ID extraction
            # Patterns: #1234, order_1234, ORD-1234, đơn 1234
            match = re.search(r'(order[_\s-]?[\w\d]+|#\d+|ord[_\s-]?\d+|đơn[\s]?\d+|\d{4,})', text.lower())
            if match:
                order_id = match.group(0).replace("#", "").replace("đơn", "").replace("order", "").replace("_", "").replace("-", "").strip()
                entities["order_id"] = order_id
                
        elif mapped == "staff_check_stock":
            q = _extract_product_query(text)
            if q:
                entities["product_query"] = q
                
        elif mapped == "staff_customer_lookup":
            # Extract customer info (email, phone, name)
            email_match = re.search(r'[\w\.-]+@[\w\.-]+\.\w+', text)
            phone_match = re.search(r'(\+?84|0)\d{9,10}', text)
            
            if email_match:
                entities["customer_email"] = email_match.group(0)
            elif phone_match:
                entities["customer_phone"] = phone_match.group(0)
            else:
                # Fallback to query extraction
                q = _extract_product_query(text)
                if q:
                    entities["customer_query"] = q
        
        print(f"[IntentClassifier] Result: intent={mapped}, sub={sub_intent}, entities={entities}")
        return IntentResult(
            intent=mapped,
            sub_intent=sub_intent,
            confidence=conf,
            entities=entities,
        )

In [ ]:
# --- AGENT 3: ORCHESTRATOR (app/agents/orchestrator.py) ---
# from typing import Optional, Tuple, Dict
# from app.models.agent_types import ActionPlan, ToolResults, ProcessedInput, IntentResult
# from app.agents.specialized.sales_agent import SalesAgent
# from app.agents.specialized.order_agent import OrderAgent
# from app.agents.specialized.staff_agent import StaffAgent
# from app.agents.specialized.manager_agent import ManagerAgent

class Orchestrator:
    def __init__(self):
        self.agents = {
            "sales": MockSpecializedAgent("sales"),
            "order": MockSpecializedAgent("order"),
            "staff": MockSpecializedAgent("staff"),
            "manager": MockSpecializedAgent("manager")
        }

    async def run(
        self,
        processed: ProcessedInput,
        intent: IntentResult,
    ) -> Tuple[ActionPlan, Optional[ToolResults]]:
        """
        Hybrid Orchestrator:
        1. Executable Tag -> Execute Tool directly
        2. Scope Tag -> Route to Specialized Agent (Scoped NLU)
        3. No Tag -> Global NLU -> Route to Agent
        """
        print(f"[Orchestrator] Routing... Tag={processed.tag}, Intent={intent.intent}")
        
        # 1. Executable Tag (e.g., "action:view_cart")
        if processed.tag and processed.tag.startswith("action:"):
            action = processed.tag.split(":", 1)[1]
            print(f"[Orchestrator] Direct action: {action}")
            
            if action == "view_cart":
                return ActionPlan(tools=["cart.view"], next_step="show_cart"), None
            elif action == "order_list":
                return ActionPlan(tools=["order.list"], next_step="show_orders"), None
            elif action == "order_track":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "reorder":
                return ActionPlan(tools=["order.reorder"], next_step="confirm_reorder"), None
            elif action == "logout":
                return ActionPlan(tools=["auth.logout"], next_step="confirm_logout"), None
            elif action == "chat_human":
                return ActionPlan(tools=["system.escalate"], next_step="escalate_chat"), None
            elif action == "my_reviews":
                return ActionPlan(tools=["product.reviews"], next_step="show_reviews"), None
            elif action == "check_stock":
                return ActionPlan(tools=[], next_step="ask_product_name"), None # Need product name
            elif action == "check_price":
                return ActionPlan(tools=[], next_step="ask_product_name"), None # Need product name
            elif action == "staff_lookup_order":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "staff_customer_lookup":
                return ActionPlan(tools=[], next_step="ask_customer_info"), None
            elif action == "update_order_status":
                return ActionPlan(tools=[], next_step="ask_order_id_status"), None
            elif action == "print_shipping_label":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "sales_report":
                return ActionPlan(tools=["manager.report_sales"], next_step="show_report"), None
            elif action == "top_products":
                return ActionPlan(tools=["manager.top_products"], next_step="show_top_products"), None
            elif action == "customer_analytics":
                return ActionPlan(tools=["manager.customer_analytics"], next_step="show_analytics"), None
            elif action.startswith("cart.add_item"):
                try:
                    params = {}
                    if "?" in action:
                        query = action.split("?", 1)[1]
                        for pair in query.split("&"):
                            if "=" in pair:
                                k, v = pair.split("=", 1)
                                params[k] = v
                    
                    intent.entities["product_id"] = params.get("product_id")
                    intent.entities["variant_id"] = params.get("variant_id")
                    intent.entities["quantity"] = 1
                    
                    return ActionPlan(tools=["cart.add"], next_step="confirm_add_to_cart"), None
                except Exception as e:
                    print(f"Error parsing cart.add_item tag: {e}")
                    return ActionPlan(tools=[], next_step="error"), None
            
            # Add more direct actions here
        
        # 2. Scope Tag (e.g., "scope:order")
        if processed.tag and processed.tag.startswith("scope:"):
            scope = processed.tag.split(":", 1)[1]
            agent = self.agents.get(scope)
            if agent:
                print(f"[Orchestrator] Scoped routing to: {scope}")
                # Agent performs its own scoped NLU if intent is generic/unknown or we want to re-evaluate
                # For now, we pass None as intent to force agent to re-classify within scope
                return await agent.process(processed, intent=None)
        
        # 3. Implicit Routing (Global NLU)
        # We use the intent passed from the global classifier to decide which agent to call
        target_agent = self._route_by_intent(intent.intent)
        if target_agent:
            print(f"[Orchestrator] Implicit routing to: {target_agent.name}")
            return await target_agent.process(processed, intent)
            
        # Default fallback
        print("[Orchestrator] No agent found, fallback.")
        return ActionPlan(tools=[], next_step=None), None

    def _route_by_intent(self, intent_name: str):
        """Route intent to appropriate specialized agent"""
        for agent in self.agents.values():
            if agent.can_handle(intent_name):
                return agent
        return self.agents["sales"] # Default to sales if unknown
    
    def get_agent_suggestions(self, intent: str) -> list:
        """Get suggested actions based on intent - for better UX"""
        suggestions = {
            "product_inquiry": [
                {"label": "Xem giỏ hàng", "action": "action:view_cart"},
                {"label": "Sản phẩm gợi ý", "action": "action:my_recommendations"},
            ],
            "cart_view": [
                {"label": "Thanh toán ngay", "action": "action:checkout"},
                {"label": "Tiếp tục mua sắm", "action": "action:continue_shopping"},
            ],
            "order_tracking": [
                {"label": "Đơn hàng của tôi", "action": "action:order_list"},
                {"label": "Liên hệ hỗ trợ", "action": "action:chat_human"},
            ],
            "general": [
                {"label": "Tìm sản phẩm", "action": "action:product_search"},
                {"label": "Xem giỏ hàng", "action": "action:view_cart"},
                {"label": "Kiểm tra đơn hàng", "action": "action:order_list"},
            ],
        }
        return suggestions.get(intent, suggestions["general"])

In [ ]:
# --- AGENT 4: EXECUTOR (app/agents/executor.py) ---
# import logging
# from typing import Optional, Dict, Any
# from app.models.agent_types import ProcessedInput, IntentResult, ActionPlan, ToolResults
# from app.tools import ...

def _merge_tool_result(results: ToolResults, tool_result: Any):
    if tool_result is None:
        return
    if isinstance(tool_result, ToolResults):
        results.ok = results.ok and tool_result.ok
        if tool_result.errors:
            results.errors.extend(tool_result.errors)
        if tool_result.timings_ms:
            results.timings_ms.update(tool_result.timings_ms)
        results.data = tool_result.data
    else:
        results.data = tool_result

class Executor:
    async def run(
        self, 
        processed: ProcessedInput, 
        intent: IntentResult, 
        plan: ActionPlan
    ) -> Optional[ToolResults]:
        if not plan.tools:
            return None

        results = ToolResults(ok=True)
        
        for tool_name in plan.tools:
            try:
                logger.info(f"[Executor] Running tool: {tool_name}")
                
                # Cart Tools
                if tool_name == "cart.view":
                    if processed.session_ctx.cart_id:
                        res = await cart_tools.view_cart(processed.session_ctx.cart_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing cart_id")
                elif tool_name == "cart.add":
                    # Extract product and quantity from entities
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    qty = intent.entities.get("quantity", 1)
                    variant_id = intent.entities.get("variant_id")
                    
                    if processed.session_ctx.cart_id:
                        if variant_id:
                            # If we have explicit variant_id (e.g. from UI action), use it directly
                            res = await cart_tools.add_to_cart(processed.session_ctx.cart_id, variant_id, qty)
                        else:
                            # Otherwise try smart add (search -> add)
                            res = await cart_tools.add_to_cart_smart(processed.session_ctx.cart_id, query, qty)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing cart_id")
                
                # Product Tools
                elif tool_name == "product.search":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await product_tools.search_products(query)
                    _merge_tool_result(results, res)
                elif tool_name == "product.recommend":
                    # The tool currently only accepts limit, not customer_id
                    res = await product_recommend_tool.recommend_products(limit=5)
                    _merge_tool_result(results, res)
                elif tool_name == "product.detail":
                    # Try to get product_id from entities first
                    product_id = intent.entities.get("product_id")
                    
                    # If no product_id in entities, try to get from context (last products shown)
                    if not product_id and processed.session_ctx.last_product_ids:
                        product_id = processed.session_ctx.last_product_ids[0]
                        logger.info(f"[Executor] Using context product_id: {product_id}")
                    
                    if product_id:
                        res = await product_tools.get_product_details(product_id)
                        _merge_tool_result(results, res)
                    else:
                        # Fallback: try to search if we have a query
                        # Extract product name from text (remove "chi tiết", "xem", etc)
                        import re
                        query = intent.entities.get("product_query")
                        if not query:
                            # Try to extract product name from cleaned text
                            text_lower = processed.cleaned_text.lower()
                            # Remove common phrases (order matters - remove longer phrases first)
                            for phrase in ["cho tôi xem chi tiết", "cho tôi xem", "xem chi tiết", "chi tiết của", "chi tiết", "xem", "tôi muốn", "muốn"]:
                                text_lower = text_lower.replace(phrase, "")
                            query = text_lower.strip()
                        
                        if query and len(query) >= 2:
                            logger.info(f"[Executor] Searching for product: '{query}'")
                            res = await product_tools.search_products(query, limit=1)
                            _merge_tool_result(results, res)
                            # Store search query in entities for response_generator
                            intent.entities["product_query"] = query
                        else:
                            results.ok = False
                            results.errors.append("Missing product_id or query")
                            # Still store a placeholder for template
                            intent.entities["product_query"] = "sản phẩm bạn cần"

                # Order Tools
                elif tool_name == "order.track":
                    order_id = intent.entities.get("order_id")
                    email = intent.entities.get("email") # Might need to ask user
                    if order_id:
                        res = await order_tools.lookup_order(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "order.list":
                    if processed.customer_id:
                        res = await order_tools.list_orders(processed.customer_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "order.reorder":
                    # Assuming we reorder the last order or specific order
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await order_tools.reorder(order_id)
                        _merge_tool_result(results, res)
                    else:
                        # Maybe list orders to pick?
                        results.ok = False
                        results.errors.append("Missing order_id")

                # Staff Tools
                elif tool_name == "staff.check_stock":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await staff_tools.check_stock(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.check_price":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await staff_tools.check_price(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.customer_lookup":
                    query = intent.entities.get("customer_query", processed.cleaned_text)
                    res = await staff_tools.lookup_customer(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.order_history":
                    customer_id = intent.entities.get("customer_id")
                    if customer_id:
                        res = await staff_tools.get_customer_order_history(customer_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "staff.create_order":
                    if processed.customer_id:
                        res = await staff_tools.create_draft_order(processed.customer_id, [])
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "staff.lookup_order":
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await staff_tools.lookup_order(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "staff.update_order":
                    order_id = intent.entities.get("order_id")
                    status = intent.entities.get("status", "processing")
                    if order_id:
                        res = await staff_tools.update_order_status(order_id, status)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "staff.print_label":
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await staff_tools.print_shipping_label(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                
                # Manager Tools
                elif tool_name == "manager.report_sales":
                    res = await report_tools.get_sales_report()
                    _merge_tool_result(results, res)
                elif tool_name == "manager.top_products":
                    res = await report_tools.get_top_products()
                    _merge_tool_result(results, res)
                elif tool_name == "manager.customer_analytics":
                    res = await report_tools.get_customer_analytics()
                    _merge_tool_result(results, res)

                # System/Auth Tools
                elif tool_name == "system.escalate":
                    # Just a marker, maybe log it
                    results.data = {"status": "escalated", "message": "Connecting to human agent..."}
                elif tool_name == "auth.logout":
                    # In a real app, this might invalidate session
                    results.data = {"status": "logged_out", "message": "You have been logged out."}
                
                # Product Tools (Extra)
                elif tool_name == "product.reviews":
                    # Mock
                    results.data = {"reviews": []}

                else:
                    logger.warning(f"[Executor] Unknown tool: {tool_name}")
                    results.errors.append(f"Unknown tool: {tool_name}")

            except Exception as e:
                logger.error(f"[Executor] Error running {tool_name}: {e}")
                results.ok = False
                results.errors.append(str(e))
        
        return results

In [ ]:
# --- AGENT 5: RESPONSE GENERATOR (app/agents/response_generator.py) ---
# import os
# import yaml
# import json
# import logging
# from typing import List, Dict, Any, Optional
# from openai import AsyncOpenAI
# from openai.types.chat import ChatCompletionSystemMessageParam, ChatCompletionUserMessageParam
# from app.config import GOOGLE_API_KEY, GEMINI_BASE_URL, GEMINI_MODEL, SYSTEM_PROMPT_VI
# from app.models.api_models import ChatResponse, QuickReply, ProductInfo
# from app.models.agent_types import ProcessedInput, IntentResult, ActionPlan, ToolResults
# from app.agents.response_templates import get_response_template

class ResponseGenerator:
    def __init__(self):
        if GOOGLE_API_KEY == "YOUR_API_KEY_HERE":
            self.client = None
        else:
            from openai import AsyncOpenAI
            self.client = AsyncOpenAI(api_key=GOOGLE_API_KEY, base_url=GEMINI_BASE_URL)
        self.context_config = CONTEXT_CONFIG

    def _load_context_config(self):
        return CONTEXT_CONFIG

    async def run(
        self, 
        processed: ProcessedInput, 
        intent: IntentResult, 
        plan: ActionPlan, 
        tool_res: Optional[ToolResults]
    ) -> ChatResponse:
        
        # 1. Generate Text Response
        response_text = await self._generate_llm_response(processed, intent, tool_res)
        
        # 2. Generate Quick Replies (Options)
        quick_replies = self._generate_quick_replies(processed)
        
        # 3. Format Products if available
        products = []
        if tool_res and tool_res.ok and isinstance(tool_res.data, list):
            for item in tool_res.data:
                # Try to map to ProductInfo
                try:
                    # If item is already ProductInfo, append directly
                    if isinstance(item, ProductInfo):
                        products.append(item)
                    # Handle if item is dict
                    elif isinstance(item, dict):
                        # Filter keys that match ProductInfo
                        valid_keys = ProductInfo.model_fields.keys()
                        filtered = {k: v for k, v in item.items() if k in valid_keys}
                        # Ensure required fields
                        if "id" in filtered and "title" in filtered and "handle" in filtered:
                            products.append(ProductInfo(**filtered))
                    # Handle if item is object
                    elif hasattr(item, "id") and hasattr(item, "title"):
                        # Manually map variants if they exist in the object but weren't automatically mapped
                        variants = getattr(item, "variants", [])
                        # If variants are already ProductVariantInfo objects, use them.
                        # If they are dicts, convert them.
                        mapped_variants = []
                        for v in variants:
                            if isinstance(v, dict):
                                mapped_variants.append(v) # Pydantic will handle dict -> model
                            else:
                                mapped_variants.append(v)

                        products.append(ProductInfo(
                            id=item.id,
                            title=item.title,
                            handle=item.handle,
                            thumbnail=getattr(item, "thumbnail", None),
                            price=getattr(item, "price", None),
                            currency_code=getattr(item, "currency_code", None),
                            variants=mapped_variants
                        ))
                except Exception as e:
                    logger.warning(f"Failed to map product item: {e}")
        
        # Add product-specific quick replies if products found
        if products:
            # Add "Buy Now" button first
            quick_replies.insert(0, QuickReply(label="Mua ngay", value=f"Đặt hàng {products[0].title}"))
            # Add "View Details" button second
            quick_replies.insert(1, QuickReply(label="Xem chi tiết", value=f"Chi tiết {products[0].title}"))

        return ChatResponse(
            response=response_text,
            session_id=processed.session_id,
            products=products,
            quick_replies=quick_replies,
            metadata={"intent": intent.intent}
        )

    async def _generate_llm_response(self, processed, intent, tool_res) -> str:
        # Try template-based response first for common cases
        template_context = {}
        
        print(f"[ResponseGen] intent={intent.intent}, tool_res={'exists' if tool_res else 'None'}, tool_res.ok={tool_res.ok if tool_res else 'N/A'}, data_len={len(tool_res.data) if tool_res and tool_res.ok else 0}")
        
        # PRIORITY 1: Handle FAQ intents with templates (avoid LLM hallucination)
        if intent.intent in ["faq_shipping", "faq_payment", "faq_return", "faq_promo"]:
            print(f"[ResponseGen] Using FAQ template for {intent.intent}")
            return get_response_template(intent.intent)
        
        # PRIORITY 2: Handle thank_you/goodbye
        if intent.intent == "thank_you":
            return get_response_template("thank_you")
        elif intent.intent == "goodbye":
            return get_response_template("goodbye")
        
        if tool_res:
            if tool_res.ok and isinstance(tool_res.data, list):
                template_context['count'] = len(tool_res.data)
                if len(tool_res.data) > 0:
                    first_item = tool_res.data[0]
                    if isinstance(first_item, dict):
                        template_context['product'] = first_item.get('title', '')
                    elif hasattr(first_item, 'title'):
                        template_context['product'] = first_item.title
            else:
                # tool_res.ok = False means tool failed/no results
                template_context['count'] = 0
        
        # PRIORITY 3: Handle product intents with templates (even without tool_res)
        if intent.intent == "general" and (not tool_res or not tool_res.ok):
            print("[ResponseGen] Using greeting template")
            return get_response_template("greeting")
        elif intent.intent in ["product_inquiry", "product_detail"]:
            # Get product query from entities (may be set by executor)
            query = intent.entities.get("product_query")
            if query:
                template_context['query'] = query
            else:
                # Fallback: extract from processed text
                template_context['query'] = "sản phẩm bạn cần"
            
            if template_context.get('count', 0) > 0:
                print(f"[ResponseGen] Using product_found template, count={template_context['count']}")
                return get_response_template("product_found", template_context)
            else:
                # Product not found - use template to avoid LLM hallucination
                print(f"[ResponseGen] Using product_not_found template, query={template_context.get('query')}")
                return get_response_template("product_not_found", template_context)
        elif intent.intent == "product_recommend":
            if tool_res.ok and template_context.get('count', 0) > 0:
                return get_response_template("product_recommend")
            else:
                return "Xin lỗi, shop đang cập nhật sản phẩm. Bạn quay lại sau nhé!"
        elif intent.intent == "cart_add":
            if tool_res.ok:
                return get_response_template("cart_added", template_context)
            else:
                return "Xin lỗi, tôi cần biết sản phẩm nào bạn muốn thêm vào giỏ hàng."
        
        # PRIORITY 4: Fallback to LLM only for complex cases
        # But prevent hallucination by checking tool_res.ok
        if tool_res and not tool_res.ok:
            # Tool failed - don't let LLM make up data
            print(f"[ResponseGen] Tool failed, using error template")
            if intent.intent in ["product_inquiry", "product_detail"]:
                return get_response_template("product_not_found", template_context)
            return "Xin lỗi, tôi không tìm thấy thông tin bạn cần. Bạn có thể hỏi lại được không?"
        
        # Construct prompt for LLM (only when tool succeeded or no tool needed)
        print(f"[ResponseGen] Falling through to LLM generation")
        context_str = ""
        if tool_res:
            if tool_res.ok:
                # Truncate data if too large
                data_str = json.dumps(tool_res.data, ensure_ascii=False, default=str)
                if len(data_str) > 3000: # Increased limit for better context
                    data_str = data_str[:3000] + "...(truncated)"
                context_str = f"Tool Result: {data_str}"
            else:
                context_str = f"Tool Error: {tool_res.errors}"
        
        # Add currency hint if available in tool result (e.g. from product search)
        currency_hint = ""
        if tool_res and tool_res.ok and isinstance(tool_res.data, list) and len(tool_res.data) > 0:
            first_item = tool_res.data[0]
            if isinstance(first_item, dict) and "currency_code" in first_item:
                 currency_hint = f"\nLưu ý: Đơn vị tiền tệ đang sử dụng là {first_item.get('currency_code', 'N/A').upper()}."
            elif hasattr(first_item, "currency_code"):
                 currency_hint = f"\nLưu ý: Đơn vị tiền tệ đang sử dụng là {getattr(first_item, 'currency_code', 'N/A').upper()}."

        messages = [
            {"role": "system", "content": SYSTEM_PROMPT_VI + currency_hint},
            {"role": "user", "content": f"User Input: {processed.text}\nIntent: {intent.intent}\nContext: {context_str}"}
        ]
        
        if self.client:
            try:
                completion = await self.client.chat.completions.create(
                    model=GEMINI_MODEL,
                    messages=messages,
                    temperature=0.7
                )
                return completion.choices[0].message.content or ""
            except Exception as e:
                logger.error(f"LLM Error: {e}")
                return get_response_template("error_generic")
        else:
            return f"[MOCK LLM RESPONSE] Based on {intent.intent} and data: {tool_res.data if tool_res else 'None'}"

    def _generate_quick_replies(self, processed) -> List[QuickReply]:
        nodes = self.context_config.get("nodes", [])
        
        # Helper to find node by ID recursively
        def find_node(node_list, target_id):
            for node in node_list:
                if node.get("id") == target_id:
                    return node
                if "children" in node:
                    found = find_node(node["children"], target_id)
                    if found: return found
            return None

        # Determine current context node
        current_node = None
        if processed.tag:
            # If tag is like "action:view_cart", we might not find a node if it's a leaf.
            # But if tag is a group ID, we might.
            # Let's try to find by ID first (assuming tag might be ID)
            current_node = find_node(nodes, processed.tag)
        
        # If no specific node found, default to user root
        if not current_node:
            user_root_id = f"{processed.user_type}_root"
            current_node = find_node(nodes, user_root_id)

        if not current_node:
            return []

        options = []
        # Return children of the current node
        for child in current_node.get("children", []):
            # Determine value: prefer value, then tag, then id
            val = child.get("value") or child.get("tag") or child.get("id")
            options.append(QuickReply(
                label=child["label"],
                value=val,
                metadata={"type": child.get("type")}
            ))
            
        return options

In [ ]:
# --- DATA LOADING ---
# Upload 'multi_agent_test_cases_full.csv' to Colab files
try:
    df = pd.read_csv('multi_agent_test_cases_full.csv')
    print(f"Loaded {len(df)} test cases.")
except FileNotFoundError:
    print("File not found. Creating sample data...")
    data = {
        "id": [1, 2, 3],
        "user_input": ["Xin chào shop", "Tìm balo JanSport màu đen", "Kiểm tra đơn hàng #123"],
        "expected_intent_classifier_intent": ["general", "product_inquiry", "order_tracking"],
        "expected_executor_tools_called": ["", "product.search", "order.track"]
    }
    df = pd.DataFrame(data)
    print("Created sample data.")

df.head()

In [ ]:
# --- EVALUATION LOOP ---

async def evaluate_row(row):
    # Initialize Agents
    input_processor = InputProcessor()
    intent_classifier = IntentClassifier()
    orchestrator = Orchestrator()
    executor = Executor()
    response_generator = ResponseGenerator()

    # 1. Input
    req = ChatRequest(session_id=f"sess_{row['id']}", message=row['user_input'])
    processed = await input_processor.run(req)
    
    # 2. Intent
    intent_res = await intent_classifier.run(processed)
    
    # 3. Orchestrator
    plan, _ = await orchestrator.run(processed, intent_res)
    
    # 4. Executor
    tool_res = await executor.run(processed, intent_res, plan)
    
    # 5. Response
    response = await response_generator.run(processed, intent_res, plan, tool_res)
    
    # --- CHECKS ---
    results = {
        "id": row['id'],
        "input": row['user_input'],
        "intent_actual": intent_res.intent,
        "intent_expected": row.get('expected_intent_classifier_intent'),
        "intent_match": intent_res.intent == row.get('expected_intent_classifier_intent'),
        
        "tools_actual": ",".join(plan.tools),
        "tools_expected": row.get('expected_executor_tools_called', ""),
        # Simple check: is expected tool in actual tools?
        "tools_match": str(row.get('expected_executor_tools_called', "")) in ",".join(plan.tools) if pd.notna(row.get('expected_executor_tools_called')) else True,
        
        "response_preview": response.response[:50] + "..."
    }
    return results

# Run Evaluation
results_list = []
print("Starting evaluation...")
for index, row in df.iterrows():
    res = await evaluate_row(row)
    results_list.append(res)
    print(f"Row {row['id']}: Intent Match={res['intent_match']}, Tools Match={res['tools_match']}")

results_df = pd.DataFrame(results_list)

In [ ]:
# --- SUMMARY REPORT ---
print("\n--- EVALUATION SUMMARY ---")
total = len(results_df)
intent_acc = results_df['intent_match'].mean() * 100
tools_acc = results_df['tools_match'].mean() * 100

print(f"Total Test Cases: {total}")
print(f"Intent Accuracy: {intent_acc:.2f}%")
print(f"Tool Selection Accuracy: {tools_acc:.2f}%")

print("\n--- FAILED CASES ---")
failed = results_df[~results_df['intent_match'] | ~results_df['tools_match']]
if not failed.empty:
    display(failed[['id', 'input', 'intent_actual', 'intent_expected', 'tools_actual', 'tools_expected']])
else:
    print("All cases passed!")

In [ ]:
!pip install pydantic openai pandas python-dotenv nest_asyncio pyyaml

In [ ]:
import os
import json
import re
import yaml
import asyncio
import logging
import pandas as pd
import nest_asyncio
from typing import List, Dict, Any, Optional, Tuple
from pydantic import BaseModel, Field
from datetime import datetime
from enum import Enum

# Apply nest_asyncio to allow async loops in notebook
nest_asyncio.apply()

# Configure Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("Evaluation")

In [ ]:
# --- CONFIGURATION ---
# Replace with your actual API Key
GOOGLE_API_KEY = "YOUR_API_KEY_HERE" 
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
GEMINI_MODEL = "gemini-1.5-flash"

SYSTEM_PROMPT_VI = """
Bạn là trợ lý ảo AI chuyên nghiệp của cửa hàng JanSport Việt Nam.
Nhiệm vụ của bạn là hỗ trợ khách hàng tìm kiếm sản phẩm, kiểm tra đơn hàng và giải đáp thắc mắc.
Luôn trả lời bằng tiếng Việt, giọng điệu thân thiện, nhiệt tình và chuyên nghiệp.
"""

# Mock Context Config (from context_config.yaml)
CONTEXT_CONFIG = {
    "nodes": [
        {
            "id": "guest_root",
            "label": "Guest Root",
            "children": [
                {"label": "Tìm sản phẩm", "value": "action:product_search"},
                {"label": "Tra cứu đơn hàng", "value": "action:order_track"}
            ]
        },
        {
            "id": "customer_root",
            "label": "Customer Root",
            "children": [
                {"label": "Đơn hàng của tôi", "value": "action:order_list"},
                {"label": "Giỏ hàng", "value": "action:view_cart"}
            ]
        }
    ]
}

In [ ]:
# --- DATA MODELS ---
# Combined from app/models/agent_types.py and app/models/api_models.py

class SessionContext(BaseModel):
    last_messages: List[Dict[str, Any]] = Field(default_factory=list)
    cart_id: Optional[str] = None
    last_product_ids: List[str] = Field(default_factory=list)

class ProcessedInput(BaseModel):
    session_id: str
    customer_id: Optional[str] = None
    text: str
    cleaned_text: str
    language: str = "vi"  # "vi" | "en"
    user_type: str = "guest"  # "guest" | "customer"
    tag: Optional[str] = None
    session_ctx: SessionContext = Field(default_factory=SessionContext)

class IntentResult(BaseModel):
    intent: str
    sub_intent: Optional[str] = None
    confidence: float = 0.0
    entities: Dict[str, Any] = Field(default_factory=dict)

class ActionPlan(BaseModel):
    tools: List[str] = Field(default_factory=list)
    required_data: List[str] = Field(default_factory=list)
    next_step: Optional[str] = None
    escalate: bool = False

class ToolResults(BaseModel):
    ok: bool
    data: Any = None
    errors: List[str] = Field(default_factory=list)
    timings_ms: Dict[str, int] = Field(default_factory=dict)

# From api_models.py
class ChatActionType(str, Enum):
    NAVIGATE = "navigate"
    API_CALL = "api_call"
    FORM = "form"
    SHOW_PRODUCTS = "show_products"
    REQUEST_ORDER_INFO = "request_order_info"
    GUIDE_TO_CART = "guide_to_cart"
    NONE = "none"

class ChatAction(BaseModel):
    type: ChatActionType = ChatActionType.NONE
    payload: Dict[str, Any] = Field(default_factory=dict)

class QuickReply(BaseModel):
    label: str
    value: str
    metadata: Optional[Dict[str, Any]] = None

class ProductVariantInfo(BaseModel):
    id: str
    title: str
    price: Optional[str] = None
    currency_code: Optional[str] = None

class ProductInfo(BaseModel):
    id: str
    title: str
    handle: str
    thumbnail: Optional[str] = None
    price: Optional[str] = None
    currency_code: Optional[str] = None
    variants: List[ProductVariantInfo] = Field(default_factory=list)
    
class ChatRequest(BaseModel):
    message: str
    session_id: str
    customer_id: Optional[str] = None
    tag: Optional[str] = None
    language: Optional[str] = None
    metadata: Dict[str, Any] = Field(default_factory=dict)

class ChatResponse(BaseModel):
    response: str
    session_id: str
    products: List[ProductInfo] = Field(default_factory=list)
    quick_replies: List[QuickReply] = Field(default_factory=list)
    metadata: Dict[str, Any] = Field(default_factory=dict)

In [ ]:
# --- RESPONSE TEMPLATES (app/agents/response_templates.py) ---
# Copied exactly from source

RESPONSE_TEMPLATES = {
    # Greetings
    "greeting": [
        "Xin chào! Tôi là trợ lý ảo của shop. Tôi có thể giúp gì cho bạn hôm nay? 😊",
        "Chào bạn! Rất vui được hỗ trợ bạn. Bạn cần tìm sản phẩm gì không?",
        "Hello! Chào mừng bạn đến với shop. Tôi có thể tư vấn gì cho bạn?"
    ],
    
    # Product Search Success
    "product_found": [
        "Mình tìm thấy {count} sản phẩm phù hợp với '{query}' ạ:",
        "Đây là {count} sản phẩm mình tìm được cho bạn về '{query}':",
        "Shop có {count} sản phẩm '{query}' như sau:"
    ],
    
    # Product Not Found
    "product_not_found": [
        "Rất tiếc, shop không tìm thấy sản phẩm '{query}' bạn cần 😔\nBạn có thể thử tìm kiếm sản phẩm tương tự không?",
        "Hiện tại shop chưa có sản phẩm '{query}' ạ. Bạn muốn xem sản phẩm khác không?",
        "Mình không tìm thấy '{query}' trong kho. Bạn có thể mô tả chi tiết hơn hoặc xem sản phẩm tương tự?"
    ],
    
    # Product Recommend
    "product_recommend": [
        "🌟 Đây là một số sản phẩm hot mà bạn có thể thích:",
        "Mình gợi ý cho bạn vài sản phẩm đang bán chạy nha:",
        "Đây là top sản phẩm được yêu thích nhất của shop:"
    ],
    
    # Cart Actions
    "cart_added": [
        "Đã thêm {product} vào giỏ hàng! Bạn muốn tiếp tục mua sắm hay thanh toán luôn?",
        "OK! Đã cho {product} vào giỏ rồi nhé. Mua thêm gì nữa không?"
    ],
    
    "cart_view_empty": [
        "Giỏ hàng của bạn đang trống. Tìm sản phẩm ngay nhé! 🛍️",
        "Bạn chưa có sản phẩm nào trong giỏ. Cùng khám phá shop nhé!"
    ],
    
    "cart_view_has_items": [
        "Giỏ hàng của bạn có {count} sản phẩm:",
        "Bạn đang có {count} món trong giỏ:"
    ],
    
    # Order Tracking
    "order_found": [
        "Đơn hàng #{order_id} của bạn:\nTrạng thái: {status}\nDự kiến giao: {delivery_date}",
        "Mình check đơn #{order_id} rồi:\n📦 {status}\n🚚 Giao dự kiến: {delivery_date}"
    ],
    
    "order_not_found": [
        "Không tìm thấy đơn hàng #{order_id}. Bạn kiểm tra lại mã đơn giúp mình nhé!",
        "Mình không thấy đơn #{order_id} trong hệ thống. Bạn có chắc mã đúng không?"
    ],
    
    # FAQ
    "faq_shipping": [
        "🚚 Thông tin vận chuyển:\n- Nội thành Hà Nội/HCM: 2-3 ngày, phí 30.000₫\n- Ngoại thành: 3-5 ngày, phí 50.000₫\n- Miễn phí ship cho đơn hàng trên 500.000₫\n- Hỗ trợ COD toàn quốc",
        "Shop ship nhanh trong 2-3 ngày nội thành nha! Phí 30k, FREE SHIP cho đơn trên 500k. Bạn có thể thanh toán COD khi nhận hàng ạ!",
        "Vận chuyển:\n• Giao hàng toàn quốc\n• Nội thành: 2-3 ngày (30k)\n• Ngoại thành: 3-5 ngày (50k)\n• FREE SHIP đơn > 500k\n• Hỗ trợ COD"
    ],
    
    "faq_return": [
        "🔄 Chính sách đổi trả:\n- Đổi trả miễn phí trong 7 ngày\n- Sản phẩm chưa sử dụng, còn nguyên tem mác\n- Miễn phí đổi size/màu (nếu còn hàng)\n- Hoàn tiền trong 3-5 ngày làm việc",
        "Bạn được đổi trả miễn phí trong 7 ngày nếu sản phẩm còn nguyên tem mác nha! Đổi size/màu miễn phí luôn ạ."
    ],
    
    "faq_payment": [
        "💳 Các hình thức thanh toán:\n- COD (tiền mặt khi nhận hàng)\n- Chuyển khoản\n- Thẻ tín dụng/ghi nợ\n- Ví điện tử (Momo, ZaloPay)",
        "Bạn có thể thanh toán bằng COD, chuyển khoản, thẻ, hoặc ví Momo/ZaloPay nha!"
    ],
    
    "faq_promo": [
        "🎁 Khuyến mãi hiện tại:\n- Giảm 10% đơn đầu tiên\n- Free ship đơn >500k\n- Tích điểm đổi quà",
        "Bạn mới có ưu đãi giảm 10% đơn đầu + freeship đơn >500k đó! 🎉"
    ],
    
    # Thank you & Goodbye
    "thank_you": [
        "Không có gì! Rất vui được hỗ trợ bạn. Cần gì cứ gọi mình nhé! 😊",
        "Hehe, có gì đâu! Cần gì cứ nhắn mình nha!",
        "Luôn sẵn sàng giúp bạn! Mua sắm vui vẻ nhé! 🛍️"
    ],
    
    "goodbye": [
        "Tạm biệt! Hẹn gặp lại bạn sớm. Chúc bạn một ngày tốt lành! 👋",
        "Bye bye! Nhớ ghé shop thường xuyên nha! 😊",
        "Hẹn gặp lại! Mua sắm vui vẻ nhé! 🛍️"
    ],
    
    # Errors
    "error_generic": [
        "Xin lỗi, có lỗi xảy ra. Bạn có thể thử lại không?",
        "Ối, có gì đó không ổn. Mình thử lại nhé!"
    ],
    
    "error_no_products": [
        "Shop đang cập nhật sản phẩm. Bạn quay lại sau nhé!",
        "Hiện tại sản phẩm đang hết hàng. Mình sẽ thông báo khi có hàng!"
    ],
    
    # Staff/Manager
    "staff_success": [
        "Đã xử lý xong! Kết quả:",
        "Done! Thông tin bạn cần:"
    ],
    
    "manager_report": [
        "📊 Báo cáo {report_type}:\n{data}",
        "Đây là thống kê {report_type} bạn yêu cầu:\n{data}"
    ],
    
    # Clarification needed
    "need_clarification": [
        "Mình chưa hiểu rõ lắm. Bạn có thể nói rõ hơn được không?",
        "Hmm, bạn muốn tìm gì nhỉ? Mô tả chi tiết giúp mình nha!",
        "Bạn cần tìm sản phẩm gì? Cho mình biết thêm thông tin nhé!"
    ],
    
    "ask_product_details": [
        "Bạn muốn tìm loại nào? (màu, size, giá...)",
        "Cho mình biết thêm chi tiết: màu sắc, kích thước, giá cả?"
    ],
    
    "ask_order_id": [
        "Bạn cho mình mã đơn hàng để check nhé! (VD: #1234 hoặc ORD-5678)",
        "Mã đơn hàng của bạn là gì ạ?"
    ]
}

def get_response_template(intent: str, context: Optional[dict] = None) -> str:
    """Get appropriate response template based on intent and context"""
    import random
    
    context = context or {}
    templates = RESPONSE_TEMPLATES.get(intent, RESPONSE_TEMPLATES["need_clarification"])
    
    # Select random template for variety
    template = random.choice(templates)
    
    # Format with context if available
    try:
        if context:
            template = template.format(**context)
    except KeyError:
        pass  # Use template as-is if formatting fails
    
    return template

In [ ]:
# --- MOCKS FOR EXTERNAL DEPENDENCIES ---

# Mock Tools to simulate Medusa/Redis calls
class MockTools:
    @staticmethod
    async def search_products(query, limit=5):
        return ToolResults(ok=True, data=[
            {"id": "p_1", "title": f"JanSport SuperBreak - {query}", "handle": "superbreak", "price": "500000", "currency_code": "vnd"},
            {"id": "p_2", "title": f"JanSport Big Student - {query}", "handle": "big-student", "price": "800000", "currency_code": "vnd"}
        ])
    
    @staticmethod
    async def get_product_details(product_id):
        return ToolResults(ok=True, data={"id": product_id, "title": "JanSport Detail", "description": "Great bag", "price": "500000", "currency_code": "vnd"})

    @staticmethod
    async def recommend_products(limit=5):
        return ToolResults(ok=True, data=[{"id": "rec_1", "title": "Trending Bag", "handle": "trending", "price": "600000", "currency_code": "vnd"}])

    @staticmethod
    async def view_cart(cart_id):
        return ToolResults(ok=True, data={"id": cart_id, "items": []})

    @staticmethod
    async def add_to_cart(cart_id, variant_id, qty):
        return ToolResults(ok=True, data={"cart_id": cart_id, "added": True})
        
    @staticmethod
    async def add_to_cart_smart(cart_id, query, qty):
        return ToolResults(ok=True, data={"cart_id": cart_id, "added": True, "product": query})

    @staticmethod
    async def lookup_order(order_id):
        if order_id == "123":
            return ToolResults(ok=True, data={"id": "123", "status": "shipped", "delivery_date": "2023-12-25"})
        return ToolResults(ok=False, errors=["Order not found"])
    
    @staticmethod
    async def list_orders(customer_id):
        return ToolResults(ok=True, data=[{"id": "123", "status": "shipped"}])

    # Staff/Manager Mocks
    @staticmethod
    async def check_stock(query): return ToolResults(ok=True, data={"product": query, "stock": 10})
    @staticmethod
    async def check_price(query): return ToolResults(ok=True, data={"product": query, "price": 500000})
    @staticmethod
    async def lookup_customer(query): return ToolResults(ok=True, data={"id": "cust_1", "name": "Test User"})
    @staticmethod
    async def get_customer_order_history(cid): return ToolResults(ok=True, data=[])
    @staticmethod
    async def create_draft_order(cid, items): return ToolResults(ok=True, data={"id": "draft_1"})
    @staticmethod
    async def update_order_status(oid, status): return ToolResults(ok=True, data={"id": oid, "status": status})
    @staticmethod
    async def print_shipping_label(oid): return ToolResults(ok=True, data={"url": "http://label.pdf"})
    @staticmethod
    async def get_sales_report(): return ToolResults(ok=True, data={"total_sales": 10000000})
    @staticmethod
    async def get_top_products(): return ToolResults(ok=True, data=[{"title": "Bag A", "sales": 100}])
    @staticmethod
    async def get_customer_analytics(): return ToolResults(ok=True, data={"new_users": 50})

# Mock Specialized Agents (to avoid importing their complex logic)
class MockSpecializedAgent:
    def __init__(self, name): self.name = name
    def can_handle(self, intent): return False 
    async def process(self, processed, intent): 
        # Simple pass-through for testing
        return ActionPlan(tools=[], next_step=f"mock_{self.name}"), None

# Define tool objects for Executor to use
cart_tools = MockTools
product_tools = MockTools
product_recommend_tool = MockTools
order_tools = MockTools
staff_tools = MockTools
report_tools = MockTools
system_tools = MockTools
customer_tools = MockTools

In [ ]:
# --- AGENT 1: INPUT PROCESSOR (app/agents/input_processor.py) ---
# from typing import Optional, Dict, Any, List
# import re
# import asyncpg
# from app.models import ChatRequest
# from app.models.agent_types import ProcessedInput, SessionContext

def _normalize_text(text: str) -> str:
    # Trim, collapse spaces; keep diacritics (for VI) and punctuation minimally
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text


def _detect_language_heuristic(text: str) -> str:
    t = text.lower()
    # Very lightweight heuristic: VI keywords/diacritics vs common EN markers
    if any(ch in t for ch in "ăâêôơưđáàảãạấầẩẫậắằẳẵặéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ"):
        return "vi"
    if any(word in t for word in [" the ", " and ", " price ", " order ", " product ", " hello "]):
        return "en"
    # Fallback: prefer VI as primary market
    return "vi"


async def _fetch_session_context(pool: Any, session_id: str) -> SessionContext:
    # MOCK IMPLEMENTATION
    ctx = SessionContext()
    # Simulate some context if needed
    return ctx


class InputProcessor:
    async def run(self, req: ChatRequest, pool: Any = None) -> ProcessedInput:
        print(f"[InputProcessor] Processing request: session={req.session_id}, msg='{req.message}'")
        cleaned = _normalize_text(req.message)
        lang = req.language or _detect_language_heuristic(cleaned)
        
        # Determine user_type:
        user_type = "guest"
        if req.metadata and "user_type" in req.metadata:
            user_type = req.metadata["user_type"]
        elif req.customer_id:
            user_type = "customer"
            
        session_ctx = await _fetch_session_context(pool, req.session_id)
        print(f"[InputProcessor] Context loaded: last_intent={session_ctx.last_messages[0]['intent'] if session_ctx.last_messages else 'None'}")

        return ProcessedInput(
            session_id=req.session_id,
            customer_id=req.customer_id,
            text=req.message,
            cleaned_text=cleaned,
            language=lang,
            user_type=user_type,
            tag=req.tag,
            session_ctx=session_ctx,
        )

In [ ]:
# --- AGENT 2: INTENT CLASSIFIER (app/agents/intent_classifier.py) ---
# from typing import Dict, List
# import re
# from app.models.agent_types import ProcessedInput, IntentResult

VI_KEYWORDS: Dict[str, List[str]] = {
    "GREETING": ["xin chào", "chào", "alo", "chào bạn", "hello", "hi", "hey", "chào buổi", "chào admin", "chào shop"],
    "PRODUCT.SEARCH": [
        "tìm", "tìm kiếm", "có không", "còn không", "có bán", "muốn mua", "giá", "sản phẩm", 
        "mua", "đặt", "kiếm", "tương tự", "giống", "cần", "shop có", "bên này có", "có hàng",
        "còn hàng", "hết hàng chưa", "còn size", "có màu", "loại", "dòng", "model"
    ],
    "PRODUCT.DETAIL": [
        "chi tiết", "thông số", "xem kỹ", "thông tin", "cụ thể", "mô tả", "spec", 
        "đặc điểm", "tính năng", "chất liệu", "kích thước", "size", "màu sắc", "xuất xứ"
    ],
    "PRODUCT.RECOMMEND": [
        "gợi ý cho tôi", "gợi ý cho mình", "gợi ý vài", "nên mua gì", "mua gì", 
        "hot trend", "bán chạy", "phổ biến", "đề xuất", "recommend me",
        "trending", "mới nhất", "best seller", "top sản phẩm", "sản phẩm tốt", "sản phẩm hay", "đáng mua nhất",
        "xem sản phẩm nào", "có sản phẩm nào", "sản phẩm nào tốt"
    ],
    "ORDER.TRACK": [
        "tra cứu", "kiểm tra", "đơn hàng", "ở đâu", "tình trạng", "tracking", "giao hàng",
        "ship", "vận chuyển", "đã giao chưa", "khi nào nhận", "bao giờ về", "đến chưa"
    ],
    "ORDER.CANCEL": ["hủy đơn", "không mua nữa", "huỷ đơn", "cancel", "không muốn", "đổi ý", "không lấy"],
    "ORDER.RETURN": [
        "đổi trả", "hoàn tiền", "bảo hành", "trả hàng", "return", "refund", "warranty",
        "đổi hàng", "lỗi", "hư", "không vừa", "sai", "không đúng"
    ],
    "CART.VIEW": ["xem giỏ", "giỏ hàng", "trong giỏ", "cart", "giỏ", "đã chọn", "đã thêm"],
    "CART.ADD": [
        "thêm vào giỏ", "bỏ vào giỏ", "mua cái này", "lấy cái này", "đặt cái này", 
        "lấy cho", "thêm 1", "đặt 1", "đặt hàng", "mua ngay", "chốt", "order",
        "cho vào giỏ", "bỏ giỏ", "add", "thêm", "bỏ", "cho tôi", "cho em",
        "mình lấy", "mình mua", "lấy luôn", "mua luôn"
    ],
    "CART.REMOVE": ["xóa khỏi giỏ", "bỏ ra", "xóa sản phẩm", "bỏ sản phẩm", "remove", "không lấy", "bỏ"],
    "CART.UPDATE": ["đổi số lượng", "thay đổi", "sửa giỏ", "update cart", "tăng", "giảm", "đổi size", "đổi màu"],
    "CHECKOUT": ["thanh toán", "checkout", "đặt luôn", "mua luôn", "pay", "trả tiền", "hoàn tất"],
    "ACCOUNT.LOGIN": ["đăng nhập", "login", "sign in", "vào tài khoản"],
    "ACCOUNT.REGISTER": ["đăng ký", "tạo tài khoản", "sign up", "register", "mở tài khoản"],
    "FAQ.SHIPPING": [
        "phí ship", "vận chuyển", "bao lâu", "giao hàng", "ship", "shipping",
        "delivery", "mất bao lâu", "khi nào nhận", "ship cod", "ship nhanh"
    ],
    "FAQ.PAYMENT": [
        "thanh toán", "chuyển khoản", "cod", "trả tiền", "payment", 
        "hình thức thanh toán", "trả góp", "ví", "momo", "zalopay", "banking"
    ],
    "FAQ.PROMO": ["khuyến mãi", "giảm giá", "sale", "voucher", "mã giảm", "coupon", "promotion", "ưu đãi"],
    "FAQ.RETURN": [
        "đổi trả", "chính sách đổi", "return policy", "hoàn tiền", "refund",
        "bảo hành", "warranty", "trả hàng", "đổi hàng", "chính sách hoàn"
    ],
    "SUPPORT.ESCALATE": [
        "nhân viên", "người thật", "không phải bot", "tư vấn", "hỗ trợ", 
        "admin", "staff", "human", "real person", "agent"
    ],
    "THANK": ["cảm ơn", "thanks", "thank you", "cám ơn", "ơn"],
    "GOODBYE": ["tạm biệt", "bye", "goodbye", "hẹn gặp lại", "đi đây"],
    # Staff / Manager Keywords
    "STAFF.CHECK_STOCK": ["check kho", "tồn kho", "kiểm kho", "số lượng tồn", "inventory", "stock"],
    "STAFF.CUSTOMER_LOOKUP": ["tìm khách", "check info khách", "thông tin khách", "lookup customer"],
    "STAFF.ORDER_HISTORY": ["lịch sử mua", "đơn cũ của khách", "khách mua gì", "order history"],
    "STAFF.CREATE_ORDER": ["tạo đơn giúp", "lên đơn", "tạo đơn", "create order", "draft order"],
    "MANAGER.REPORT_SALES": ["doanh thu", "báo cáo bán hàng", "doanh số", "sales report", "revenue"],
    "MANAGER.REPORT_CHATBOT": ["hiệu quả bot", "bot chat bao nhiêu", "thống kê bot", "chatbot stats"],
    "MANAGER.CONFIG_UPDATE": ["tắt bot", "bật bot", "chỉnh prompt", "cấu hình", "config"],
}

EN_KEYWORDS: Dict[str, List[str]] = {
    "GREETING": ["hello", "hi", "hey"],
    "PRODUCT.SEARCH": ["search", "find", "looking for", "do you have", "want to buy", "price", "product", "buy", "order"],
    "PRODUCT.DETAIL": ["detail", "specs", "info", "more info", "specification"],
    "PRODUCT.RECOMMEND": ["recommend", "suggest", "hot trend", "best seller", "popular"],
    "ORDER.TRACK": ["track", "check order", "order status", "where is", "shipping"],
    "ORDER.CANCEL": ["cancel order", "cancel"],
    "ORDER.RETURN": ["return", "refund", "warranty"],
    "CART.VIEW": ["view cart", "my cart", "shopping cart"],
    "CART.ADD": ["add to cart", "buy this", "get this", "order this"],
    "CART.REMOVE": ["remove", "delete item", "remove from cart"],
    "ACCOUNT.LOGIN": ["login", "sign in"],
    "ACCOUNT.REGISTER": ["register", "sign up"],
    "FAQ.SHIPPING": ["shipping cost", "delivery time", "shipping fee"],
    "FAQ.PAYMENT": ["payment", "cod", "bank transfer"],
    "SUPPORT.ESCALATE": ["human", "staff", "real person", "agent"],
    # Staff / Manager Keywords
    "STAFF.CHECK_STOCK": ["check stock", "inventory", "stock level"],
    "STAFF.CUSTOMER_LOOKUP": ["lookup customer", "find customer", "user info"],
    "STAFF.ORDER_HISTORY": ["customer order history", "order history"],
    "STAFF.CREATE_ORDER": ["create order", "draft order"],
    "MANAGER.REPORT_SALES": ["sales report", "revenue", "sales stats"],
    "MANAGER.REPORT_CHATBOT": ["chatbot stats", "bot performance"],
    "MANAGER.CONFIG_UPDATE": ["disable bot", "enable bot", "update config"],
}


def _match_score(text: str, phrases: List[str]) -> int:
    t = f" {text.lower()} "
    score = 0
    for p in phrases:
        if f" {p} " in t:
            score += 2 if len(p.split()) > 1 else 1
    return score


def _extract_product_query(cleaned_text: str) -> str:
    s = cleaned_text
    
    # Check if this is a context reference (not a search query)
    context_patterns = [
        r"\b(cái|sản phẩm|sp)\s+(này|kia|đó|ấy|vừa|trước)",
        r"\b(đầu tiên|thứ nhất|thứ hai|cuối cùng)",
        r"\bnó\b",
        r"\bchi tiết\b.*(?!\w+)",  # "chi tiết" without specific product name
    ]
    for pattern in context_patterns:
        if re.search(pattern, s.lower()):
            return ""  # Return empty to signal this needs context
    
    s = re.sub(r"[?,.]", " ", s) # Remove punctuation
    # remove common Vietnamese/English query anchors
    remove = [
        "tìm", "tìm kiếm", "có không", "còn không", "có bán", "muốn mua",
        "search", "find", "looking for", "do you have", "want to buy",
        "sản phẩm", "product", "mua", "đặt", "kiếm", "cho tôi", "cho anh", "cho em", "cho mình",
        "tương tự", "similar",
        "tôi", "anh", "em", "bạn", "mình", "cái", "chiếc", "là", "của",
        "giá", "bao nhiêu", "price", "cost", "how much", "tiền",
        "thông tin", "xem", "về", "info", "about",
        # Staff keywords
        "check kho", "tồn kho", "kiểm kho", "số lượng tồn",
        "check stock", "inventory", "stock level",
        "tìm khách", "check info khách", "thông tin khách", "khách hàng", "khách",
        "lookup customer", "find customer", "user info",
        "nhân viên", "quản lý", "staff", "manager"
    ]
    low = s.lower()
    for r in remove:
        low = low.replace(r, " ")
    low = re.sub(r"\s+", " ", low).strip()
    return low


class IntentClassifier:
    async def run(self, processed: ProcessedInput) -> IntentResult:
        print(f"[IntentClassifier] Classifying: '{processed.cleaned_text}'")
        text = processed.cleaned_text
        lang = processed.language or "vi"

        # Choose keyword table
        table = VI_KEYWORDS if lang == "vi" else EN_KEYWORDS

        # Compute scores
        scores: Dict[str, int] = {}
        for key, phrases in table.items():
            scores[key] = _match_score(text, phrases)
        
        # Boost PRODUCT.RECOMMEND score if recommend phrases are detected
        # This prevents "gợi ý sản phẩm" from being classified as product_inquiry
        recommend_boost_phrases = ["gợi ý cho", "đề xuất cho", "recommend me", "gợi ý vài", "gợi ý một", "nên mua gì"]
        if any(phrase in text.lower() for phrase in recommend_boost_phrases):
            if "PRODUCT.RECOMMEND" in scores:
                scores["PRODUCT.RECOMMEND"] += 5  # Strong boost to override PRODUCT.SEARCH
        
        # Boost PRODUCT.DETAIL for context reference patterns (e.g., "sản phẩm đầu tiên", "cái này")
        context_patterns = [
            r'\b(cái|sản phẩm|sp)\s+(này|kia|đó|ấy|vừa|trước)',
            r'\b(đầu tiên|thứ nhất|thứ hai|thứ ba|cuối cùng)',
            r'\bnó\b',
        ]
        for pattern in context_patterns:
            if re.search(pattern, text.lower()):
                if "PRODUCT.DETAIL" in scores:
                    scores["PRODUCT.DETAIL"] += 10  # Very strong boost to override PRODUCT.SEARCH
                break

        # Get top 3 intents for multi-intent detection
        sorted_scores = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
        top_key = sorted_scores[0][0] if sorted_scores else "UNKNOWN"
        top_score = sorted_scores[0][1] if sorted_scores else 0
        
        # Detect secondary intent (e.g., "tìm balo giá rẻ" = PRODUCT.SEARCH + PRODUCT.PRICE)
        secondary_intent = None
        if len(sorted_scores) > 1 and sorted_scores[1][1] >= 2:
            secondary_intent = sorted_scores[1][0]
        
        print(f"[IntentClassifier] Top: {top_key}({top_score}), Secondary: {secondary_intent}")

        # Map to system intents used by backend
        intent_map = {
            "GREETING": "general",
            "PRODUCT.SEARCH": "product_inquiry",
            "PRODUCT.DETAIL": "product_detail",
            "PRODUCT.RECOMMEND": "product_recommend",
            "ORDER.TRACK": "order_tracking",
            "ORDER.CANCEL": "order_cancel",
            "ORDER.RETURN": "order_return",
            "CART.VIEW": "cart_view",
            "CART.ADD": "cart_add",
            "CART.REMOVE": "cart_remove",
            "CART.UPDATE": "cart_update",
            "CHECKOUT": "checkout",
            "ACCOUNT.LOGIN": "account_login",
            "ACCOUNT.REGISTER": "account_register",
            "FAQ.SHIPPING": "faq_shipping",
            "FAQ.PAYMENT": "faq_payment",
            "FAQ.PROMO": "faq_promo",
            "FAQ.RETURN": "faq_return",
            "SUPPORT.ESCALATE": "human_escalation",
            "THANK": "thank_you",
            "GOODBYE": "goodbye",
            "STAFF.CHECK_STOCK": "staff_check_stock",
            "STAFF.CUSTOMER_LOOKUP": "staff_customer_lookup",
            "STAFF.ORDER_HISTORY": "staff_order_history",
            "STAFF.CREATE_ORDER": "staff_create_order",
            "MANAGER.REPORT_SALES": "manager_report_sales",
            "MANAGER.REPORT_CHATBOT": "manager_report_chatbot",
            "MANAGER.CONFIG_UPDATE": "manager_config_update",
        }

        mapped = intent_map.get(top_key, "general")
        sub_intent = intent_map.get(secondary_intent) if secondary_intent else None

        # Enhanced confidence scoring
        if top_score >= 4:
            conf = 0.95
        elif top_score >= 3:
            conf = 0.85
        elif top_score >= 2:
            conf = 0.70
        elif top_score >= 1:
            conf = 0.50
        else:
            conf = 0.30

        entities = {}
        
        # Enhanced entity extraction
        if mapped == "product_inquiry" or mapped == "cart_add" or mapped == "product_detail":
            q = _extract_product_query(text)
            # For product_detail, empty query means context reference
            if mapped == "product_detail" and q == "":
                entities["product_query"] = ""  # Signal to use context
            elif q and len(q) >= 2:
                entities["product_query"] = q
            
            # Extract quantity
            qty_match = re.search(r'(\d+)\s*(cái|chiếc|con|bộ|món|sản phẩm|sp)', text.lower())
            if qty_match:
                entities["quantity"] = int(qty_match.group(1))
            else:
                entities["quantity"] = 1
                
        elif mapped == "order_tracking":
            # Enhanced order ID extraction
            # Patterns: #1234, order_1234, ORD-1234, đơn 1234
            match = re.search(r'(order[_\s-]?[\w\d]+|#\d+|ord[_\s-]?\d+|đơn[\s]?\d+|\d{4,})', text.lower())
            if match:
                order_id = match.group(0).replace("#", "").replace("đơn", "").replace("order", "").replace("_", "").replace("-", "").strip()
                entities["order_id"] = order_id
                
        elif mapped == "staff_check_stock":
            q = _extract_product_query(text)
            if q:
                entities["product_query"] = q
                
        elif mapped == "staff_customer_lookup":
            # Extract customer info (email, phone, name)
            email_match = re.search(r'[\w\.-]+@[\w\.-]+\.\w+', text)
            phone_match = re.search(r'(\+?84|0)\d{9,10}', text)
            
            if email_match:
                entities["customer_email"] = email_match.group(0)
            elif phone_match:
                entities["customer_phone"] = phone_match.group(0)
            else:
                # Fallback to query extraction
                q = _extract_product_query(text)
                if q:
                    entities["customer_query"] = q
        
        print(f"[IntentClassifier] Result: intent={mapped}, sub={sub_intent}, entities={entities}")
        return IntentResult(
            intent=mapped,
            sub_intent=sub_intent,
            confidence=conf,
            entities=entities,
        )

In [ ]:
# --- AGENT 3: ORCHESTRATOR (app/agents/orchestrator.py) ---
# from typing import Optional, Tuple, Dict
# from app.models.agent_types import ActionPlan, ToolResults, ProcessedInput, IntentResult
# from app.agents.specialized.sales_agent import SalesAgent
# from app.agents.specialized.order_agent import OrderAgent
# from app.agents.specialized.staff_agent import StaffAgent
# from app.agents.specialized.manager_agent import ManagerAgent

class Orchestrator:
    def __init__(self):
        self.agents = {
            "sales": MockSpecializedAgent("sales"),
            "order": MockSpecializedAgent("order"),
            "staff": MockSpecializedAgent("staff"),
            "manager": MockSpecializedAgent("manager")
        }

    async def run(
        self,
        processed: ProcessedInput,
        intent: IntentResult,
    ) -> Tuple[ActionPlan, Optional[ToolResults]]:
        """
        Hybrid Orchestrator:
        1. Executable Tag -> Execute Tool directly
        2. Scope Tag -> Route to Specialized Agent (Scoped NLU)
        3. No Tag -> Global NLU -> Route to Agent
        """
        print(f"[Orchestrator] Routing... Tag={processed.tag}, Intent={intent.intent}")
        
        # 1. Executable Tag (e.g., "action:view_cart")
        if processed.tag and processed.tag.startswith("action:"):
            action = processed.tag.split(":", 1)[1]
            print(f"[Orchestrator] Direct action: {action}")
            
            if action == "view_cart":
                return ActionPlan(tools=["cart.view"], next_step="show_cart"), None
            elif action == "order_list":
                return ActionPlan(tools=["order.list"], next_step="show_orders"), None
            elif action == "order_track":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "reorder":
                return ActionPlan(tools=["order.reorder"], next_step="confirm_reorder"), None
            elif action == "logout":
                return ActionPlan(tools=["auth.logout"], next_step="confirm_logout"), None
            elif action == "chat_human":
                return ActionPlan(tools=["system.escalate"], next_step="escalate_chat"), None
            elif action == "my_reviews":
                return ActionPlan(tools=["product.reviews"], next_step="show_reviews"), None
            elif action == "check_stock":
                return ActionPlan(tools=[], next_step="ask_product_name"), None # Need product name
            elif action == "check_price":
                return ActionPlan(tools=[], next_step="ask_product_name"), None # Need product name
            elif action == "staff_lookup_order":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "staff_customer_lookup":
                return ActionPlan(tools=[], next_step="ask_customer_info"), None
            elif action == "update_order_status":
                return ActionPlan(tools=[], next_step="ask_order_id_status"), None
            elif action == "print_shipping_label":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "sales_report":
                return ActionPlan(tools=["manager.report_sales"], next_step="show_report"), None
            elif action == "top_products":
                return ActionPlan(tools=["manager.top_products"], next_step="show_top_products"), None
            elif action == "customer_analytics":
                return ActionPlan(tools=["manager.customer_analytics"], next_step="show_analytics"), None
            elif action.startswith("cart.add_item"):
                try:
                    params = {}
                    if "?" in action:
                        query = action.split("?", 1)[1]
                        for pair in query.split("&"):
                            if "=" in pair:
                                k, v = pair.split("=", 1)
                                params[k] = v
                    
                    intent.entities["product_id"] = params.get("product_id")
                    intent.entities["variant_id"] = params.get("variant_id")
                    intent.entities["quantity"] = 1
                    
                    return ActionPlan(tools=["cart.add"], next_step="confirm_add_to_cart"), None
                except Exception as e:
                    print(f"Error parsing cart.add_item tag: {e}")
                    return ActionPlan(tools=[], next_step="error"), None
            
            # Add more direct actions here
        
        # 2. Scope Tag (e.g., "scope:order")
        if processed.tag and processed.tag.startswith("scope:"):
            scope = processed.tag.split(":", 1)[1]
            agent = self.agents.get(scope)
            if agent:
                print(f"[Orchestrator] Scoped routing to: {scope}")
                # Agent performs its own scoped NLU if intent is generic/unknown or we want to re-evaluate
                # For now, we pass None as intent to force agent to re-classify within scope
                return await agent.process(processed, intent=None)
        
        # 3. Implicit Routing (Global NLU)
        # We use the intent passed from the global classifier to decide which agent to call
        target_agent = self._route_by_intent(intent.intent)
        if target_agent:
            print(f"[Orchestrator] Implicit routing to: {target_agent.name}")
            return await target_agent.process(processed, intent)
            
        # Default fallback
        print("[Orchestrator] No agent found, fallback.")
        return ActionPlan(tools=[], next_step=None), None

    def _route_by_intent(self, intent_name: str):
        """Route intent to appropriate specialized agent"""
        for agent in self.agents.values():
            if agent.can_handle(intent_name):
                return agent
        return self.agents["sales"] # Default to sales if unknown
    
    def get_agent_suggestions(self, intent: str) -> list:
        """Get suggested actions based on intent - for better UX"""
        suggestions = {
            "product_inquiry": [
                {"label": "Xem giỏ hàng", "action": "action:view_cart"},
                {"label": "Sản phẩm gợi ý", "action": "action:my_recommendations"},
            ],
            "cart_view": [
                {"label": "Thanh toán ngay", "action": "action:checkout"},
                {"label": "Tiếp tục mua sắm", "action": "action:continue_shopping"},
            ],
            "order_tracking": [
                {"label": "Đơn hàng của tôi", "action": "action:order_list"},
                {"label": "Liên hệ hỗ trợ", "action": "action:chat_human"},
            ],
            "general": [
                {"label": "Tìm sản phẩm", "action": "action:product_search"},
                {"label": "Xem giỏ hàng", "action": "action:view_cart"},
                {"label": "Kiểm tra đơn hàng", "action": "action:order_list"},
            ],
        }
        return suggestions.get(intent, suggestions["general"])

In [ ]:
# --- AGENT 4: EXECUTOR (app/agents/executor.py) ---
# import logging
# from typing import Optional, Dict, Any
# from app.models.agent_types import ProcessedInput, IntentResult, ActionPlan, ToolResults
# from app.tools import ...

def _merge_tool_result(results: ToolResults, tool_result: Any):
    if tool_result is None:
        return
    if isinstance(tool_result, ToolResults):
        results.ok = results.ok and tool_result.ok
        if tool_result.errors:
            results.errors.extend(tool_result.errors)
        if tool_result.timings_ms:
            results.timings_ms.update(tool_result.timings_ms)
        results.data = tool_result.data
    else:
        results.data = tool_result

class Executor:
    async def run(
        self, 
        processed: ProcessedInput, 
        intent: IntentResult, 
        plan: ActionPlan
    ) -> Optional[ToolResults]:
        if not plan.tools:
            return None

        results = ToolResults(ok=True)
        
        for tool_name in plan.tools:
            try:
                logger.info(f"[Executor] Running tool: {tool_name}")
                
                # Cart Tools
                if tool_name == "cart.view":
                    if processed.session_ctx.cart_id:
                        res = await cart_tools.view_cart(processed.session_ctx.cart_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing cart_id")
                elif tool_name == "cart.add":
                    # Extract product and quantity from entities
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    qty = intent.entities.get("quantity", 1)
                    variant_id = intent.entities.get("variant_id")
                    
                    if processed.session_ctx.cart_id:
                        if variant_id:
                            # If we have explicit variant_id (e.g. from UI action), use it directly
                            res = await cart_tools.add_to_cart(processed.session_ctx.cart_id, variant_id, qty)
                        else:
                            # Otherwise try smart add (search -> add)
                            res = await cart_tools.add_to_cart_smart(processed.session_ctx.cart_id, query, qty)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing cart_id")
                
                # Product Tools
                elif tool_name == "product.search":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await product_tools.search_products(query)
                    _merge_tool_result(results, res)
                elif tool_name == "product.recommend":
                    # The tool currently only accepts limit, not customer_id
                    res = await product_recommend_tool.recommend_products(limit=5)
                    _merge_tool_result(results, res)
                elif tool_name == "product.detail":
                    # Try to get product_id from entities first
                    product_id = intent.entities.get("product_id")
                    
                    # If no product_id in entities, try to get from context (last products shown)
                    if not product_id and processed.session_ctx.last_product_ids:
                        product_id = processed.session_ctx.last_product_ids[0]
                        logger.info(f"[Executor] Using context product_id: {product_id}")
                    
                    if product_id:
                        res = await product_tools.get_product_details(product_id)
                        _merge_tool_result(results, res)
                    else:
                        # Fallback: try to search if we have a query
                        # Extract product name from text (remove "chi tiết", "xem", etc)
                        import re
                        query = intent.entities.get("product_query")
                        if not query:
                            # Try to extract product name from cleaned text
                            text_lower = processed.cleaned_text.lower()
                            # Remove common phrases (order matters - remove longer phrases first)
                            for phrase in ["cho tôi xem chi tiết", "cho tôi xem", "xem chi tiết", "chi tiết của", "chi tiết", "xem", "tôi muốn", "muốn"]:
                                text_lower = text_lower.replace(phrase, "")
                            query = text_lower.strip()
                        
                        if query and len(query) >= 2:
                            logger.info(f"[Executor] Searching for product: '{query}'")
                            res = await product_tools.search_products(query, limit=1)
                            _merge_tool_result(results, res)
                            # Store search query in entities for response_generator
                            intent.entities["product_query"] = query
                        else:
                            results.ok = False
                            results.errors.append("Missing product_id or query")
                            # Still store a placeholder for template
                            intent.entities["product_query"] = "sản phẩm bạn cần"

                # Order Tools
                elif tool_name == "order.track":
                    order_id = intent.entities.get("order_id")
                    email = intent.entities.get("email") # Might need to ask user
                    if order_id:
                        res = await order_tools.lookup_order(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "order.list":
                    if processed.customer_id:
                        res = await order_tools.list_orders(processed.customer_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "order.reorder":
                    # Assuming we reorder the last order or specific order
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await order_tools.reorder(order_id)
                        _merge_tool_result(results, res)
                    else:
                        # Maybe list orders to pick?
                        results.ok = False
                        results.errors.append("Missing order_id")

                # Staff Tools
                elif tool_name == "staff.check_stock":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await staff_tools.check_stock(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.check_price":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await staff_tools.check_price(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.customer_lookup":
                    query = intent.entities.get("customer_query", processed.cleaned_text)
                    res = await staff_tools.lookup_customer(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.order_history":
                    customer_id = intent.entities.get("customer_id")
                    if customer_id:
                        res = await staff_tools.get_customer_order_history(customer_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "staff.create_order":
                    if processed.customer_id:
                        res = await staff_tools.create_draft_order(processed.customer_id, [])
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "staff.lookup_order":
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await staff_tools.lookup_order(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "staff.update_order":
                    order_id = intent.entities.get("order_id")
                    status = intent.entities.get("status", "processing")
                    if order_id:
                        res = await staff_tools.update_order_status(order_id, status)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "staff.print_label":
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await staff_tools.print_shipping_label(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                
                # Manager Tools
                elif tool_name == "manager.report_sales":
                    res = await report_tools.get_sales_report()
                    _merge_tool_result(results, res)
                elif tool_name == "manager.top_products":
                    res = await report_tools.get_top_products()
                    _merge_tool_result(results, res)
                elif tool_name == "manager.customer_analytics":
                    res = await report_tools.get_customer_analytics()
                    _merge_tool_result(results, res)

                # System/Auth Tools
                elif tool_name == "system.escalate":
                    # Just a marker, maybe log it
                    results.data = {"status": "escalated", "message": "Connecting to human agent..."}
                elif tool_name == "auth.logout":
                    # In a real app, this might invalidate session
                    results.data = {"status": "logged_out", "message": "You have been logged out."}
                
                # Product Tools (Extra)
                elif tool_name == "product.reviews":
                    # Mock
                    results.data = {"reviews": []}

                else:
                    logger.warning(f"[Executor] Unknown tool: {tool_name}")
                    results.errors.append(f"Unknown tool: {tool_name}")

            except Exception as e:
                logger.error(f"[Executor] Error running {tool_name}: {e}")
                results.ok = False
                results.errors.append(str(e))
        
        return results

In [ ]:
# --- AGENT 5: RESPONSE GENERATOR (app/agents/response_generator.py) ---
# import os
# import yaml
# import json
# import logging
# from typing import List, Dict, Any, Optional
# from openai import AsyncOpenAI
# from openai.types.chat import ChatCompletionSystemMessageParam, ChatCompletionUserMessageParam
# from app.config import GOOGLE_API_KEY, GEMINI_BASE_URL, GEMINI_MODEL, SYSTEM_PROMPT_VI
# from app.models.api_models import ChatResponse, QuickReply, ProductInfo
# from app.models.agent_types import ProcessedInput, IntentResult, ActionPlan, ToolResults
# from app.agents.response_templates import get_response_template

class ResponseGenerator:
    def __init__(self):
        if GOOGLE_API_KEY == "YOUR_API_KEY_HERE":
            self.client = None
        else:
            from openai import AsyncOpenAI
            self.client = AsyncOpenAI(api_key=GOOGLE_API_KEY, base_url=GEMINI_BASE_URL)
        self.context_config = CONTEXT_CONFIG

    def _load_context_config(self):
        return CONTEXT_CONFIG

    async def run(
        self, 
        processed: ProcessedInput, 
        intent: IntentResult, 
        plan: ActionPlan, 
        tool_res: Optional[ToolResults]
    ) -> ChatResponse:
        
        # 1. Generate Text Response
        response_text = await self._generate_llm_response(processed, intent, tool_res)
        
        # 2. Generate Quick Replies (Options)
        quick_replies = self._generate_quick_replies(processed)
        
        # 3. Format Products if available
        products = []
        if tool_res and tool_res.ok and isinstance(tool_res.data, list):
            for item in tool_res.data:
                # Try to map to ProductInfo
                try:
                    # If item is already ProductInfo, append directly
                    if isinstance(item, ProductInfo):
                        products.append(item)
                    # Handle if item is dict
                    elif isinstance(item, dict):
                        # Filter keys that match ProductInfo
                        valid_keys = ProductInfo.model_fields.keys()
                        filtered = {k: v for k, v in item.items() if k in valid_keys}
                        # Ensure required fields
                        if "id" in filtered and "title" in filtered and "handle" in filtered:
                            products.append(ProductInfo(**filtered))
                    # Handle if item is object
                    elif hasattr(item, "id") and hasattr(item, "title"):
                        # Manually map variants if they exist in the object but weren't automatically mapped
                        variants = getattr(item, "variants", [])
                        # If variants are already ProductVariantInfo objects, use them.
                        # If they are dicts, convert them.
                        mapped_variants = []
                        for v in variants:
                            if isinstance(v, dict):
                                mapped_variants.append(v) # Pydantic will handle dict -> model
                            else:
                                mapped_variants.append(v)

                        products.append(ProductInfo(
                            id=item.id,
                            title=item.title,
                            handle=item.handle,
                            thumbnail=getattr(item, "thumbnail", None),
                            price=getattr(item, "price", None),
                            currency_code=getattr(item, "currency_code", None),
                            variants=mapped_variants
                        ))
                except Exception as e:
                    logger.warning(f"Failed to map product item: {e}")
        
        # Add product-specific quick replies if products found
        if products:
            # Add "Buy Now" button first
            quick_replies.insert(0, QuickReply(label="Mua ngay", value=f"Đặt hàng {products[0].title}"))
            # Add "View Details" button second
            quick_replies.insert(1, QuickReply(label="Xem chi tiết", value=f"Chi tiết {products[0].title}"))

        return ChatResponse(
            response=response_text,
            session_id=processed.session_id,
            products=products,
            quick_replies=quick_replies,
            metadata={"intent": intent.intent}
        )

    async def _generate_llm_response(self, processed, intent, tool_res) -> str:
        # Try template-based response first for common cases
        template_context = {}
        
        print(f"[ResponseGen] intent={intent.intent}, tool_res={'exists' if tool_res else 'None'}, tool_res.ok={tool_res.ok if tool_res else 'N/A'}, data_len={len(tool_res.data) if tool_res and tool_res.ok else 0}")
        
        # PRIORITY 1: Handle FAQ intents with templates (avoid LLM hallucination)
        if intent.intent in ["faq_shipping", "faq_payment", "faq_return", "faq_promo"]:
            print(f"[ResponseGen] Using FAQ template for {intent.intent}")
            return get_response_template(intent.intent)
        
        # PRIORITY 2: Handle thank_you/goodbye
        if intent.intent == "thank_you":
            return get_response_template("thank_you")
        elif intent.intent == "goodbye":
            return get_response_template("goodbye")
        
        if tool_res:
            if tool_res.ok and isinstance(tool_res.data, list):
                template_context['count'] = len(tool_res.data)
                if len(tool_res.data) > 0:
                    first_item = tool_res.data[0]
                    if isinstance(first_item, dict):
                        template_context['product'] = first_item.get('title', '')
                    elif hasattr(first_item, 'title'):
                        template_context['product'] = first_item.title
            else:
                # tool_res.ok = False means tool failed/no results
                template_context['count'] = 0
        
        # PRIORITY 3: Handle product intents with templates (even without tool_res)
        if intent.intent == "general" and (not tool_res or not tool_res.ok):
            print("[ResponseGen] Using greeting template")
            return get_response_template("greeting")
        elif intent.intent in ["product_inquiry", "product_detail"]:
            # Get product query from entities (may be set by executor)
            query = intent.entities.get("product_query")
            if query:
                template_context['query'] = query
            else:
                # Fallback: extract from processed text
                template_context['query'] = "sản phẩm bạn cần"
            
            if template_context.get('count', 0) > 0:
                print(f"[ResponseGen] Using product_found template, count={template_context['count']}")
                return get_response_template("product_found", template_context)
            else:
                # Product not found - use template to avoid LLM hallucination
                print(f"[ResponseGen] Using product_not_found template, query={template_context.get('query')}")
                return get_response_template("product_not_found", template_context)
        elif intent.intent == "product_recommend":
            if tool_res.ok and template_context.get('count', 0) > 0:
                return get_response_template("product_recommend")
            else:
                return "Xin lỗi, shop đang cập nhật sản phẩm. Bạn quay lại sau nhé!"
        elif intent.intent == "cart_add":
            if tool_res.ok:
                return get_response_template("cart_added", template_context)
            else:
                return "Xin lỗi, tôi cần biết sản phẩm nào bạn muốn thêm vào giỏ hàng."
        
        # PRIORITY 4: Fallback to LLM only for complex cases
        # But prevent hallucination by checking tool_res.ok
        if tool_res and not tool_res.ok:
            # Tool failed - don't let LLM make up data
            print(f"[ResponseGen] Tool failed, using error template")
            if intent.intent in ["product_inquiry", "product_detail"]:
                return get_response_template("product_not_found", template_context)
            return "Xin lỗi, tôi không tìm thấy thông tin bạn cần. Bạn có thể hỏi lại được không?"
        
        # Construct prompt for LLM (only when tool succeeded or no tool needed)
        print(f"[ResponseGen] Falling through to LLM generation")
        context_str = ""
        if tool_res:
            if tool_res.ok:
                # Truncate data if too large
                data_str = json.dumps(tool_res.data, ensure_ascii=False, default=str)
                if len(data_str) > 3000: # Increased limit for better context
                    data_str = data_str[:3000] + "...(truncated)"
                context_str = f"Tool Result: {data_str}"
            else:
                context_str = f"Tool Error: {tool_res.errors}"
        
        # Add currency hint if available in tool result (e.g. from product search)
        currency_hint = ""
        if tool_res and tool_res.ok and isinstance(tool_res.data, list) and len(tool_res.data) > 0:
            first_item = tool_res.data[0]
            if isinstance(first_item, dict) and "currency_code" in first_item:
                 currency_hint = f"\nLưu ý: Đơn vị tiền tệ đang sử dụng là {first_item.get('currency_code', 'N/A').upper()}."
            elif hasattr(first_item, "currency_code"):
                 currency_hint = f"\nLưu ý: Đơn vị tiền tệ đang sử dụng là {getattr(first_item, 'currency_code', 'N/A').upper()}."

        messages = [
            {"role": "system", "content": SYSTEM_PROMPT_VI + currency_hint},
            {"role": "user", "content": f"User Input: {processed.text}\nIntent: {intent.intent}\nContext: {context_str}"}
        ]
        
        if self.client:
            try:
                completion = await self.client.chat.completions.create(
                    model=GEMINI_MODEL,
                    messages=messages,
                    temperature=0.7
                )
                return completion.choices[0].message.content or ""
            except Exception as e:
                logger.error(f"LLM Error: {e}")
                return get_response_template("error_generic")
        else:
            return f"[MOCK LLM RESPONSE] Based on {intent.intent} and data: {tool_res.data if tool_res else 'None'}"

    def _generate_quick_replies(self, processed) -> List[QuickReply]:
        nodes = self.context_config.get("nodes", [])
        
        # Helper to find node by ID recursively
        def find_node(node_list, target_id):
            for node in node_list:
                if node.get("id") == target_id:
                    return node
                if "children" in node:
                    found = find_node(node["children"], target_id)
                    if found: return found
            return None

        # Determine current context node
        current_node = None
        if processed.tag:
            # If tag is like "action:view_cart", we might not find a node if it's a leaf.
            # But if tag is a group ID, we might.
            # Let's try to find by ID first (assuming tag might be ID)
            current_node = find_node(nodes, processed.tag)
        
        # If no specific node found, default to user root
        if not current_node:
            user_root_id = f"{processed.user_type}_root"
            current_node = find_node(nodes, user_root_id)

        if not current_node:
            return []

        options = []
        # Return children of the current node
        for child in current_node.get("children", []):
            # Determine value: prefer value, then tag, then id
            val = child.get("value") or child.get("tag") or child.get("id")
            options.append(QuickReply(
                label=child["label"],
                value=val,
                metadata={"type": child.get("type")}
            ))
            
        return options

In [ ]:
# --- DATA LOADING ---
# Upload 'multi_agent_test_cases_full.csv' to Colab files
try:
    df = pd.read_csv('multi_agent_test_cases_full.csv')
    print(f"Loaded {len(df)} test cases.")
except FileNotFoundError:
    print("File not found. Creating sample data...")
    data = {
        "id": [1, 2, 3],
        "user_input": ["Xin chào shop", "Tìm balo JanSport màu đen", "Kiểm tra đơn hàng #123"],
        "expected_intent_classifier_intent": ["general", "product_inquiry", "order_tracking"],
        "expected_executor_tools_called": ["", "product.search", "order.track"]
    }
    df = pd.DataFrame(data)
    print("Created sample data.")

df.head()

In [ ]:
# --- EVALUATION LOOP ---

async def evaluate_row(row):
    # Initialize Agents
    input_processor = InputProcessor()
    intent_classifier = IntentClassifier()
    orchestrator = Orchestrator()
    executor = Executor()
    response_generator = ResponseGenerator()

    # 1. Input
    req = ChatRequest(session_id=f"sess_{row['id']}", message=row['user_input'])
    processed = await input_processor.run(req)
    
    # 2. Intent
    intent_res = await intent_classifier.run(processed)
    
    # 3. Orchestrator
    plan, _ = await orchestrator.run(processed, intent_res)
    
    # 4. Executor
    tool_res = await executor.run(processed, intent_res, plan)
    
    # 5. Response
    response = await response_generator.run(processed, intent_res, plan, tool_res)
    
    # --- CHECKS ---
    results = {
        "id": row['id'],
        "input": row['user_input'],
        "intent_actual": intent_res.intent,
        "intent_expected": row.get('expected_intent_classifier_intent'),
        "intent_match": intent_res.intent == row.get('expected_intent_classifier_intent'),
        
        "tools_actual": ",".join(plan.tools),
        "tools_expected": row.get('expected_executor_tools_called', ""),
        # Simple check: is expected tool in actual tools?
        "tools_match": str(row.get('expected_executor_tools_called', "")) in ",".join(plan.tools) if pd.notna(row.get('expected_executor_tools_called')) else True,
        
        "response_preview": response.response[:50] + "..."
    }
    return results

# Run Evaluation
results_list = []
print("Starting evaluation...")
for index, row in df.iterrows():
    res = await evaluate_row(row)
    results_list.append(res)
    print(f"Row {row['id']}: Intent Match={res['intent_match']}, Tools Match={res['tools_match']}")

results_df = pd.DataFrame(results_list)

In [ ]:
# --- SUMMARY REPORT ---
print("\n--- EVALUATION SUMMARY ---")
total = len(results_df)
intent_acc = results_df['intent_match'].mean() * 100
tools_acc = results_df['tools_match'].mean() * 100

print(f"Total Test Cases: {total}")
print(f"Intent Accuracy: {intent_acc:.2f}%")
print(f"Tool Selection Accuracy: {tools_acc:.2f}%")

print("\n--- FAILED CASES ---")
failed = results_df[~results_df['intent_match'] | ~results_df['tools_match']]
if not failed.empty:
    display(failed[['id', 'input', 'intent_actual', 'intent_expected', 'tools_actual', 'tools_expected']])
else:
    print("All cases passed!")

In [ ]:
!pip install pydantic openai pandas python-dotenv nest_asyncio pyyaml

In [ ]:
import os
import json
import re
import yaml
import asyncio
import logging
import pandas as pd
import nest_asyncio
from typing import List, Dict, Any, Optional, Tuple
from pydantic import BaseModel, Field
from datetime import datetime

# Apply nest_asyncio to allow async loops in notebook
nest_asyncio.apply()

# Configure Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("Evaluation")

In [ ]:
# --- CONFIGURATION ---
# Replace with your actual API Key
GOOGLE_API_KEY = "YOUR_API_KEY_HERE" 
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
GEMINI_MODEL = "gemini-1.5-flash"

SYSTEM_PROMPT_VI = """
Bạn là trợ lý ảo AI chuyên nghiệp của cửa hàng JanSport Việt Nam.
Nhiệm vụ của bạn là hỗ trợ khách hàng tìm kiếm sản phẩm, kiểm tra đơn hàng và giải đáp thắc mắc.
Luôn trả lời bằng tiếng Việt, giọng điệu thân thiện, nhiệt tình và chuyên nghiệp.
"""

# Mock Context Config (from context_config.yaml)
CONTEXT_CONFIG = {
    "nodes": [
        {
            "id": "guest_root",
            "label": "Guest Root",
            "children": [
                {"label": "Tìm sản phẩm", "value": "action:product_search"},
                {"label": "Tra cứu đơn hàng", "value": "action:order_track"}
            ]
        },
        {
            "id": "customer_root",
            "label": "Customer Root",
            "children": [
                {"label": "Đơn hàng của tôi", "value": "action:order_list"},
                {"label": "Giỏ hàng", "value": "action:view_cart"}
            ]
        }
    ]
}

In [ ]:
# --- DATA MODELS (app/models/agent_types.py) ---
# Copied exactly from source

class SessionContext(BaseModel):
    last_messages: List[Dict[str, Any]] = Field(default_factory=list)
    cart_id: Optional[str] = None
    last_product_ids: List[str] = Field(default_factory=list)


class ProcessedInput(BaseModel):
    session_id: str
    customer_id: Optional[str] = None
    text: str
    cleaned_text: str
    language: str = "vi"  # "vi" | "en"
    user_type: str = "guest"  # "guest" | "customer"
    tag: Optional[str] = None
    session_ctx: SessionContext = Field(default_factory=SessionContext)


class IntentResult(BaseModel):
    intent: str
    sub_intent: Optional[str] = None
    confidence: float = 0.0
    entities: Dict[str, Any] = Field(default_factory=dict)


class ActionPlan(BaseModel):
    tools: List[str] = Field(default_factory=list)
    required_data: List[str] = Field(default_factory=list)
    next_step: Optional[str] = None
    escalate: bool = False


class ToolResults(BaseModel):
    ok: bool
    data: Any = None
    errors: List[str] = Field(default_factory=list)
    timings_ms: Dict[str, int] = Field(default_factory=dict)

# Additional Models needed for Response Generator
class ProductInfo(BaseModel):
    id: str
    title: str
    handle: str
    thumbnail: Optional[str] = None
    price: Optional[int] = None
    currency_code: Optional[str] = None
    variants: List[Dict[str, Any]] = Field(default_factory=list)

class QuickReply(BaseModel):
    label: str
    value: str
    metadata: Optional[Dict[str, Any]] = None

class ChatResponse(BaseModel):
    response: str
    session_id: str
    products: List[ProductInfo] = Field(default_factory=list)
    quick_replies: List[QuickReply] = Field(default_factory=list)
    metadata: Dict[str, Any] = Field(default_factory=dict)

class ChatRequest(BaseModel):
    session_id: str
    message: str
    customer_id: Optional[str] = None
    language: Optional[str] = None
    metadata: Optional[Dict[str, Any]] = None
    tag: Optional[str] = None

In [ ]:
# --- RESPONSE TEMPLATES (app/agents/response_templates.py) ---
# Copied exactly from source

RESPONSE_TEMPLATES = {
    # Greetings
    "greeting": [
        "Xin chào! Tôi là trợ lý ảo của shop. Tôi có thể giúp gì cho bạn hôm nay? 😊",
        "Chào bạn! Rất vui được hỗ trợ bạn. Bạn cần tìm sản phẩm gì không?",
        "Hello! Chào mừng bạn đến với shop. Tôi có thể tư vấn gì cho bạn?"
    ],
    
    # Product Search Success
    "product_found": [
        "Mình tìm thấy {count} sản phẩm phù hợp với '{query}' ạ:",
        "Đây là {count} sản phẩm mình tìm được cho bạn về '{query}':",
        "Shop có {count} sản phẩm '{query}' như sau:"
    ],
    
    # Product Not Found
    "product_not_found": [
        "Rất tiếc, shop không tìm thấy sản phẩm '{query}' bạn cần 😔\nBạn có thể thử tìm kiếm sản phẩm tương tự không?",
        "Hiện tại shop chưa có sản phẩm '{query}' ạ. Bạn muốn xem sản phẩm khác không?",
        "Mình không tìm thấy '{query}' trong kho. Bạn có thể mô tả chi tiết hơn hoặc xem sản phẩm tương tự?"
    ],
    
    # Product Recommend
    "product_recommend": [
        "🌟 Đây là một số sản phẩm hot mà bạn có thể thích:",
        "Mình gợi ý cho bạn vài sản phẩm đang bán chạy nha:",
        "Đây là top sản phẩm được yêu thích nhất của shop:"
    ],
    
    # Cart Actions
    "cart_added": [
        "Đã thêm {product} vào giỏ hàng! Bạn muốn tiếp tục mua sắm hay thanh toán luôn?",
        "OK! Đã cho {product} vào giỏ rồi nhé. Mua thêm gì nữa không?"
    ],
    
    "cart_view_empty": [
        "Giỏ hàng của bạn đang trống. Tìm sản phẩm ngay nhé! 🛍️",
        "Bạn chưa có sản phẩm nào trong giỏ. Cùng khám phá shop nhé!"
    ],
    
    "cart_view_has_items": [
        "Giỏ hàng của bạn có {count} sản phẩm:",
        "Bạn đang có {count} món trong giỏ:"
    ],
    
    # Order Tracking
    "order_found": [
        "Đơn hàng #{order_id} của bạn:\nTrạng thái: {status}\nDự kiến giao: {delivery_date}",
        "Mình check đơn #{order_id} rồi:\n📦 {status}\n🚚 Giao dự kiến: {delivery_date}"
    ],
    
    "order_not_found": [
        "Không tìm thấy đơn hàng #{order_id}. Bạn kiểm tra lại mã đơn giúp mình nhé!",
        "Mình không thấy đơn #{order_id} trong hệ thống. Bạn có chắc mã đúng không?"
    ],
    
    # FAQ
    "faq_shipping": [
        "🚚 Thông tin vận chuyển:\n- Nội thành Hà Nội/HCM: 2-3 ngày, phí 30.000₫\n- Ngoại thành: 3-5 ngày, phí 50.000₫\n- Miễn phí ship cho đơn hàng trên 500.000₫\n- Hỗ trợ COD toàn quốc",
        "Shop ship nhanh trong 2-3 ngày nội thành nha! Phí 30k, FREE SHIP cho đơn trên 500k. Bạn có thể thanh toán COD khi nhận hàng ạ!",
        "Vận chuyển:\n• Giao hàng toàn quốc\n• Nội thành: 2-3 ngày (30k)\n• Ngoại thành: 3-5 ngày (50k)\n• FREE SHIP đơn > 500k\n• Hỗ trợ COD"
    ],
    
    "faq_return": [
        "🔄 Chính sách đổi trả:\n- Đổi trả miễn phí trong 7 ngày\n- Sản phẩm chưa sử dụng, còn nguyên tem mác\n- Miễn phí đổi size/màu (nếu còn hàng)\n- Hoàn tiền trong 3-5 ngày làm việc",
        "Bạn được đổi trả miễn phí trong 7 ngày nếu sản phẩm còn nguyên tem mác nha! Đổi size/màu miễn phí luôn ạ."
    ],
    
    "faq_payment": [
        "💳 Các hình thức thanh toán:\n- COD (tiền mặt khi nhận hàng)\n- Chuyển khoản\n- Thẻ tín dụng/ghi nợ\n- Ví điện tử (Momo, ZaloPay)",
        "Bạn có thể thanh toán bằng COD, chuyển khoản, thẻ, hoặc ví Momo/ZaloPay nha!"
    ],
    
    "faq_promo": [
        "🎁 Khuyến mãi hiện tại:\n- Giảm 10% đơn đầu tiên\n- Free ship đơn >500k\n- Tích điểm đổi quà",
        "Bạn mới có ưu đãi giảm 10% đơn đầu + freeship đơn >500k đó! 🎉"
    ],
    
    # Thank you & Goodbye
    "thank_you": [
        "Không có gì! Rất vui được hỗ trợ bạn. Cần gì cứ gọi mình nhé! 😊",
        "Hehe, có gì đâu! Cần gì cứ nhắn mình nha!",
        "Luôn sẵn sàng giúp bạn! Mua sắm vui vẻ nhé! 🛍️"
    ],
    
    "goodbye": [
        "Tạm biệt! Hẹn gặp lại bạn sớm. Chúc bạn một ngày tốt lành! 👋",
        "Bye bye! Nhớ ghé shop thường xuyên nha! 😊",
        "Hẹn gặp lại! Mua sắm vui vẻ nhé! 🛍️"
    ],
    
    # Errors
    "error_generic": [
        "Xin lỗi, có lỗi xảy ra. Bạn có thể thử lại không?",
        "Ối, có gì đó không ổn. Mình thử lại nhé!"
    ],
    
    "error_no_products": [
        "Shop đang cập nhật sản phẩm. Bạn quay lại sau nhé!",
        "Hiện tại sản phẩm đang hết hàng. Mình sẽ thông báo khi có hàng!"
    ],
    
    # Staff/Manager
    "staff_success": [
        "Đã xử lý xong! Kết quả:",
        "Done! Thông tin bạn cần:"
    ],
    
    "manager_report": [
        "📊 Báo cáo {report_type}:\n{data}",
        "Đây là thống kê {report_type} bạn yêu cầu:\n{data}"
    ],
    
    # Clarification needed
    "need_clarification": [
        "Mình chưa hiểu rõ lắm. Bạn có thể nói rõ hơn được không?",
        "Hmm, bạn muốn tìm gì nhỉ? Mô tả chi tiết giúp mình nha!",
        "Bạn cần tìm sản phẩm gì? Cho mình biết thêm thông tin nhé!"
    ],
    
    "ask_product_details": [
        "Bạn muốn tìm loại nào? (màu, size, giá...)",
        "Cho mình biết thêm chi tiết: màu sắc, kích thước, giá cả?"
    ],
    
    "ask_order_id": [
        "Bạn cho mình mã đơn hàng để check nhé! (VD: #1234 hoặc ORD-5678)",
        "Mã đơn hàng của bạn là gì ạ?"
    ]
}

def get_response_template(intent: str, context: Optional[dict] = None) -> str:
    """Get appropriate response template based on intent and context"""
    import random
    
    context = context or {}
    templates = RESPONSE_TEMPLATES.get(intent, RESPONSE_TEMPLATES["need_clarification"])
    
    # Select random template for variety
    template = random.choice(templates)
    
    # Format with context if available
    try:
        if context:
            template = template.format(**context)
    except KeyError:
        pass  # Use template as-is if formatting fails
    
    return template

In [ ]:
# --- MOCKS FOR EXTERNAL DEPENDENCIES ---

# Mock Tools to simulate Medusa/Redis calls
class MockTools:
    @staticmethod
    async def search_products(query, limit=5):
        return ToolResults(ok=True, data=[
            {"id": "p_1", "title": f"JanSport SuperBreak - {query}", "handle": "superbreak", "price": 500000, "currency_code": "vnd"},
            {"id": "p_2", "title": f"JanSport Big Student - {query}", "handle": "big-student", "price": 800000, "currency_code": "vnd"}
        ])
    
    @staticmethod
    async def get_product_details(product_id):
        return ToolResults(ok=True, data={"id": product_id, "title": "JanSport Detail", "description": "Great bag", "price": 500000, "currency_code": "vnd"})

    @staticmethod
    async def recommend_products(limit=5):
        return ToolResults(ok=True, data=[{"id": "rec_1", "title": "Trending Bag", "handle": "trending", "price": 600000, "currency_code": "vnd"}])

    @staticmethod
    async def view_cart(cart_id):
        return ToolResults(ok=True, data={"id": cart_id, "items": []})

    @staticmethod
    async def add_to_cart(cart_id, variant_id, qty):
        return ToolResults(ok=True, data={"cart_id": cart_id, "added": True})
        
    @staticmethod
    async def add_to_cart_smart(cart_id, query, qty):
        return ToolResults(ok=True, data={"cart_id": cart_id, "added": True, "product": query})

    @staticmethod
    async def lookup_order(order_id):
        if order_id == "123":
            return ToolResults(ok=True, data={"id": "123", "status": "shipped", "delivery_date": "2023-12-25"})
        return ToolResults(ok=False, errors=["Order not found"])
    
    @staticmethod
    async def list_orders(customer_id):
        return ToolResults(ok=True, data=[{"id": "123", "status": "shipped"}])

    # Staff/Manager Mocks
    @staticmethod
    async def check_stock(query): return ToolResults(ok=True, data={"product": query, "stock": 10})
    @staticmethod
    async def check_price(query): return ToolResults(ok=True, data={"product": query, "price": 500000})
    @staticmethod
    async def lookup_customer(query): return ToolResults(ok=True, data={"id": "cust_1", "name": "Test User"})
    @staticmethod
    async def get_customer_order_history(cid): return ToolResults(ok=True, data=[])
    @staticmethod
    async def create_draft_order(cid, items): return ToolResults(ok=True, data={"id": "draft_1"})
    @staticmethod
    async def update_order_status(oid, status): return ToolResults(ok=True, data={"id": oid, "status": status})
    @staticmethod
    async def print_shipping_label(oid): return ToolResults(ok=True, data={"url": "http://label.pdf"})
    @staticmethod
    async def get_sales_report(): return ToolResults(ok=True, data={"total_sales": 10000000})
    @staticmethod
    async def get_top_products(): return ToolResults(ok=True, data=[{"title": "Bag A", "sales": 100}])
    @staticmethod
    async def get_customer_analytics(): return ToolResults(ok=True, data={"new_users": 50})

# Mock Specialized Agents (to avoid importing their complex logic)
class MockSpecializedAgent:
    def __init__(self, name): self.name = name
    def can_handle(self, intent): return False 
    async def process(self, processed, intent): 
        # Simple pass-through for testing
        return ActionPlan(tools=[], next_step=f"mock_{self.name}"), None

# Define tool objects for Executor to use
cart_tools = MockTools
product_tools = MockTools
product_recommend_tool = MockTools
order_tools = MockTools
staff_tools = MockTools
report_tools = MockTools
system_tools = MockTools
customer_tools = MockTools

In [ ]:
# --- AGENT 1: INPUT PROCESSOR (app/agents/input_processor.py) ---

def _normalize_text(text: str) -> str:
    # Trim, collapse spaces; keep diacritics (for VI) and punctuation minimally
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text


def _detect_language_heuristic(text: str) -> str:
    t = text.lower()
    # Very lightweight heuristic: VI keywords/diacritics vs common EN markers
    if any(ch in t for ch in "ăâêôơưđáàảãạấầẩẫậắằẳẵặéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ"):
        return "vi"
    if any(word in t for word in [" the ", " and ", " price ", " order ", " product ", " hello "]):
        return "en"
    # Fallback: prefer VI as primary market
    return "vi"


async def _fetch_session_context(pool: Any, session_id: str) -> SessionContext:
    # MOCK IMPLEMENTATION
    ctx = SessionContext()
    # Simulate some context if needed
    return ctx


class InputProcessor:
    async def run(self, req: ChatRequest, pool: Any = None) -> ProcessedInput:
        print(f"[InputProcessor] Processing request: session={req.session_id}, msg='{req.message}'")
        cleaned = _normalize_text(req.message)
        lang = req.language or _detect_language_heuristic(cleaned)
        
        # Determine user_type:
        user_type = "guest"
        if req.metadata and "user_type" in req.metadata:
            user_type = req.metadata["user_type"]
        elif req.customer_id:
            user_type = "customer"
            
        session_ctx = await _fetch_session_context(pool, req.session_id)
        print(f"[InputProcessor] Context loaded: last_intent={session_ctx.last_messages[0]['intent'] if session_ctx.last_messages else 'None'}")

        return ProcessedInput(
            session_id=req.session_id,
            customer_id=req.customer_id,
            text=req.message,
            cleaned_text=cleaned,
            language=lang,
            user_type=user_type,
            tag=req.tag,
            session_ctx=session_ctx,
        )

In [ ]:
# --- AGENT 2: INTENT CLASSIFIER (app/agents/intent_classifier.py) ---
# FULL SOURCE CODE

VI_KEYWORDS: Dict[str, List[str]] = {
    "GREETING": ["xin chào", "chào", "alo", "chào bạn", "hello", "hi", "hey", "chào buổi", "chào admin", "chào shop"],
    "PRODUCT.SEARCH": [
        "tìm", "tìm kiếm", "có không", "còn không", "có bán", "muốn mua", "giá", "sản phẩm", 
        "mua", "đặt", "kiếm", "tương tự", "giống", "cần", "shop có", "bên này có", "có hàng",
        "còn hàng", "hết hàng chưa", "còn size", "có màu", "loại", "dòng", "model"
    ],
    "PRODUCT.DETAIL": [
        "chi tiết", "thông số", "xem kỹ", "thông tin", "cụ thể", "mô tả", "spec", 
        "đặc điểm", "tính năng", "chất liệu", "kích thước", "size", "màu sắc", "xuất xứ"
    ],
    "PRODUCT.RECOMMEND": [
        "gợi ý cho tôi", "gợi ý cho mình", "gợi ý vài", "nên mua gì", "mua gì", 
        "hot trend", "bán chạy", "phổ biến", "đề xuất", "recommend me",
        "trending", "mới nhất", "best seller", "top sản phẩm", "sản phẩm tốt", "sản phẩm hay", "đáng mua nhất",
        "xem sản phẩm nào", "có sản phẩm nào", "sản phẩm nào tốt"
    ],
    "ORDER.TRACK": [
        "tra cứu", "kiểm tra", "đơn hàng", "ở đâu", "tình trạng", "tracking", "giao hàng",
        "ship", "vận chuyển", "đã giao chưa", "khi nào nhận", "bao giờ về", "đến chưa"
    ],
    "ORDER.CANCEL": ["hủy đơn", "không mua nữa", "huỷ đơn", "cancel", "không muốn", "đổi ý", "không lấy"],
    "ORDER.RETURN": [
        "đổi trả", "hoàn tiền", "bảo hành", "trả hàng", "return", "refund", "warranty",
        "đổi hàng", "lỗi", "hư", "không vừa", "sai", "không đúng"
    ],
    "CART.VIEW": ["xem giỏ", "giỏ hàng", "trong giỏ", "cart", "giỏ", "đã chọn", "đã thêm"],
    "CART.ADD": [
        "thêm vào giỏ", "bỏ vào giỏ", "mua cái này", "lấy cái này", "đặt cái này", 
        "lấy cho", "thêm 1", "đặt 1", "đặt hàng", "mua ngay", "chốt", "order",
        "cho vào giỏ", "bỏ giỏ", "add", "thêm", "bỏ", "cho tôi", "cho em",
        "mình lấy", "mình mua", "lấy luôn", "mua luôn"
    ],
    "CART.REMOVE": ["xóa khỏi giỏ", "bỏ ra", "xóa sản phẩm", "bỏ sản phẩm", "remove", "không lấy", "bỏ"],
    "CART.UPDATE": ["đổi số lượng", "thay đổi", "sửa giỏ", "update cart", "tăng", "giảm", "đổi size", "đổi màu"],
    "CHECKOUT": ["thanh toán", "checkout", "đặt luôn", "mua luôn", "pay", "trả tiền", "hoàn tất"],
    "ACCOUNT.LOGIN": ["đăng nhập", "login", "sign in", "vào tài khoản"],
    "ACCOUNT.REGISTER": ["đăng ký", "tạo tài khoản", "sign up", "register", "mở tài khoản"],
    "FAQ.SHIPPING": [
        "phí ship", "vận chuyển", "bao lâu", "giao hàng", "ship", "shipping",
        "delivery", "mất bao lâu", "khi nào nhận", "ship cod", "ship nhanh"
    ],
    "FAQ.PAYMENT": [
        "thanh toán", "chuyển khoản", "cod", "trả tiền", "payment", 
        "hình thức thanh toán", "trả góp", "ví", "momo", "zalopay", "banking"
    ],
    "FAQ.PROMO": ["khuyến mãi", "giảm giá", "sale", "voucher", "mã giảm", "coupon", "promotion", "ưu đãi"],
    "FAQ.RETURN": [
        "đổi trả", "chính sách đổi", "return policy", "hoàn tiền", "refund",
        "bảo hành", "warranty", "trả hàng", "đổi hàng", "chính sách hoàn"
    ],
    "SUPPORT.ESCALATE": [
        "nhân viên", "người thật", "không phải bot", "tư vấn", "hỗ trợ", 
        "admin", "staff", "human", "real person", "agent"
    ],
    "THANK": ["cảm ơn", "thanks", "thank you", "cám ơn", "ơn"],
    "GOODBYE": ["tạm biệt", "bye", "goodbye", "hẹn gặp lại", "đi đây"],
    # Staff / Manager Keywords
    "STAFF.CHECK_STOCK": ["check kho", "tồn kho", "kiểm kho", "số lượng tồn", "inventory", "stock"],
    "STAFF.CUSTOMER_LOOKUP": ["tìm khách", "check info khách", "thông tin khách", "lookup customer"],
    "STAFF.ORDER_HISTORY": ["lịch sử mua", "đơn cũ của khách", "khách mua gì", "order history"],
    "STAFF.CREATE_ORDER": ["tạo đơn giúp", "lên đơn", "tạo đơn", "create order", "draft order"],
    "MANAGER.REPORT_SALES": ["doanh thu", "báo cáo bán hàng", "doanh số", "sales report", "revenue"],
    "MANAGER.REPORT_CHATBOT": ["hiệu quả bot", "bot chat bao nhiêu", "thống kê bot", "chatbot stats"],
    "MANAGER.CONFIG_UPDATE": ["tắt bot", "bật bot", "chỉnh prompt", "cấu hình", "config"],
}

EN_KEYWORDS: Dict[str, List[str]] = {
    "GREETING": ["hello", "hi", "hey"],
    "PRODUCT.SEARCH": ["search", "find", "looking for", "do you have", "want to buy", "price", "product", "buy", "order"],
    "PRODUCT.DETAIL": ["detail", "specs", "info", "more info", "specification"],
    "PRODUCT.RECOMMEND": ["recommend", "suggest", "hot trend", "best seller", "popular"],
    "ORDER.TRACK": ["track", "check order", "order status", "where is", "shipping"],
    "ORDER.CANCEL": ["cancel order", "cancel"],
    "ORDER.RETURN": ["return", "refund", "warranty"],
    "CART.VIEW": ["view cart", "my cart", "shopping cart"],
    "CART.ADD": ["add to cart", "buy this", "get this", "order this"],
    "CART.REMOVE": ["remove", "delete item", "remove from cart"],
    "ACCOUNT.LOGIN": ["login", "sign in"],
    "ACCOUNT.REGISTER": ["register", "sign up"],
    "FAQ.SHIPPING": ["shipping cost", "delivery time", "shipping fee"],
    "FAQ.PAYMENT": ["payment", "cod", "bank transfer"],
    "SUPPORT.ESCALATE": ["human", "staff", "real person", "agent"],
    # Staff / Manager Keywords
    "STAFF.CHECK_STOCK": ["check stock", "inventory", "stock level"],
    "STAFF.CUSTOMER_LOOKUP": ["lookup customer", "find customer", "user info"],
    "STAFF.ORDER_HISTORY": ["customer order history", "order history"],
    "STAFF.CREATE_ORDER": ["create order", "draft order"],
    "MANAGER.REPORT_SALES": ["sales report", "revenue", "sales stats"],
    "MANAGER.REPORT_CHATBOT": ["chatbot stats", "bot performance"],
    "MANAGER.CONFIG_UPDATE": ["disable bot", "enable bot", "update config"],
}


def _match_score(text: str, phrases: List[str]) -> int:
    t = f" {text.lower()} "
    score = 0
    for p in phrases:
        if f" {p} " in t:
            score += 2 if len(p.split()) > 1 else 1
    return score


def _extract_product_query(cleaned_text: str) -> str:
    s = cleaned_text
    
    # Check if this is a context reference (not a search query)
    context_patterns = [
        r"\b(cái|sản phẩm|sp)\s+(này|kia|đó|ấy|vừa|trước)",
        r"\b(đầu tiên|thứ nhất|thứ hai|cuối cùng)",
        r"\bnó\b",
        r"\bchi tiết\b.*(?!\w+)",  # "chi tiết" without specific product name
    ]
    for pattern in context_patterns:
        if re.search(pattern, s.lower()):
            return ""  # Return empty to signal this needs context
    
    s = re.sub(r"[?,.]", " ", s) # Remove punctuation
    # remove common Vietnamese/English query anchors
    remove = [
        "tìm", "tìm kiếm", "có không", "còn không", "có bán", "muốn mua",
        "search", "find", "looking for", "do you have", "want to buy",
        "sản phẩm", "product", "mua", "đặt", "kiếm", "cho tôi", "cho anh", "cho em", "cho mình",
        "tương tự", "similar",
        "tôi", "anh", "em", "bạn", "mình", "cái", "chiếc", "là", "của",
        "giá", "bao nhiêu", "price", "cost", "how much", "tiền",
        "thông tin", "xem", "về", "info", "about",
        # Staff keywords
        "check kho", "tồn kho", "kiểm kho", "số lượng tồn",
        "check stock", "inventory", "stock level",
        "tìm khách", "check info khách", "thông tin khách", "khách hàng", "khách",
        "lookup customer", "find customer", "user info",
        "nhân viên", "quản lý", "staff", "manager"
    ]
    low = s.lower()
    for r in remove:
        low = low.replace(r, " ")
    low = re.sub(r"\s+", " ", low).strip()
    return low


class IntentClassifier:
    async def run(self, processed: ProcessedInput) -> IntentResult:
        print(f"[IntentClassifier] Classifying: '{processed.cleaned_text}'")
        text = processed.cleaned_text
        lang = processed.language or "vi"

        # Choose keyword table
        table = VI_KEYWORDS if lang == "vi" else EN_KEYWORDS

        # Compute scores
        scores: Dict[str, int] = {}
        for key, phrases in table.items():
            scores[key] = _match_score(text, phrases)
        
        # Boost PRODUCT.RECOMMEND score if recommend phrases are detected
        # This prevents "gợi ý sản phẩm" from being classified as product_inquiry
        recommend_boost_phrases = ["gợi ý cho", "đề xuất cho", "recommend me", "gợi ý vài", "gợi ý một", "nên mua gì"]
        if any(phrase in text.lower() for phrase in recommend_boost_phrases):
            if "PRODUCT.RECOMMEND" in scores:
                scores["PRODUCT.RECOMMEND"] += 5  # Strong boost to override PRODUCT.SEARCH
        
        # Boost PRODUCT.DETAIL for context reference patterns (e.g., "sản phẩm đầu tiên", "cái này")
        context_patterns = [
            r'\b(cái|sản phẩm|sp)\s+(này|kia|đó|ấy|vừa|trước)',
            r'\b(đầu tiên|thứ nhất|thứ hai|thứ ba|cuối cùng)',
            r'\bnó\b',
        ]
        for pattern in context_patterns:
            if re.search(pattern, text.lower()):
                if "PRODUCT.DETAIL" in scores:
                    scores["PRODUCT.DETAIL"] += 10  # Very strong boost to override PRODUCT.SEARCH
                break

        # Get top 3 intents for multi-intent detection
        sorted_scores = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
        top_key = sorted_scores[0][0] if sorted_scores else "UNKNOWN"
        top_score = sorted_scores[0][1] if sorted_scores else 0
        
        # Detect secondary intent (e.g., "tìm balo giá rẻ" = PRODUCT.SEARCH + PRODUCT.PRICE)
        secondary_intent = None
        if len(sorted_scores) > 1 and sorted_scores[1][1] >= 2:
            secondary_intent = sorted_scores[1][0]
        
        print(f"[IntentClassifier] Top: {top_key}({top_score}), Secondary: {secondary_intent}")

        # Map to system intents used by backend
        intent_map = {
            "GREETING": "general",
            "PRODUCT.SEARCH": "product_inquiry",
            "PRODUCT.DETAIL": "product_detail",
            "PRODUCT.RECOMMEND": "product_recommend",
            "ORDER.TRACK": "order_tracking",
            "ORDER.CANCEL": "order_cancel",
            "ORDER.RETURN": "order_return",
            "CART.VIEW": "cart_view",
            "CART.ADD": "cart_add",
            "CART.REMOVE": "cart_remove",
            "CART.UPDATE": "cart_update",
            "CHECKOUT": "checkout",
            "ACCOUNT.LOGIN": "account_login",
            "ACCOUNT.REGISTER": "account_register",
            "FAQ.SHIPPING": "faq_shipping",
            "FAQ.PAYMENT": "faq_payment",
            "FAQ.PROMO": "faq_promo",
            "FAQ.RETURN": "faq_return",
            "SUPPORT.ESCALATE": "human_escalation",
            "THANK": "thank_you",
            "GOODBYE": "goodbye",
            "STAFF.CHECK_STOCK": "staff_check_stock",
            "STAFF.CUSTOMER_LOOKUP": "staff_customer_lookup",
            "STAFF.ORDER_HISTORY": "staff_order_history",
            "STAFF.CREATE_ORDER": "staff_create_order",
            "MANAGER.REPORT_SALES": "manager_report_sales",
            "MANAGER.REPORT_CHATBOT": "manager_report_chatbot",
            "MANAGER.CONFIG_UPDATE": "manager_config_update",
        }

        mapped = intent_map.get(top_key, "general")
        sub_intent = intent_map.get(secondary_intent) if secondary_intent else None

        # Enhanced confidence scoring
        if top_score >= 4:
            conf = 0.95
        elif top_score >= 3:
            conf = 0.85
        elif top_score >= 2:
            conf = 0.70
        elif top_score >= 1:
            conf = 0.50
        else:
            conf = 0.30

        entities = {}
        
        # Enhanced entity extraction
        if mapped == "product_inquiry" or mapped == "cart_add" or mapped == "product_detail":
            q = _extract_product_query(text)
            # For product_detail, empty query means context reference
            if mapped == "product_detail" and q == "":
                entities["product_query"] = ""  # Signal to use context
            elif q and len(q) >= 2:
                entities["product_query"] = q
            
            # Extract quantity
            qty_match = re.search(r'(\d+)\s*(cái|chiếc|con|bộ|món|sản phẩm|sp)', text.lower())
            if qty_match:
                entities["quantity"] = int(qty_match.group(1))
            else:
                entities["quantity"] = 1
                
        elif mapped == "order_tracking":
            # Enhanced order ID extraction
            # Patterns: #1234, order_1234, ORD-1234, đơn 1234
            match = re.search(r'(order[_\s-]?[\w\d]+|#\d+|ord[_\s-]?\d+|đơn[\s]?\d+|\d{4,})', text.lower())
            if match:
                order_id = match.group(0).replace("#", "").replace("đơn", "").replace("order", "").replace("_", "").replace("-", "").strip()
                entities["order_id"] = order_id
                
        elif mapped == "staff_check_stock":
            q = _extract_product_query(text)
            if q:
                entities["product_query"] = q
                
        elif mapped == "staff_customer_lookup":
            # Extract customer info (email, phone, name)
            email_match = re.search(r'[\w\.-]+@[\w\.-]+\.\w+', text)
            phone_match = re.search(r'(\+?84|0)\d{9,10}', text)
            
            if email_match:
                entities["customer_email"] = email_match.group(0)
            elif phone_match:
                entities["customer_phone"] = phone_match.group(0)
            else:
                # Fallback to query extraction
                q = _extract_product_query(text)
                if q:
                    entities["customer_query"] = q
        
        print(f"[IntentClassifier] Result: intent={mapped}, sub={sub_intent}, entities={entities}")
        return IntentResult(
            intent=mapped,
            sub_intent=sub_intent,
            confidence=conf,
            entities=entities,
        )

In [ ]:
# --- AGENT 3: ORCHESTRATOR (app/agents/orchestrator.py) ---

class Orchestrator:
    def __init__(self):
        self.agents = {
            "sales": MockSpecializedAgent("sales"),
            "order": MockSpecializedAgent("order"),
            "staff": MockSpecializedAgent("staff"),
            "manager": MockSpecializedAgent("manager")
        }

    async def run(
        self,
        processed: ProcessedInput,
        intent: IntentResult,
    ) -> Tuple[ActionPlan, Optional[ToolResults]]:
        """
        Hybrid Orchestrator:
        1. Executable Tag -> Execute Tool directly
        2. Scope Tag -> Route to Specialized Agent (Scoped NLU)
        3. No Tag -> Global NLU -> Route to Agent
        """
        print(f"[Orchestrator] Routing... Tag={processed.tag}, Intent={intent.intent}")
        
        # 1. Executable Tag (e.g., "action:view_cart")
        if processed.tag and processed.tag.startswith("action:"):
            action = processed.tag.split(":", 1)[1]
            print(f"[Orchestrator] Direct action: {action}")
            
            if action == "view_cart":
                return ActionPlan(tools=["cart.view"], next_step="show_cart"), None
            elif action == "order_list":
                return ActionPlan(tools=["order.list"], next_step="show_orders"), None
            elif action == "order_track":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "reorder":
                return ActionPlan(tools=["order.reorder"], next_step="confirm_reorder"), None
            elif action == "logout":
                return ActionPlan(tools=["auth.logout"], next_step="confirm_logout"), None
            elif action == "chat_human":
                return ActionPlan(tools=["system.escalate"], next_step="escalate_chat"), None
            elif action == "my_reviews":
                return ActionPlan(tools=["product.reviews"], next_step="show_reviews"), None
            elif action == "check_stock":
                return ActionPlan(tools=[], next_step="ask_product_name"), None # Need product name
            elif action == "check_price":
                return ActionPlan(tools=[], next_step="ask_product_name"), None # Need product name
            elif action == "staff_lookup_order":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "staff_customer_lookup":
                return ActionPlan(tools=[], next_step="ask_customer_info"), None
            elif action == "update_order_status":
                return ActionPlan(tools=[], next_step="ask_order_id_status"), None
            elif action == "print_shipping_label":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "sales_report":
                return ActionPlan(tools=["manager.report_sales"], next_step="show_report"), None
            elif action == "top_products":
                return ActionPlan(tools=["manager.top_products"], next_step="show_top_products"), None
            elif action == "customer_analytics":
                return ActionPlan(tools=["manager.customer_analytics"], next_step="show_analytics"), None
            elif action.startswith("cart.add_item"):
                try:
                    params = {}
                    if "?" in action:
                        query = action.split("?", 1)[1]
                        for pair in query.split("&"):
                            if "=" in pair:
                                k, v = pair.split("=", 1)
                                params[k] = v
                    
                    intent.entities["product_id"] = params.get("product_id")
                    intent.entities["variant_id"] = params.get("variant_id")
                    intent.entities["quantity"] = 1
                    
                    return ActionPlan(tools=["cart.add"], next_step="confirm_add_to_cart"), None
                except Exception as e:
                    print(f"Error parsing cart.add_item tag: {e}")
                    return ActionPlan(tools=[], next_step="error"), None
            
            # Add more direct actions here
        
        # 2. Scope Tag (e.g., "scope:order")
        if processed.tag and processed.tag.startswith("scope:"):
            scope = processed.tag.split(":", 1)[1]
            agent = self.agents.get(scope)
            if agent:
                print(f"[Orchestrator] Scoped routing to: {scope}")
                # Agent performs its own scoped NLU if intent is generic/unknown or we want to re-evaluate
                # For now, we pass None as intent to force agent to re-classify within scope
                return await agent.process(processed, intent=None)
        
        # 3. Implicit Routing (Global NLU)
        # We use the intent passed from the global classifier to decide which agent to call
        target_agent = self._route_by_intent(intent.intent)
        if target_agent:
            print(f"[Orchestrator] Implicit routing to: {target_agent.name}")
            return await target_agent.process(processed, intent)
            
        # Default fallback
        print("[Orchestrator] No agent found, fallback.")
        return ActionPlan(tools=[], next_step=None), None

    def _route_by_intent(self, intent_name: str):
        """Route intent to appropriate specialized agent"""
        for agent in self.agents.values():
            if agent.can_handle(intent_name):
                return agent
        return self.agents["sales"] # Default to sales if unknown
    
    def get_agent_suggestions(self, intent: str) -> list:
        """Get suggested actions based on intent - for better UX"""
        suggestions = {
            "product_inquiry": [
                {"label": "Xem giỏ hàng", "action": "action:view_cart"},
                {"label": "Sản phẩm gợi ý", "action": "action:my_recommendations"},
            ],
            "cart_view": [
                {"label": "Thanh toán ngay", "action": "action:checkout"},
                {"label": "Tiếp tục mua sắm", "action": "action:continue_shopping"},
            ],
            "order_tracking": [
                {"label": "Đơn hàng của tôi", "action": "action:order_list"},
                {"label": "Liên hệ hỗ trợ", "action": "action:chat_human"},
            ],
            "general": [
                {"label": "Tìm sản phẩm", "action": "action:product_search"},
                {"label": "Xem giỏ hàng", "action": "action:view_cart"},
                {"label": "Kiểm tra đơn hàng", "action": "action:order_list"},
            ],
        }
        return suggestions.get(intent, suggestions["general"])

In [ ]:
# --- AGENT 4: EXECUTOR (app/agents/executor.py) ---

def _merge_tool_result(results: ToolResults, tool_result: Any):
    if tool_result is None:
        return
    if isinstance(tool_result, ToolResults):
        results.ok = results.ok and tool_result.ok
        if tool_result.errors:
            results.errors.extend(tool_result.errors)
        if tool_result.timings_ms:
            results.timings_ms.update(tool_result.timings_ms)
        results.data = tool_result.data
    else:
        results.data = tool_result

class Executor:
    async def run(
        self, 
        processed: ProcessedInput, 
        intent: IntentResult, 
        plan: ActionPlan
    ) -> Optional[ToolResults]:
        if not plan.tools:
            return None

        results = ToolResults(ok=True)
        
        for tool_name in plan.tools:
            try:
                logger.info(f"[Executor] Running tool: {tool_name}")
                
                # Cart Tools
                if tool_name == "cart.view":
                    if processed.session_ctx.cart_id:
                        res = await cart_tools.view_cart(processed.session_ctx.cart_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing cart_id")
                elif tool_name == "cart.add":
                    # Extract product and quantity from entities
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    qty = intent.entities.get("quantity", 1)
                    variant_id = intent.entities.get("variant_id")
                    
                    if processed.session_ctx.cart_id:
                        if variant_id:
                            # If we have explicit variant_id (e.g. from UI action), use it directly
                            res = await cart_tools.add_to_cart(processed.session_ctx.cart_id, variant_id, qty)
                        else:
                            # Otherwise try smart add (search -> add)
                            res = await cart_tools.add_to_cart_smart(processed.session_ctx.cart_id, query, qty)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing cart_id")
                
                # Product Tools
                elif tool_name == "product.search":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await product_tools.search_products(query)
                    _merge_tool_result(results, res)
                elif tool_name == "product.recommend":
                    # The tool currently only accepts limit, not customer_id
                    res = await product_recommend_tool.recommend_products(limit=5)
                    _merge_tool_result(results, res)
                elif tool_name == "product.detail":
                    # Try to get product_id from entities first
                    product_id = intent.entities.get("product_id")
                    
                    # If no product_id in entities, try to get from context (last products shown)
                    if not product_id and processed.session_ctx.last_product_ids:
                        product_id = processed.session_ctx.last_product_ids[0]
                        logger.info(f"[Executor] Using context product_id: {product_id}")
                    
                    if product_id:
                        res = await product_tools.get_product_details(product_id)
                        _merge_tool_result(results, res)
                    else:
                        # Fallback: try to search if we have a query
                        # Extract product name from text (remove "chi tiết", "xem", etc)
                        import re
                        query = intent.entities.get("product_query")
                        if not query:
                            # Try to extract product name from cleaned text
                            text_lower = processed.cleaned_text.lower()
                            # Remove common phrases (order matters - remove longer phrases first)
                            for phrase in ["cho tôi xem chi tiết", "cho tôi xem", "xem chi tiết", "chi tiết của", "chi tiết", "xem", "tôi muốn", "muốn"]:
                                text_lower = text_lower.replace(phrase, "")
                            query = text_lower.strip()
                        
                        if query and len(query) >= 2:
                            logger.info(f"[Executor] Searching for product: '{query}'")
                            res = await product_tools.search_products(query, limit=1)
                            _merge_tool_result(results, res)
                            # Store search query in entities for response_generator
                            intent.entities["product_query"] = query
                        else:
                            results.ok = False
                            results.errors.append("Missing product_id or query")
                            # Still store a placeholder for template
                            intent.entities["product_query"] = "sản phẩm bạn cần"

                # Order Tools
                elif tool_name == "order.track":
                    order_id = intent.entities.get("order_id")
                    email = intent.entities.get("email") # Might need to ask user
                    if order_id:
                        res = await order_tools.lookup_order(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "order.list":
                    if processed.customer_id:
                        res = await order_tools.list_orders(processed.customer_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "order.reorder":
                    # Assuming we reorder the last order or specific order
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await order_tools.reorder(order_id)
                        _merge_tool_result(results, res)
                    else:
                        # Maybe list orders to pick?
                        results.ok = False
                        results.errors.append("Missing order_id")

                # Staff Tools
                elif tool_name == "staff.check_stock":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await staff_tools.check_stock(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.check_price":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await staff_tools.check_price(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.customer_lookup":
                    query = intent.entities.get("customer_query", processed.cleaned_text)
                    res = await staff_tools.lookup_customer(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.order_history":
                    customer_id = intent.entities.get("customer_id")
                    if customer_id:
                        res = await staff_tools.get_customer_order_history(customer_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "staff.create_order":
                    if processed.customer_id:
                        res = await staff_tools.create_draft_order(processed.customer_id, [])
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "staff.lookup_order":
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await staff_tools.lookup_order(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "staff.update_order":
                    order_id = intent.entities.get("order_id")
                    status = intent.entities.get("status", "processing")
                    if order_id:
                        res = await staff_tools.update_order_status(order_id, status)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "staff.print_label":
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await staff_tools.print_shipping_label(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                
                # Manager Tools
                elif tool_name == "manager.report_sales":
                    res = await report_tools.get_sales_report()
                    _merge_tool_result(results, res)
                elif tool_name == "manager.top_products":
                    res = await report_tools.get_top_products()
                    _merge_tool_result(results, res)
                elif tool_name == "manager.customer_analytics":
                    res = await report_tools.get_customer_analytics()
                    _merge_tool_result(results, res)

                # System/Auth Tools
                elif tool_name == "system.escalate":
                    # Just a marker, maybe log it
                    results.data = {"status": "escalated", "message": "Connecting to human agent..."}
                elif tool_name == "auth.logout":
                    # In a real app, this might invalidate session
                    results.data = {"status": "logged_out", "message": "You have been logged out."}
                
                # Product Tools (Extra)
                elif tool_name == "product.reviews":
                    # Mock
                    results.data = {"reviews": []}

                else:
                    logger.warning(f"[Executor] Unknown tool: {tool_name}")
                    results.errors.append(f"Unknown tool: {tool_name}")

            except Exception as e:
                logger.error(f"[Executor] Error running {tool_name}: {e}")
                results.ok = False
                results.errors.append(str(e))
        
        return results

In [ ]:
# --- AGENT 5: RESPONSE GENERATOR (app/agents/response_generator.py) ---

class ResponseGenerator:
    def __init__(self):
        if GOOGLE_API_KEY == "YOUR_API_KEY_HERE":
            self.client = None
        else:
            from openai import AsyncOpenAI
            self.client = AsyncOpenAI(api_key=GOOGLE_API_KEY, base_url=GEMINI_BASE_URL)
        self.context_config = CONTEXT_CONFIG

    def _load_context_config(self):
        return CONTEXT_CONFIG

    async def run(
        self, 
        processed: ProcessedInput, 
        intent: IntentResult, 
        plan: ActionPlan, 
        tool_res: Optional[ToolResults]
    ) -> ChatResponse:
        
        # 1. Generate Text Response
        response_text = await self._generate_llm_response(processed, intent, tool_res)
        
        # 2. Generate Quick Replies (Options)
        quick_replies = self._generate_quick_replies(processed)
        
        # 3. Format Products if available
        products = []
        if tool_res and tool_res.ok and isinstance(tool_res.data, list):
            for item in tool_res.data:
                # Try to map to ProductInfo
                try:
                    # If item is already ProductInfo, append directly
                    if isinstance(item, ProductInfo):
                        products.append(item)
                    # Handle if item is dict
                    elif isinstance(item, dict):
                        # Filter keys that match ProductInfo
                        valid_keys = ProductInfo.model_fields.keys()
                        filtered = {k: v for k, v in item.items() if k in valid_keys}
                        # Ensure required fields
                        if "id" in filtered and "title" in filtered and "handle" in filtered:
                            products.append(ProductInfo(**filtered))
                    # Handle if item is object
                    elif hasattr(item, "id") and hasattr(item, "title"):
                        # Manually map variants if they exist in the object but weren't automatically mapped
                        variants = getattr(item, "variants", [])
                        # If variants are already ProductVariantInfo objects, use them.
                        # If they are dicts, convert them.
                        mapped_variants = []
                        for v in variants:
                            if isinstance(v, dict):
                                mapped_variants.append(v) # Pydantic will handle dict -> model
                            else:
                                mapped_variants.append(v)

                        products.append(ProductInfo(
                            id=item.id,
                            title=item.title,
                            handle=item.handle,
                            thumbnail=getattr(item, "thumbnail", None),
                            price=getattr(item, "price", None),
                            currency_code=getattr(item, "currency_code", None),
                            variants=mapped_variants
                        ))
                except Exception as e:
                    logger.warning(f"Failed to map product item: {e}")
        
        # Add product-specific quick replies if products found
        if products:
            # Add "Buy Now" button first
            quick_replies.insert(0, QuickReply(label="Mua ngay", value=f"Đặt hàng {products[0].title}"))
            # Add "View Details" button second
            quick_replies.insert(1, QuickReply(label="Xem chi tiết", value=f"Chi tiết {products[0].title}"))

        return ChatResponse(
            response=response_text,
            session_id=processed.session_id,
            products=products,
            quick_replies=quick_replies,
            metadata={"intent": intent.intent}
        )

    async def _generate_llm_response(self, processed, intent, tool_res) -> str:
        # Try template-based response first for common cases
        template_context = {}
        
        print(f"[ResponseGen] intent={intent.intent}, tool_res={'exists' if tool_res else 'None'}, tool_res.ok={tool_res.ok if tool_res else 'N/A'}, data_len={len(tool_res.data) if tool_res and tool_res.ok else 0}")
        
        # PRIORITY 1: Handle FAQ intents with templates (avoid LLM hallucination)
        if intent.intent in ["faq_shipping", "faq_payment", "faq_return", "faq_promo"]:
            print(f"[ResponseGen] Using FAQ template for {intent.intent}")
            return get_response_template(intent.intent)
        
        # PRIORITY 2: Handle thank_you/goodbye
        if intent.intent == "thank_you":
            return get_response_template("thank_you")
        elif intent.intent == "goodbye":
            return get_response_template("goodbye")
        
        if tool_res:
            if tool_res.ok and isinstance(tool_res.data, list):
                template_context['count'] = len(tool_res.data)
                if len(tool_res.data) > 0:
                    first_item = tool_res.data[0]
                    if isinstance(first_item, dict):
                        template_context['product'] = first_item.get('title', '')
                    elif hasattr(first_item, 'title'):
                        template_context['product'] = first_item.title
            else:
                # tool_res.ok = False means tool failed/no results
                template_context['count'] = 0
        
        # PRIORITY 3: Handle product intents with templates (even without tool_res)
        if intent.intent == "general" and (not tool_res or not tool_res.ok):
            print("[ResponseGen] Using greeting template")
            return get_response_template("greeting")
        elif intent.intent in ["product_inquiry", "product_detail"]:
            # Get product query from entities (may be set by executor)
            query = intent.entities.get("product_query")
            if query:
                template_context['query'] = query
            else:
                # Fallback: extract from processed text
                template_context['query'] = "sản phẩm bạn cần"
            
            if template_context.get('count', 0) > 0:
                print(f"[ResponseGen] Using product_found template, count={template_context['count']}")
                return get_response_template("product_found", template_context)
            else:
                # Product not found - use template to avoid LLM hallucination
                print(f"[ResponseGen] Using product_not_found template, query={template_context.get('query')}")
                return get_response_template("product_not_found", template_context)
        elif intent.intent == "product_recommend":
            if tool_res.ok and template_context.get('count', 0) > 0:
                return get_response_template("product_recommend")
            else:
                return "Xin lỗi, shop đang cập nhật sản phẩm. Bạn quay lại sau nhé!"
        elif intent.intent == "cart_add":
            if tool_res.ok:
                return get_response_template("cart_added", template_context)
            else:
                return "Xin lỗi, tôi cần biết sản phẩm nào bạn muốn thêm vào giỏ hàng."
        
        # PRIORITY 4: Fallback to LLM only for complex cases
        # But prevent hallucination by checking tool_res.ok
        if tool_res and not tool_res.ok:
            # Tool failed - don't let LLM make up data
            print(f"[ResponseGen] Tool failed, using error template")
            if intent.intent in ["product_inquiry", "product_detail"]:
                return get_response_template("product_not_found", template_context)
            return "Xin lỗi, tôi không tìm thấy thông tin bạn cần. Bạn có thể hỏi lại được không?"
        
        # Construct prompt for LLM (only when tool succeeded or no tool needed)
        print(f"[ResponseGen] Falling through to LLM generation")
        context_str = ""
        if tool_res:
            if tool_res.ok:
                # Truncate data if too large
                data_str = json.dumps(tool_res.data, ensure_ascii=False, default=str)
                if len(data_str) > 3000: # Increased limit for better context
                    data_str = data_str[:3000] + "...(truncated)"
                context_str = f"Tool Result: {data_str}"
            else:
                context_str = f"Tool Error: {tool_res.errors}"
        
        # Add currency hint if available in tool result (e.g. from product search)
        currency_hint = ""
        if tool_res and tool_res.ok and isinstance(tool_res.data, list) and len(tool_res.data) > 0:
            first_item = tool_res.data[0]
            if isinstance(first_item, dict) and "currency_code" in first_item:
                 currency_hint = f"\nLưu ý: Đơn vị tiền tệ đang sử dụng là {first_item.get('currency_code', 'N/A').upper()}."
            elif hasattr(first_item, "currency_code"):
                 currency_hint = f"\nLưu ý: Đơn vị tiền tệ đang sử dụng là {getattr(first_item, 'currency_code', 'N/A').upper()}."

        messages = [
            {"role": "system", "content": SYSTEM_PROMPT_VI + currency_hint},
            {"role": "user", "content": f"User Input: {processed.text}\nIntent: {intent.intent}\nContext: {context_str}"}
        ]
        
        if self.client:
            try:
                completion = await self.client.chat.completions.create(
                    model=GEMINI_MODEL,
                    messages=messages,
                    temperature=0.7
                )
                return completion.choices[0].message.content or ""
            except Exception as e:
                logger.error(f"LLM Error: {e}")
                return get_response_template("error_generic")
        else:
            return f"[MOCK LLM RESPONSE] Based on {intent.intent} and data: {tool_res.data if tool_res else 'None'}"

    def _generate_quick_replies(self, processed) -> List[QuickReply]:
        nodes = self.context_config.get("nodes", [])
        
        # Helper to find node by ID recursively
        def find_node(node_list, target_id):
            for node in node_list:
                if node.get("id") == target_id:
                    return node
                if "children" in node:
                    found = find_node(node["children"], target_id)
                    if found: return found
            return None

        # Determine current context node
        current_node = None
        if processed.tag:
            # If tag is like "action:view_cart", we might not find a node if it's a leaf.
            # But if tag is a group ID, we might.
            # Let's try to find by ID first (assuming tag might be ID)
            current_node = find_node(nodes, processed.tag)
        
        # If no specific node found, default to user root
        if not current_node:
            user_root_id = f"{processed.user_type}_root"
            current_node = find_node(nodes, user_root_id)

        if not current_node:
            return []

        options = []
        # Return children of the current node
        for child in current_node.get("children", []):
            # Determine value: prefer value, then tag, then id
            val = child.get("value") or child.get("tag") or child.get("id")
            options.append(QuickReply(
                label=child["label"],
                value=val,
                metadata={"type": child.get("type")}
            ))
            
        return options

In [ ]:
# --- DATA LOADING ---
# Upload 'multi_agent_test_cases_full.csv' to Colab files
try:
    df = pd.read_csv('multi_agent_test_cases_full.csv')
    print(f"Loaded {len(df)} test cases.")
except FileNotFoundError:
    print("File not found. Creating sample data...")
    data = {
        "id": [1, 2, 3],
        "user_input": ["Xin chào shop", "Tìm balo JanSport màu đen", "Kiểm tra đơn hàng #123"],
        "expected_intent_classifier_intent": ["general", "product_inquiry", "order_tracking"],
        "expected_executor_tools_called": ["", "product.search", "order.track"]
    }
    df = pd.DataFrame(data)
    print("Created sample data.")

df.head()

In [ ]:
# --- EVALUATION LOOP ---

async def evaluate_row(row):
    # Initialize Agents
    input_processor = InputProcessor()
    intent_classifier = IntentClassifier()
    orchestrator = Orchestrator()
    executor = Executor()
    response_generator = ResponseGenerator()

    # 1. Input
    req = ChatRequest(session_id=f"sess_{row['id']}", message=row['user_input'])
    processed = await input_processor.run(req)
    
    # 2. Intent
    intent_res = await intent_classifier.run(processed)
    
    # 3. Orchestrator
    plan, _ = await orchestrator.run(processed, intent_res)
    
    # 4. Executor
    tool_res = await executor.run(processed, intent_res, plan)
    
    # 5. Response
    response = await response_generator.run(processed, intent_res, plan, tool_res)
    
    # --- CHECKS ---
    results = {
        "id": row['id'],
        "input": row['user_input'],
        "intent_actual": intent_res.intent,
        "intent_expected": row.get('expected_intent_classifier_intent'),
        "intent_match": intent_res.intent == row.get('expected_intent_classifier_intent'),
        
        "tools_actual": ",".join(plan.tools),
        "tools_expected": row.get('expected_executor_tools_called', ""),
        # Simple check: is expected tool in actual tools?
        "tools_match": str(row.get('expected_executor_tools_called', "")) in ",".join(plan.tools) if pd.notna(row.get('expected_executor_tools_called')) else True,
        
        "response_preview": response.response[:50] + "..."
    }
    return results

# Run Evaluation
results_list = []
print("Starting evaluation...")
for index, row in df.iterrows():
    res = await evaluate_row(row)
    results_list.append(res)
    print(f"Row {row['id']}: Intent Match={res['intent_match']}, Tools Match={res['tools_match']}")

results_df = pd.DataFrame(results_list)

In [ ]:
# --- SUMMARY REPORT ---
print("\n--- EVALUATION SUMMARY ---")
total = len(results_df)
intent_acc = results_df['intent_match'].mean() * 100
tools_acc = results_df['tools_match'].mean() * 100

print(f"Total Test Cases: {total}")
print(f"Intent Accuracy: {intent_acc:.2f}%")
print(f"Tool Selection Accuracy: {tools_acc:.2f}%")

print("\n--- FAILED CASES ---")
failed = results_df[~results_df['intent_match'] | ~results_df['tools_match']]
if not failed.empty:
    display(failed[['id', 'input', 'intent_actual', 'intent_expected', 'tools_actual', 'tools_expected']])
else:
    print("All cases passed!")

In [ ]:
!pip install pydantic openai pandas python-dotenv nest_asyncio pyyaml

In [ ]:
import os
import json
import re
import yaml
import asyncio
import logging
import pandas as pd
import nest_asyncio
from typing import List, Dict, Any, Optional, Tuple
from pydantic import BaseModel, Field
from datetime import datetime

# Apply nest_asyncio to allow async loops in notebook
nest_asyncio.apply()

# Configure Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("Evaluation")

In [ ]:
# --- CONFIGURATION ---
# Replace with your actual API Key
GOOGLE_API_KEY = "YOUR_API_KEY_HERE" 
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
GEMINI_MODEL = "gemini-1.5-flash"

SYSTEM_PROMPT_VI = """
Bạn là trợ lý ảo AI chuyên nghiệp của cửa hàng JanSport Việt Nam.
Nhiệm vụ của bạn là hỗ trợ khách hàng tìm kiếm sản phẩm, kiểm tra đơn hàng và giải đáp thắc mắc.
Luôn trả lời bằng tiếng Việt, giọng điệu thân thiện, nhiệt tình và chuyên nghiệp.
"""

# Mock Context Config (from context_config.yaml)
CONTEXT_CONFIG = {
    "nodes": [
        {
            "id": "guest_root",
            "label": "Guest Root",
            "children": [
                {"label": "Tìm sản phẩm", "value": "action:product_search"},
                {"label": "Tra cứu đơn hàng", "value": "action:order_track"}
            ]
        },
        {
            "id": "customer_root",
            "label": "Customer Root",
            "children": [
                {"label": "Đơn hàng của tôi", "value": "action:order_list"},
                {"label": "Giỏ hàng", "value": "action:view_cart"}
            ]
        }
    ]
}

In [ ]:
# --- DATA MODELS (app/models/agent_types.py) ---
# Copied exactly from source

class SessionContext(BaseModel):
    last_messages: List[Dict[str, Any]] = Field(default_factory=list)
    cart_id: Optional[str] = None
    last_product_ids: List[str] = Field(default_factory=list)


class ProcessedInput(BaseModel):
    session_id: str
    customer_id: Optional[str] = None
    text: str
    cleaned_text: str
    language: str = "vi"  # "vi" | "en"
    user_type: str = "guest"  # "guest" | "customer"
    tag: Optional[str] = None
    session_ctx: SessionContext = Field(default_factory=SessionContext)


class IntentResult(BaseModel):
    intent: str
    sub_intent: Optional[str] = None
    confidence: float = 0.0
    entities: Dict[str, Any] = Field(default_factory=dict)


class ActionPlan(BaseModel):
    tools: List[str] = Field(default_factory=list)
    required_data: List[str] = Field(default_factory=list)
    next_step: Optional[str] = None
    escalate: bool = False


class ToolResults(BaseModel):
    ok: bool
    data: Any = None
    errors: List[str] = Field(default_factory=list)
    timings_ms: Dict[str, int] = Field(default_factory=dict)

# Additional Models needed for Response Generator
class ProductInfo(BaseModel):
    id: str
    title: str
    handle: str
    thumbnail: Optional[str] = None
    price: Optional[int] = None
    currency_code: Optional[str] = None
    variants: List[Dict[str, Any]] = Field(default_factory=list)

class QuickReply(BaseModel):
    label: str
    value: str
    metadata: Optional[Dict[str, Any]] = None

class ChatResponse(BaseModel):
    response: str
    session_id: str
    products: List[ProductInfo] = Field(default_factory=list)
    quick_replies: List[QuickReply] = Field(default_factory=list)
    metadata: Dict[str, Any] = Field(default_factory=dict)

class ChatRequest(BaseModel):
    session_id: str
    message: str
    customer_id: Optional[str] = None
    language: Optional[str] = None
    metadata: Optional[Dict[str, Any]] = None
    tag: Optional[str] = None

In [ ]:
# --- RESPONSE TEMPLATES (app/agents/response_templates.py) ---
# Copied exactly from source

RESPONSE_TEMPLATES = {
    # Greetings
    "greeting": [
        "Xin chào! Tôi là trợ lý ảo của shop. Tôi có thể giúp gì cho bạn hôm nay? 😊",
        "Chào bạn! Rất vui được hỗ trợ bạn. Bạn cần tìm sản phẩm gì không?",
        "Hello! Chào mừng bạn đến với shop. Tôi có thể tư vấn gì cho bạn?"
    ],
    
    # Product Search Success
    "product_found": [
        "Mình tìm thấy {count} sản phẩm phù hợp với '{query}' ạ:",
        "Đây là {count} sản phẩm mình tìm được cho bạn về '{query}':",
        "Shop có {count} sản phẩm '{query}' như sau:"
    ],
    
    # Product Not Found
    "product_not_found": [
        "Rất tiếc, shop không tìm thấy sản phẩm '{query}' bạn cần 😔\nBạn có thể thử tìm kiếm sản phẩm tương tự không?",
        "Hiện tại shop chưa có sản phẩm '{query}' ạ. Bạn muốn xem sản phẩm khác không?",
        "Mình không tìm thấy '{query}' trong kho. Bạn có thể mô tả chi tiết hơn hoặc xem sản phẩm tương tự?"
    ],
    
    # Product Recommend
    "product_recommend": [
        "🌟 Đây là một số sản phẩm hot mà bạn có thể thích:",
        "Mình gợi ý cho bạn vài sản phẩm đang bán chạy nha:",
        "Đây là top sản phẩm được yêu thích nhất của shop:"
    ],
    
    # Cart Actions
    "cart_added": [
        "Đã thêm {product} vào giỏ hàng! Bạn muốn tiếp tục mua sắm hay thanh toán luôn?",
        "OK! Đã cho {product} vào giỏ rồi nhé. Mua thêm gì nữa không?"
    ],
    
    "cart_view_empty": [
        "Giỏ hàng của bạn đang trống. Tìm sản phẩm ngay nhé! 🛍️",
        "Bạn chưa có sản phẩm nào trong giỏ. Cùng khám phá shop nhé!"
    ],
    
    "cart_view_has_items": [
        "Giỏ hàng của bạn có {count} sản phẩm:",
        "Bạn đang có {count} món trong giỏ:"
    ],
    
    # Order Tracking
    "order_found": [
        "Đơn hàng #{order_id} của bạn:\nTrạng thái: {status}\nDự kiến giao: {delivery_date}",
        "Mình check đơn #{order_id} rồi:\n📦 {status}\n🚚 Giao dự kiến: {delivery_date}"
    ],
    
    "order_not_found": [
        "Không tìm thấy đơn hàng #{order_id}. Bạn kiểm tra lại mã đơn giúp mình nhé!",
        "Mình không thấy đơn #{order_id} trong hệ thống. Bạn có chắc mã đúng không?"
    ],
    
    # FAQ
    "faq_shipping": [
        "🚚 Thông tin vận chuyển:\n- Nội thành Hà Nội/HCM: 2-3 ngày, phí 30.000₫\n- Ngoại thành: 3-5 ngày, phí 50.000₫\n- Miễn phí ship cho đơn hàng trên 500.000₫\n- Hỗ trợ COD toàn quốc",
        "Shop ship nhanh trong 2-3 ngày nội thành nha! Phí 30k, FREE SHIP cho đơn trên 500k. Bạn có thể thanh toán COD khi nhận hàng ạ!",
        "Vận chuyển:\n• Giao hàng toàn quốc\n• Nội thành: 2-3 ngày (30k)\n• Ngoại thành: 3-5 ngày (50k)\n• FREE SHIP đơn > 500k\n• Hỗ trợ COD"
    ],
    
    "faq_return": [
        "🔄 Chính sách đổi trả:\n- Đổi trả miễn phí trong 7 ngày\n- Sản phẩm chưa sử dụng, còn nguyên tem mác\n- Miễn phí đổi size/màu (nếu còn hàng)\n- Hoàn tiền trong 3-5 ngày làm việc",
        "Bạn được đổi trả miễn phí trong 7 ngày nếu sản phẩm còn nguyên tem mác nha! Đổi size/màu miễn phí luôn ạ."
    ],
    
    "faq_payment": [
        "💳 Các hình thức thanh toán:\n✓ COD (tiền mặt khi nhận hàng)\n✓ Chuyển khoản\n✓ Thẻ tín dụng/ghi nợ\n✓ Ví điện tử (Momo, ZaloPay)",
        "Bạn có thể thanh toán bằng COD, chuyển khoản, thẻ, hoặc ví Momo/ZaloPay nha!"
    ],
    
    "faq_promo": [
        "🎁 Khuyến mãi hiện tại:\n- Giảm 10% đơn đầu tiên\n- Free ship đơn >500k\n- Tích điểm đổi quà",
        "Bạn mới có ưu đãi giảm 10% đơn đầu + freeship đơn >500k đó! 🎉"
    ],
    
    # Thank you & Goodbye
    "thank_you": [
        "Không có gì! Rất vui được hỗ trợ bạn. Cần gì cứ gọi mình nhé! 😊",
        "Hehe, có gì đâu! Cần gì cứ nhắn mình nha!",
        "Luôn sẵn sàng giúp bạn! Mua sắm vui vẻ nhé! 🛍️"
    ],
    
    "goodbye": [
        "Tạm biệt! Hẹn gặp lại bạn sớm. Chúc bạn một ngày tốt lành! 👋",
        "Bye bye! Nhớ ghé shop thường xuyên nha! 😊",
        "Hẹn gặp lại! Mua sắm vui vẻ nhé! 🛍️"
    ],
    
    # Errors
    "error_generic": [
        "Xin lỗi, có lỗi xảy ra. Bạn có thể thử lại không?",
        "Ối, có gì đó không ổn. Mình thử lại nhé!"
    ],
    
    "error_no_products": [
        "Shop đang cập nhật sản phẩm. Bạn quay lại sau nhé!",
        "Hiện tại sản phẩm đang hết hàng. Mình sẽ thông báo khi có hàng!"
    ],
    
    # Staff/Manager
    "staff_success": [
        "Đã xử lý xong! Kết quả:",
        "Done! Thông tin bạn cần:"
    ],
    
    "manager_report": [
        "📊 Báo cáo {report_type}:\n{data}",
        "Đây là thống kê {report_type} bạn yêu cầu:\n{data}"
    ],
    
    # Clarification needed
    "need_clarification": [
        "Mình chưa hiểu rõ lắm. Bạn có thể nói rõ hơn được không?",
        "Hmm, bạn muốn tìm gì nhỉ? Mô tả chi tiết giúp mình nha!",
        "Bạn cần tìm sản phẩm gì? Cho mình biết thêm thông tin nhé!"
    ],
    
    "ask_product_details": [
        "Bạn muốn tìm loại nào? (màu, size, giá...)",
        "Cho mình biết thêm chi tiết: màu sắc, kích thước, giá cả?"
    ],
    
    "ask_order_id": [
        "Bạn cho mình mã đơn hàng để check nhé! (VD: #1234 hoặc ORD-5678)",
        "Mã đơn hàng của bạn là gì ạ?"
    ]
}

def get_response_template(intent: str, context: Optional[dict] = None) -> str:
    """Get appropriate response template based on intent and context"""
    import random
    
    context = context or {}
    templates = RESPONSE_TEMPLATES.get(intent, RESPONSE_TEMPLATES["need_clarification"])
    
    # Select random template for variety
    template = random.choice(templates)
    
    # Format with context if available
    try:
        if context:
            template = template.format(**context)
    except KeyError:
        pass  # Use template as-is if formatting fails
    
    return template

In [ ]:
# --- MOCKS FOR EXTERNAL DEPENDENCIES ---

# Mock Tools to simulate Medusa/Redis calls
class MockTools:
    @staticmethod
    async def search_products(query, limit=5):
        return ToolResults(ok=True, data=[
            {"id": "p_1", "title": f"JanSport SuperBreak - {query}", "handle": "superbreak", "price": 500000, "currency_code": "vnd"},
            {"id": "p_2", "title": f"JanSport Big Student - {query}", "handle": "big-student", "price": 800000, "currency_code": "vnd"}
        ])
    
    @staticmethod
    async def get_product_details(product_id):
        return ToolResults(ok=True, data={"id": product_id, "title": "JanSport Detail", "description": "Great bag", "price": 500000, "currency_code": "vnd"})

    @staticmethod
    async def recommend_products(limit=5):
        return ToolResults(ok=True, data=[{"id": "rec_1", "title": "Trending Bag", "handle": "trending", "price": 600000, "currency_code": "vnd"}])

    @staticmethod
    async def view_cart(cart_id):
        return ToolResults(ok=True, data={"id": cart_id, "items": []})

    @staticmethod
    async def add_to_cart(cart_id, variant_id, qty):
        return ToolResults(ok=True, data={"cart_id": cart_id, "added": True})
        
    @staticmethod
    async def add_to_cart_smart(cart_id, query, qty):
        return ToolResults(ok=True, data={"cart_id": cart_id, "added": True, "product": query})

    @staticmethod
    async def lookup_order(order_id):
        if order_id == "123":
            return ToolResults(ok=True, data={"id": "123", "status": "shipped", "delivery_date": "2023-12-25"})
        return ToolResults(ok=False, errors=["Order not found"])
    
    @staticmethod
    async def list_orders(customer_id):
        return ToolResults(ok=True, data=[{"id": "123", "status": "shipped"}])

    # Staff/Manager Mocks
    @staticmethod
    async def check_stock(query): return ToolResults(ok=True, data={"product": query, "stock": 10})
    @staticmethod
    async def check_price(query): return ToolResults(ok=True, data={"product": query, "price": 500000})
    @staticmethod
    async def lookup_customer(query): return ToolResults(ok=True, data={"id": "cust_1", "name": "Test User"})
    @staticmethod
    async def get_customer_order_history(cid): return ToolResults(ok=True, data=[])
    @staticmethod
    async def create_draft_order(cid, items): return ToolResults(ok=True, data={"id": "draft_1"})
    @staticmethod
    async def update_order_status(oid, status): return ToolResults(ok=True, data={"id": oid, "status": status})
    @staticmethod
    async def print_shipping_label(oid): return ToolResults(ok=True, data={"url": "http://label.pdf"})
    @staticmethod
    async def get_sales_report(): return ToolResults(ok=True, data={"total_sales": 10000000})
    @staticmethod
    async def get_top_products(): return ToolResults(ok=True, data=[{"title": "Bag A", "sales": 100}])
    @staticmethod
    async def get_customer_analytics(): return ToolResults(ok=True, data={"new_users": 50})

# Mock Specialized Agents (to avoid importing their complex logic)
class MockSpecializedAgent:
    def __init__(self, name): self.name = name
    def can_handle(self, intent): return False 
    async def process(self, processed, intent): 
        # Simple pass-through for testing
        return ActionPlan(tools=[], next_step=f"mock_{self.name}"), None

# Define tool objects for Executor to use
cart_tools = MockTools
product_tools = MockTools
product_recommend_tool = MockTools
order_tools = MockTools
staff_tools = MockTools
report_tools = MockTools
system_tools = MockTools
customer_tools = MockTools

In [ ]:
# --- AGENT 1: INPUT PROCESSOR (app/agents/input_processor.py) ---

def _normalize_text(text: str) -> str:
    # Trim, collapse spaces; keep diacritics (for VI) and punctuation minimally
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text


def _detect_language_heuristic(text: str) -> str:
    t = text.lower()
    # Very lightweight heuristic: VI keywords/diacritics vs common EN markers
    if any(ch in t for ch in "ăâêôơưđáàảãạấầẩẫậắằẳẵặéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ"):
        return "vi"
    if any(word in t for word in [" the ", " and ", " price ", " order ", " product ", " hello "]):
        return "en"
    # Fallback: prefer VI as primary market
    return "vi"


async def _fetch_session_context(pool: Any, session_id: str) -> SessionContext:
    # MOCK IMPLEMENTATION
    ctx = SessionContext()
    # Simulate some context if needed
    return ctx


class InputProcessor:
    async def run(self, req: ChatRequest, pool: Any = None) -> ProcessedInput:
        print(f"[InputProcessor] Processing request: session={req.session_id}, msg='{req.message}'")
        cleaned = _normalize_text(req.message)
        lang = req.language or _detect_language_heuristic(cleaned)
        
        # Determine user_type:
        user_type = "guest"
        if req.metadata and "user_type" in req.metadata:
            user_type = req.metadata["user_type"]
        elif req.customer_id:
            user_type = "customer"
            
        session_ctx = await _fetch_session_context(pool, req.session_id)
        print(f"[InputProcessor] Context loaded: last_intent={session_ctx.last_messages[0]['intent'] if session_ctx.last_messages else 'None'}")

        return ProcessedInput(
            session_id=req.session_id,
            customer_id=req.customer_id,
            text=req.message,
            cleaned_text=cleaned,
            language=lang,
            user_type=user_type,
            tag=req.tag,
            session_ctx=session_ctx,
        )

In [ ]:
# --- AGENT 2: INTENT CLASSIFIER (app/agents/intent_classifier.py) ---
# FULL SOURCE CODE

VI_KEYWORDS: Dict[str, List[str]] = {
    "GREETING": ["xin chào", "chào", "alo", "chào bạn", "hello", "hi", "hey", "chào buổi", "chào admin", "chào shop"],
    "PRODUCT.SEARCH": [
        "tìm", "tìm kiếm", "có không", "còn không", "có bán", "muốn mua", "giá", "sản phẩm", 
        "mua", "đặt", "kiếm", "tương tự", "giống", "cần", "shop có", "bên này có", "có hàng",
        "còn hàng", "hết hàng chưa", "còn size", "có màu", "loại", "dòng", "model"
    ],
    "PRODUCT.DETAIL": [
        "chi tiết", "thông số", "xem kỹ", "thông tin", "cụ thể", "mô tả", "spec", 
        "đặc điểm", "tính năng", "chất liệu", "kích thước", "size", "màu sắc", "xuất xứ"
    ],
    "PRODUCT.RECOMMEND": [
        "gợi ý cho tôi", "gợi ý cho mình", "gợi ý vài", "nên mua gì", "mua gì", 
        "hot trend", "bán chạy", "phổ biến", "đề xuất", "recommend me",
        "trending", "mới nhất", "best seller", "top sản phẩm", "sản phẩm tốt", "sản phẩm hay", "đáng mua nhất",
        "xem sản phẩm nào", "có sản phẩm nào", "sản phẩm nào tốt"
    ],
    "ORDER.TRACK": [
        "tra cứu", "kiểm tra", "đơn hàng", "ở đâu", "tình trạng", "tracking", "giao hàng",
        "ship", "vận chuyển", "đã giao chưa", "khi nào nhận", "bao giờ về", "đến chưa"
    ],
    "ORDER.CANCEL": ["hủy đơn", "không mua nữa", "huỷ đơn", "cancel", "không muốn", "đổi ý", "không lấy"],
    "ORDER.RETURN": [
        "đổi trả", "hoàn tiền", "bảo hành", "trả hàng", "return", "refund", "warranty",
        "đổi hàng", "lỗi", "hư", "không vừa", "sai", "không đúng"
    ],
    "CART.VIEW": ["xem giỏ", "giỏ hàng", "trong giỏ", "cart", "giỏ", "đã chọn", "đã thêm"],
    "CART.ADD": [
        "thêm vào giỏ", "bỏ vào giỏ", "mua cái này", "lấy cái này", "đặt cái này", 
        "lấy cho", "thêm 1", "đặt 1", "đặt hàng", "mua ngay", "chốt", "order",
        "cho vào giỏ", "bỏ giỏ", "add", "thêm", "bỏ", "cho tôi", "cho em",
        "mình lấy", "mình mua", "lấy luôn", "mua luôn"
    ],
    "CART.REMOVE": ["xóa khỏi giỏ", "bỏ ra", "xóa sản phẩm", "bỏ sản phẩm", "remove", "không lấy", "bỏ"],
    "CART.UPDATE": ["đổi số lượng", "thay đổi", "sửa giỏ", "update cart", "tăng", "giảm", "đổi size", "đổi màu"],
    "CHECKOUT": ["thanh toán", "checkout", "đặt luôn", "mua luôn", "pay", "trả tiền", "hoàn tất"],
    "ACCOUNT.LOGIN": ["đăng nhập", "login", "sign in", "vào tài khoản"],
    "ACCOUNT.REGISTER": ["đăng ký", "tạo tài khoản", "sign up", "register", "mở tài khoản"],
    "FAQ.SHIPPING": [
        "phí ship", "vận chuyển", "bao lâu", "giao hàng", "ship", "shipping",
        "delivery", "mất bao lâu", "khi nào nhận", "ship cod", "ship nhanh"
    ],
    "FAQ.PAYMENT": [
        "thanh toán", "chuyển khoản", "cod", "trả tiền", "payment", 
        "hình thức thanh toán", "trả góp", "ví", "momo", "zalopay", "banking"
    ],
    "FAQ.PROMO": ["khuyến mãi", "giảm giá", "sale", "voucher", "mã giảm", "coupon", "promotion", "ưu đãi"],
    "FAQ.RETURN": [
        "đổi trả", "chính sách đổi", "return policy", "hoàn tiền", "refund",
        "bảo hành", "warranty", "trả hàng", "đổi hàng", "chính sách hoàn"
    ],
    "SUPPORT.ESCALATE": [
        "nhân viên", "người thật", "không phải bot", "tư vấn", "hỗ trợ", 
        "admin", "staff", "human", "real person", "agent"
    ],
    "THANK": ["cảm ơn", "thanks", "thank you", "cám ơn", "ơn"],
    "GOODBYE": ["tạm biệt", "bye", "goodbye", "hẹn gặp lại", "đi đây"],
    # Staff / Manager Keywords
    "STAFF.CHECK_STOCK": ["check kho", "tồn kho", "kiểm kho", "số lượng tồn", "inventory", "stock"],
    "STAFF.CUSTOMER_LOOKUP": ["tìm khách", "check info khách", "thông tin khách", "lookup customer"],
    "STAFF.ORDER_HISTORY": ["lịch sử mua", "đơn cũ của khách", "khách mua gì", "order history"],
    "STAFF.CREATE_ORDER": ["tạo đơn giúp", "lên đơn", "tạo đơn", "create order", "draft order"],
    "MANAGER.REPORT_SALES": ["doanh thu", "báo cáo bán hàng", "doanh số", "sales report", "revenue"],
    "MANAGER.REPORT_CHATBOT": ["hiệu quả bot", "bot chat bao nhiêu", "thống kê bot", "chatbot stats"],
    "MANAGER.CONFIG_UPDATE": ["tắt bot", "bật bot", "chỉnh prompt", "cấu hình", "config"],
}

EN_KEYWORDS: Dict[str, List[str]] = {
    "GREETING": ["hello", "hi", "hey"],
    "PRODUCT.SEARCH": ["search", "find", "looking for", "do you have", "want to buy", "price", "product", "buy", "order"],
    "PRODUCT.DETAIL": ["detail", "specs", "info", "more info", "specification"],
    "PRODUCT.RECOMMEND": ["recommend", "suggest", "hot trend", "best seller", "popular"],
    "ORDER.TRACK": ["track", "check order", "order status", "where is", "shipping"],
    "ORDER.CANCEL": ["cancel order", "cancel"],
    "ORDER.RETURN": ["return", "refund", "warranty"],
    "CART.VIEW": ["view cart", "my cart", "shopping cart"],
    "CART.ADD": ["add to cart", "buy this", "get this", "order this"],
    "CART.REMOVE": ["remove", "delete item", "remove from cart"],
    "ACCOUNT.LOGIN": ["login", "sign in"],
    "ACCOUNT.REGISTER": ["register", "sign up"],
    "FAQ.SHIPPING": ["shipping cost", "delivery time", "shipping fee"],
    "FAQ.PAYMENT": ["payment", "cod", "bank transfer"],
    "SUPPORT.ESCALATE": ["human", "staff", "real person", "agent"],
    # Staff / Manager Keywords
    "STAFF.CHECK_STOCK": ["check stock", "inventory", "stock level"],
    "STAFF.CUSTOMER_LOOKUP": ["lookup customer", "find customer", "user info"],
    "STAFF.ORDER_HISTORY": ["customer order history", "order history"],
    "STAFF.CREATE_ORDER": ["create order", "draft order"],
    "MANAGER.REPORT_SALES": ["sales report", "revenue", "sales stats"],
    "MANAGER.REPORT_CHATBOT": ["chatbot stats", "bot performance"],
    "MANAGER.CONFIG_UPDATE": ["disable bot", "enable bot", "update config"],
}


def _match_score(text: str, phrases: List[str]) -> int:
    t = f" {text.lower()} "
    score = 0
    for p in phrases:
        if f" {p} " in t:
            score += 2 if len(p.split()) > 1 else 1
    return score


def _extract_product_query(cleaned_text: str) -> str:
    s = cleaned_text
    
    # Check if this is a context reference (not a search query)
    context_patterns = [
        r"\b(cái|sản phẩm|sp)\s+(này|kia|đó|ấy|vừa|trước)",
        r"\b(đầu tiên|thứ nhất|thứ hai|cuối cùng)",
        r"\bnó\b",
        r"\bchi tiết\b.*(?!\w+)",  # "chi tiết" without specific product name
    ]
    for pattern in context_patterns:
        if re.search(pattern, s.lower()):
            return ""  # Return empty to signal this needs context
    
    s = re.sub(r"[?,.]", " ", s) # Remove punctuation
    # remove common Vietnamese/English query anchors
    remove = [
        "tìm", "tìm kiếm", "có không", "còn không", "có bán", "muốn mua",
        "search", "find", "looking for", "do you have", "want to buy",
        "sản phẩm", "product", "mua", "đặt", "kiếm", "cho tôi", "cho anh", "cho em", "cho mình",
        "tương tự", "similar",
        "tôi", "anh", "em", "bạn", "mình", "cái", "chiếc", "là", "của",
        "giá", "bao nhiêu", "price", "cost", "how much", "tiền",
        "thông tin", "xem", "về", "info", "about",
        # Staff keywords
        "check kho", "tồn kho", "kiểm kho", "số lượng tồn",
        "check stock", "inventory", "stock level",
        "tìm khách", "check info khách", "thông tin khách", "khách hàng", "khách",
        "lookup customer", "find customer", "user info",
        "nhân viên", "quản lý", "staff", "manager"
    ]
    low = s.lower()
    for r in remove:
        low = low.replace(r, " ")
    low = re.sub(r"\s+", " ", low).strip()
    return low


class IntentClassifier:
    async def run(self, processed: ProcessedInput) -> IntentResult:
        print(f"[IntentClassifier] Classifying: '{processed.cleaned_text}'")
        text = processed.cleaned_text
        lang = processed.language or "vi"

        # Choose keyword table
        table = VI_KEYWORDS if lang == "vi" else EN_KEYWORDS

        # Compute scores
        scores: Dict[str, int] = {}
        for key, phrases in table.items():
            scores[key] = _match_score(text, phrases)
        
        # Boost PRODUCT.RECOMMEND score if recommend phrases are detected
        # This prevents "gợi ý sản phẩm" from being classified as product_inquiry
        recommend_boost_phrases = ["gợi ý cho", "đề xuất cho", "recommend me", "gợi ý vài", "gợi ý một", "nên mua gì"]
        if any(phrase in text.lower() for phrase in recommend_boost_phrases):
            if "PRODUCT.RECOMMEND" in scores:
                scores["PRODUCT.RECOMMEND"] += 5  # Strong boost to override PRODUCT.SEARCH
        
        # Boost PRODUCT.DETAIL for context reference patterns (e.g., "sản phẩm đầu tiên", "cái này")
        context_patterns = [
            r'\b(cái|sản phẩm|sp)\s+(này|kia|đó|ấy|vừa|trước)',
            r'\b(đầu tiên|thứ nhất|thứ hai|thứ ba|cuối cùng)',
            r'\bnó\b',
        ]
        for pattern in context_patterns:
            if re.search(pattern, text.lower()):
                if "PRODUCT.DETAIL" in scores:
                    scores["PRODUCT.DETAIL"] += 10  # Very strong boost to override PRODUCT.SEARCH
                break

        # Get top 3 intents for multi-intent detection
        sorted_scores = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
        top_key = sorted_scores[0][0] if sorted_scores else "UNKNOWN"
        top_score = sorted_scores[0][1] if sorted_scores else 0
        
        # Detect secondary intent (e.g., "tìm balo giá rẻ" = PRODUCT.SEARCH + PRODUCT.PRICE)
        secondary_intent = None
        if len(sorted_scores) > 1 and sorted_scores[1][1] >= 2:
            secondary_intent = sorted_scores[1][0]
        
        print(f"[IntentClassifier] Top: {top_key}({top_score}), Secondary: {secondary_intent}")

        # Map to system intents used by backend
        intent_map = {
            "GREETING": "general",
            "PRODUCT.SEARCH": "product_inquiry",
            "PRODUCT.DETAIL": "product_detail",
            "PRODUCT.RECOMMEND": "product_recommend",
            "ORDER.TRACK": "order_tracking",
            "ORDER.CANCEL": "order_cancel",
            "ORDER.RETURN": "order_return",
            "CART.VIEW": "cart_view",
            "CART.ADD": "cart_add",
            "CART.REMOVE": "cart_remove",
            "CART.UPDATE": "cart_update",
            "CHECKOUT": "checkout",
            "ACCOUNT.LOGIN": "account_login",
            "ACCOUNT.REGISTER": "account_register",
            "FAQ.SHIPPING": "faq_shipping",
            "FAQ.PAYMENT": "faq_payment",
            "FAQ.PROMO": "faq_promo",
            "FAQ.RETURN": "faq_return",
            "SUPPORT.ESCALATE": "human_escalation",
            "THANK": "thank_you",
            "GOODBYE": "goodbye",
            "STAFF.CHECK_STOCK": "staff_check_stock",
            "STAFF.CUSTOMER_LOOKUP": "staff_customer_lookup",
            "STAFF.ORDER_HISTORY": "staff_order_history",
            "STAFF.CREATE_ORDER": "staff_create_order",
            "MANAGER.REPORT_SALES": "manager_report_sales",
            "MANAGER.REPORT_CHATBOT": "manager_report_chatbot",
            "MANAGER.CONFIG_UPDATE": "manager_config_update",
        }

        mapped = intent_map.get(top_key, "general")
        sub_intent = intent_map.get(secondary_intent) if secondary_intent else None

        # Enhanced confidence scoring
        if top_score >= 4:
            conf = 0.95
        elif top_score >= 3:
            conf = 0.85
        elif top_score >= 2:
            conf = 0.70
        elif top_score >= 1:
            conf = 0.50
        else:
            conf = 0.30

        entities = {}
        
        # Enhanced entity extraction
        if mapped == "product_inquiry" or mapped == "cart_add" or mapped == "product_detail":
            q = _extract_product_query(text)
            # For product_detail, empty query means context reference
            if mapped == "product_detail" and q == "":
                entities["product_query"] = ""  # Signal to use context
            elif q and len(q) >= 2:
                entities["product_query"] = q
            
            # Extract quantity
            qty_match = re.search(r'(\d+)\s*(cái|chiếc|con|bộ|món|sản phẩm|sp)', text.lower())
            if qty_match:
                entities["quantity"] = int(qty_match.group(1))
            else:
                entities["quantity"] = 1
                
        elif mapped == "order_tracking":
            # Enhanced order ID extraction
            # Patterns: #1234, order_1234, ORD-1234, đơn 1234
            match = re.search(r'(order[_\s-]?[\w\d]+|#\d+|ord[_\s-]?\d+|đơn[\s]?\d+|\d{4,})', text.lower())
            if match:
                order_id = match.group(0).replace("#", "").replace("đơn", "").replace("order", "").replace("_", "").replace("-", "").strip()
                entities["order_id"] = order_id
                
        elif mapped == "staff_check_stock":
            q = _extract_product_query(text)
            if q:
                entities["product_query"] = q
                
        elif mapped == "staff_customer_lookup":
            # Extract customer info (email, phone, name)
            email_match = re.search(r'[\w\.-]+@[\w\.-]+\.\w+', text)
            phone_match = re.search(r'(\+?84|0)\d{9,10}', text)
            
            if email_match:
                entities["customer_email"] = email_match.group(0)
            elif phone_match:
                entities["customer_phone"] = phone_match.group(0)
            else:
                # Fallback to query extraction
                q = _extract_product_query(text)
                if q:
                    entities["customer_query"] = q
        
        print(f"[IntentClassifier] Result: intent={mapped}, sub={sub_intent}, entities={entities}")
        return IntentResult(
            intent=mapped,
            sub_intent=sub_intent,
            confidence=conf,
            entities=entities,
        )

In [ ]:
# --- AGENT 3: ORCHESTRATOR (app/agents/orchestrator.py) ---

class Orchestrator:
    def __init__(self):
        self.agents = {
            "sales": MockSpecializedAgent("sales"),
            "order": MockSpecializedAgent("order"),
            "staff": MockSpecializedAgent("staff"),
            "manager": MockSpecializedAgent("manager")
        }

    async def run(
        self,
        processed: ProcessedInput,
        intent: IntentResult,
    ) -> Tuple[ActionPlan, Optional[ToolResults]]:
        """
        Hybrid Orchestrator:
        1. Executable Tag -> Execute Tool directly
        2. Scope Tag -> Route to Specialized Agent (Scoped NLU)
        3. No Tag -> Global NLU -> Route to Agent
        """
        print(f"[Orchestrator] Routing... Tag={processed.tag}, Intent={intent.intent}")
        
        # 1. Executable Tag (e.g., "action:view_cart")
        if processed.tag and processed.tag.startswith("action:"):
            action = processed.tag.split(":", 1)[1]
            print(f"[Orchestrator] Direct action: {action}")
            
            if action == "view_cart":
                return ActionPlan(tools=["cart.view"], next_step="show_cart"), None
            elif action == "order_list":
                return ActionPlan(tools=["order.list"], next_step="show_orders"), None
            elif action == "order_track":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "reorder":
                return ActionPlan(tools=["order.reorder"], next_step="confirm_reorder"), None
            elif action == "logout":
                return ActionPlan(tools=["auth.logout"], next_step="confirm_logout"), None
            elif action == "chat_human":
                return ActionPlan(tools=["system.escalate"], next_step="escalate_chat"), None
            elif action == "my_reviews":
                return ActionPlan(tools=["product.reviews"], next_step="show_reviews"), None
            elif action == "check_stock":
                return ActionPlan(tools=[], next_step="ask_product_name"), None # Need product name
            elif action == "check_price":
                return ActionPlan(tools=[], next_step="ask_product_name"), None # Need product name
            elif action == "staff_lookup_order":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "staff_customer_lookup":
                return ActionPlan(tools=[], next_step="ask_customer_info"), None
            elif action == "update_order_status":
                return ActionPlan(tools=[], next_step="ask_order_id_status"), None
            elif action == "print_shipping_label":
                return ActionPlan(tools=[], next_step="ask_order_id"), None
            elif action == "sales_report":
                return ActionPlan(tools=["manager.report_sales"], next_step="show_report"), None
            elif action == "top_products":
                return ActionPlan(tools=["manager.top_products"], next_step="show_top_products"), None
            elif action == "customer_analytics":
                return ActionPlan(tools=["manager.customer_analytics"], next_step="show_analytics"), None
            elif action.startswith("cart.add_item"):
                try:
                    params = {}
                    if "?" in action:
                        query = action.split("?", 1)[1]
                        for pair in query.split("&"):
                            if "=" in pair:
                                k, v = pair.split("=", 1)
                                params[k] = v
                    
                    intent.entities["product_id"] = params.get("product_id")
                    intent.entities["variant_id"] = params.get("variant_id")
                    intent.entities["quantity"] = 1
                    
                    return ActionPlan(tools=["cart.add"], next_step="confirm_add_to_cart"), None
                except Exception as e:
                    print(f"Error parsing cart.add_item tag: {e}")
                    return ActionPlan(tools=[], next_step="error"), None
            
            # Add more direct actions here
        
        # 2. Scope Tag (e.g., "scope:order")
        if processed.tag and processed.tag.startswith("scope:"):
            scope = processed.tag.split(":", 1)[1]
            agent = self.agents.get(scope)
            if agent:
                print(f"[Orchestrator] Scoped routing to: {scope}")
                # Agent performs its own scoped NLU if intent is generic/unknown or we want to re-evaluate
                # For now, we pass None as intent to force agent to re-classify within scope
                return await agent.process(processed, intent=None)
        
        # 3. Implicit Routing (Global NLU)
        # We use the intent passed from the global classifier to decide which agent to call
        target_agent = self._route_by_intent(intent.intent)
        if target_agent:
            print(f"[Orchestrator] Implicit routing to: {target_agent.name}")
            return await target_agent.process(processed, intent)
            
        # Default fallback
        print("[Orchestrator] No agent found, fallback.")
        return ActionPlan(tools=[], next_step=None), None

    def _route_by_intent(self, intent_name: str):
        """Route intent to appropriate specialized agent"""
        for agent in self.agents.values():
            if agent.can_handle(intent_name):
                return agent
        return self.agents["sales"] # Default to sales if unknown
    
    def get_agent_suggestions(self, intent: str) -> list:
        """Get suggested actions based on intent - for better UX"""
        suggestions = {
            "product_inquiry": [
                {"label": "Xem giỏ hàng", "action": "action:view_cart"},
                {"label": "Sản phẩm gợi ý", "action": "action:my_recommendations"},
            ],
            "cart_view": [
                {"label": "Thanh toán ngay", "action": "action:checkout"},
                {"label": "Tiếp tục mua sắm", "action": "action:continue_shopping"},
            ],
            "order_tracking": [
                {"label": "Đơn hàng của tôi", "action": "action:order_list"},
                {"label": "Liên hệ hỗ trợ", "action": "action:chat_human"},
            ],
            "general": [
                {"label": "Tìm sản phẩm", "action": "action:product_search"},
                {"label": "Xem giỏ hàng", "action": "action:view_cart"},
                {"label": "Kiểm tra đơn hàng", "action": "action:order_list"},
            ],
        }
        return suggestions.get(intent, suggestions["general"])

In [ ]:
# --- AGENT 4: EXECUTOR (app/agents/executor.py) ---

def _merge_tool_result(results: ToolResults, tool_result: Any):
    if tool_result is None:
        return
    if isinstance(tool_result, ToolResults):
        results.ok = results.ok and tool_result.ok
        if tool_result.errors:
            results.errors.extend(tool_result.errors)
        if tool_result.timings_ms:
            results.timings_ms.update(tool_result.timings_ms)
        results.data = tool_result.data
    else:
        results.data = tool_result

class Executor:
    async def run(
        self, 
        processed: ProcessedInput, 
        intent: IntentResult, 
        plan: ActionPlan
    ) -> Optional[ToolResults]:
        if not plan.tools:
            return None

        results = ToolResults(ok=True)
        
        for tool_name in plan.tools:
            try:
                logger.info(f"[Executor] Running tool: {tool_name}")
                
                # Cart Tools
                if tool_name == "cart.view":
                    if processed.session_ctx.cart_id:
                        res = await cart_tools.view_cart(processed.session_ctx.cart_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing cart_id")
                elif tool_name == "cart.add":
                    # Extract product and quantity from entities
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    qty = intent.entities.get("quantity", 1)
                    variant_id = intent.entities.get("variant_id")
                    
                    if processed.session_ctx.cart_id:
                        if variant_id:
                            # If we have explicit variant_id (e.g. from UI action), use it directly
                            res = await cart_tools.add_to_cart(processed.session_ctx.cart_id, variant_id, qty)
                        else:
                            # Otherwise try smart add (search -> add)
                            res = await cart_tools.add_to_cart_smart(processed.session_ctx.cart_id, query, qty)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing cart_id")
                
                # Product Tools
                elif tool_name == "product.search":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await product_tools.search_products(query)
                    _merge_tool_result(results, res)
                elif tool_name == "product.recommend":
                    # The tool currently only accepts limit, not customer_id
                    res = await product_recommend_tool.recommend_products(limit=5)
                    _merge_tool_result(results, res)
                elif tool_name == "product.detail":
                    # Try to get product_id from entities first
                    product_id = intent.entities.get("product_id")
                    
                    # If no product_id in entities, try to get from context (last products shown)
                    if not product_id and processed.session_ctx.last_product_ids:
                        product_id = processed.session_ctx.last_product_ids[0]
                        logger.info(f"[Executor] Using context product_id: {product_id}")
                    
                    if product_id:
                        res = await product_tools.get_product_details(product_id)
                        _merge_tool_result(results, res)
                    else:
                        # Fallback: try to search if we have a query
                        # Extract product name from text (remove "chi tiết", "xem", etc)
                        import re
                        query = intent.entities.get("product_query")
                        if not query:
                            # Try to extract product name from cleaned text
                            text_lower = processed.cleaned_text.lower()
                            # Remove common phrases (order matters - remove longer phrases first)
                            for phrase in ["cho tôi xem chi tiết", "cho tôi xem", "xem chi tiết", "chi tiết của", "chi tiết", "xem", "tôi muốn", "muốn"]:
                                text_lower = text_lower.replace(phrase, "")
                            query = text_lower.strip()
                        
                        if query and len(query) >= 2:
                            logger.info(f"[Executor] Searching for product: '{query}'")
                            res = await product_tools.search_products(query, limit=1)
                            _merge_tool_result(results, res)
                            # Store search query in entities for response_generator
                            intent.entities["product_query"] = query
                        else:
                            results.ok = False
                            results.errors.append("Missing product_id or query")
                            # Still store a placeholder for template
                            intent.entities["product_query"] = "sản phẩm bạn cần"

                # Order Tools
                elif tool_name == "order.track":
                    order_id = intent.entities.get("order_id")
                    email = intent.entities.get("email") # Might need to ask user
                    if order_id:
                        res = await order_tools.lookup_order(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "order.list":
                    if processed.customer_id:
                        res = await order_tools.list_orders(processed.customer_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "order.reorder":
                    # Assuming we reorder the last order or specific order
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await order_tools.reorder(order_id)
                        _merge_tool_result(results, res)
                    else:
                        # Maybe list orders to pick?
                        results.ok = False
                        results.errors.append("Missing order_id")

                # Staff Tools
                elif tool_name == "staff.check_stock":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await staff_tools.check_stock(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.check_price":
                    query = intent.entities.get("product_query", processed.cleaned_text)
                    res = await staff_tools.check_price(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.customer_lookup":
                    query = intent.entities.get("customer_query", processed.cleaned_text)
                    res = await staff_tools.lookup_customer(query)
                    _merge_tool_result(results, res)
                elif tool_name == "staff.order_history":
                    customer_id = intent.entities.get("customer_id")
                    if customer_id:
                        res = await staff_tools.get_customer_order_history(customer_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "staff.create_order":
                    if processed.customer_id:
                        res = await staff_tools.create_draft_order(processed.customer_id, [])
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing customer_id")
                elif tool_name == "staff.lookup_order":
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await staff_tools.lookup_order(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "staff.update_order":
                    order_id = intent.entities.get("order_id")
                    status = intent.entities.get("status", "processing")
                    if order_id:
                        res = await staff_tools.update_order_status(order_id, status)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                elif tool_name == "staff.print_label":
                    order_id = intent.entities.get("order_id")
                    if order_id:
                        res = await staff_tools.print_shipping_label(order_id)
                        _merge_tool_result(results, res)
                    else:
                        results.ok = False
                        results.errors.append("Missing order_id")
                
                # Manager Tools
                elif tool_name == "manager.report_sales":
                    res = await report_tools.get_sales_report()
                    _merge_tool_result(results, res)
                elif tool_name == "manager.top_products":
                    res = await report_tools.get_top_products()
                    _merge_tool_result(results, res)
                elif tool_name == "manager.customer_analytics":
                    res = await report_tools.get_customer_analytics()
                    _merge_tool_result(results, res)

                # System/Auth Tools
                elif tool_name == "system.escalate":
                    # Just a marker, maybe log it
                    results.data = {"status": "escalated", "message": "Connecting to human agent..."}
                elif tool_name == "auth.logout":
                    # In a real app, this might invalidate session
                    results.data = {"status": "logged_out", "message": "You have been logged out."}
                
                # Product Tools (Extra)
                elif tool_name == "product.reviews":
                    # Mock
                    results.data = {"reviews": []}

                else:
                    logger.warning(f"[Executor] Unknown tool: {tool_name}")
                    results.errors.append(f"Unknown tool: {tool_name}")

            except Exception as e:
                logger.error(f"[Executor] Error running {tool_name}: {e}")
                results.ok = False
                results.errors.append(str(e))
        
        return results

In [ ]:
# --- AGENT 5: RESPONSE GENERATOR (app/agents/response_generator.py) ---

class ResponseGenerator:
    def __init__(self):
        if GOOGLE_API_KEY == "YOUR_API_KEY_HERE":
            self.client = None
        else:
            from openai import AsyncOpenAI
            self.client = AsyncOpenAI(api_key=GOOGLE_API_KEY, base_url=GEMINI_BASE_URL)
        self.context_config = CONTEXT_CONFIG

    def _load_context_config(self):
        return CONTEXT_CONFIG

    async def run(
        self, 
        processed: ProcessedInput, 
        intent: IntentResult, 
        plan: ActionPlan, 
        tool_res: Optional[ToolResults]
    ) -> ChatResponse:
        
        # 1. Generate Text Response
        response_text = await self._generate_llm_response(processed, intent, tool_res)
        
        # 2. Generate Quick Replies (Options)
        quick_replies = self._generate_quick_replies(processed)
        
        # 3. Format Products if available
        products = []
        if tool_res and tool_res.ok and isinstance(tool_res.data, list):
            for item in tool_res.data:
                # Try to map to ProductInfo
                try:
                    # If item is already ProductInfo, append directly
                    if isinstance(item, ProductInfo):
                        products.append(item)
                    # Handle if item is dict
                    elif isinstance(item, dict):
                        # Filter keys that match ProductInfo
                        valid_keys = ProductInfo.model_fields.keys()
                        filtered = {k: v for k, v in item.items() if k in valid_keys}
                        # Ensure required fields
                        if "id" in filtered and "title" in filtered and "handle" in filtered:
                            products.append(ProductInfo(**filtered))
                    # Handle if item is object
                    elif hasattr(item, "id") and hasattr(item, "title"):
                        # Manually map variants if they exist in the object but weren't automatically mapped
                        variants = getattr(item, "variants", [])
                        # If variants are already ProductVariantInfo objects, use them.
                        # If they are dicts, convert them.
                        mapped_variants = []
                        for v in variants:
                            if isinstance(v, dict):
                                mapped_variants.append(v) # Pydantic will handle dict -> model
                            else:
                                mapped_variants.append(v)

                        products.append(ProductInfo(
                            id=item.id,
                            title=item.title,
                            handle=item.handle,
                            thumbnail=getattr(item, "thumbnail", None),
                            price=getattr(item, "price", None),
                            currency_code=getattr(item, "currency_code", None),
                            variants=mapped_variants
                        ))
                except Exception as e:
                    logger.warning(f"Failed to map product item: {e}")
        
        # Add product-specific quick replies if products found
        if products:
            # Add "Buy Now" button first
            quick_replies.insert(0, QuickReply(label="Mua ngay", value=f"Đặt hàng {products[0].title}"))
            # Add "View Details" button second
            quick_replies.insert(1, QuickReply(label="Xem chi tiết", value=f"Chi tiết {products[0].title}"))

        return ChatResponse(
            response=response_text,
            session_id=processed.session_id,
            products=products,
            quick_replies=quick_replies,
            metadata={"intent": intent.intent}
        )

    async def _generate_llm_response(self, processed, intent, tool_res) -> str:
        # Try template-based response first for common cases
        template_context = {}
        
        print(f"[ResponseGen] intent={intent.intent}, tool_res={'exists' if tool_res else 'None'}, tool_res.ok={tool_res.ok if tool_res else 'N/A'}, data_len={len(tool_res.data) if tool_res and tool_res.ok else 0}")
        
        # PRIORITY 1: Handle FAQ intents with templates (avoid LLM hallucination)
        if intent.intent in ["faq_shipping", "faq_payment", "faq_return", "faq_promo"]:
            print(f"[ResponseGen] Using FAQ template for {intent.intent}")
            return get_response_template(intent.intent)
        
        # PRIORITY 2: Handle thank_you/goodbye
        if intent.intent == "thank_you":
            return get_response_template("thank_you")
        elif intent.intent == "goodbye":
            return get_response_template("goodbye")
        
        if tool_res:
            if tool_res.ok and isinstance(tool_res.data, list):
                template_context['count'] = len(tool_res.data)
                if len(tool_res.data) > 0:
                    first_item = tool_res.data[0]
                    if isinstance(first_item, dict):
                        template_context['product'] = first_item.get('title', '')
                    elif hasattr(first_item, 'title'):
                        template_context['product'] = first_item.title
            else:
                # tool_res.ok = False means tool failed/no results
                template_context['count'] = 0
        
        # PRIORITY 3: Handle product intents with templates (even without tool_res)
        if intent.intent == "general" and (not tool_res or not tool_res.ok):
            print("[ResponseGen] Using greeting template")
            return get_response_template("greeting")
        elif intent.intent in ["product_inquiry", "product_detail"]:
            # Get product query from entities (may be set by executor)
            query = intent.entities.get("product_query")
            if query:
                template_context['query'] = query
            else:
                # Fallback: extract from processed text
                template_context['query'] = "sản phẩm bạn cần"
            
            if template_context.get('count', 0) > 0:
                print(f"[ResponseGen] Using product_found template, count={template_context['count']}")
                return get_response_template("product_found", template_context)
            else:
                # Product not found - use template to avoid LLM hallucination
                print(f"[ResponseGen] Using product_not_found template, query={template_context.get('query')}")
                return get_response_template("product_not_found", template_context)
        elif intent.intent == "product_recommend":
            if tool_res.ok and template_context.get('count', 0) > 0:
                return get_response_template("product_recommend")
            else:
                return "Xin lỗi, shop đang cập nhật sản phẩm. Bạn quay lại sau nhé!"
        elif intent.intent == "cart_add":
            if tool_res.ok:
                return get_response_template("cart_added", template_context)
            else:
                return "Xin lỗi, tôi cần biết sản phẩm nào bạn muốn thêm vào giỏ hàng."
        
        # PRIORITY 4: Fallback to LLM only for complex cases
        # But prevent hallucination by checking tool_res.ok
        if tool_res and not tool_res.ok:
            # Tool failed - don't let LLM make up data
            print(f"[ResponseGen] Tool failed, using error template")
            if intent.intent in ["product_inquiry", "product_detail"]:
                return get_response_template("product_not_found", template_context)
            return "Xin lỗi, tôi không tìm thấy thông tin bạn cần. Bạn có thể hỏi lại được không?"
        
        # Construct prompt for LLM (only when tool succeeded or no tool needed)
        print(f"[ResponseGen] Falling through to LLM generation")
        context_str = ""
        if tool_res:
            if tool_res.ok:
                # Truncate data if too large
                data_str = json.dumps(tool_res.data, ensure_ascii=False, default=str)
                if len(data_str) > 3000: # Increased limit for better context
                    data_str = data_str[:3000] + "...(truncated)"
                context_str = f"Tool Result: {data_str}"
            else:
                context_str = f"Tool Error: {tool_res.errors}"
        
        # Add currency hint if available in tool result (e.g. from product search)
        currency_hint = ""
        if tool_res and tool_res.ok and isinstance(tool_res.data, list) and len(tool_res.data) > 0:
            first_item = tool_res.data[0]
            if isinstance(first_item, dict) and "currency_code" in first_item:
                 currency_hint = f"\nLưu ý: Đơn vị tiền tệ đang sử dụng là {first_item.get('currency_code', 'N/A').upper()}."
            elif hasattr(first_item, "currency_code"):
                 currency_hint = f"\nLưu ý: Đơn vị tiền tệ đang sử dụng là {getattr(first_item, 'currency_code', 'N/A').upper()}."

        messages = [
            {"role": "system", "content": SYSTEM_PROMPT_VI + currency_hint},
            {"role": "user", "content": f"User Input: {processed.text}\nIntent: {intent.intent}\nContext: {context_str}"}
        ]
        
        if self.client:
            try:
                completion = await self.client.chat.completions.create(
                    model=GEMINI_MODEL,
                    messages=messages,
                    temperature=0.7
                )
                return completion.choices[0].message.content or ""
            except Exception as e:
                logger.error(f"LLM Error: {e}")
                return get_response_template("error_generic")
        else:
            return f"[MOCK LLM RESPONSE] Based on {intent.intent} and data: {tool_res.data if tool_res else 'None'}"

    def _generate_quick_replies(self, processed) -> List[QuickReply]:
        nodes = self.context_config.get("nodes", [])
        
        # Helper to find node by ID recursively
        def find_node(node_list, target_id):
            for node in node_list:
                if node.get("id") == target_id:
                    return node
                if "children" in node:
                    found = find_node(node["children"], target_id)
                    if found: return found
            return None

        # Determine current context node
        current_node = None
        if processed.tag:
            # If tag is like "action:view_cart", we might not find a node if it's a leaf.
            # But if tag is a group ID, we might.
            # Let's try to find by ID first (assuming tag might be ID)
            current_node = find_node(nodes, processed.tag)
        
        # If no specific node found, default to user root
        if not current_node:
            user_root_id = f"{processed.user_type}_root"
            current_node = find_node(nodes, user_root_id)

        if not current_node:
            return []

        options = []
        # Return children of the current node
        for child in current_node.get("children", []):
            # Determine value: prefer value, then tag, then id
            val = child.get("value") or child.get("tag") or child.get("id")
            options.append(QuickReply(
                label=child["label"],
                value=val,
                metadata={"type": child.get("type")}
            ))
            
        return options

In [ ]:
# --- DATA LOADING ---
# Upload 'multi_agent_test_cases_full.csv' to Colab files
try:
    df = pd.read_csv('multi_agent_test_cases_full.csv')
    print(f"Loaded {len(df)} test cases.")
except FileNotFoundError:
    print("File not found. Creating sample data...")
    data = {
        "id": [1, 2, 3],
        "user_input": ["Xin chào shop", "Tìm balo JanSport màu đen", "Kiểm tra đơn hàng #123"],
        "expected_intent_classifier_intent": ["general", "product_inquiry", "order_tracking"],
        "expected_executor_tools_called": ["", "product.search", "order.track"]
    }
    df = pd.DataFrame(data)
    print("Created sample data.")

df.head()

In [ ]:
# --- EVALUATION LOOP ---

async def evaluate_row(row):
    # Initialize Agents
    input_processor = InputProcessor()
    intent_classifier = IntentClassifier()
    orchestrator = Orchestrator()
    executor = Executor()
    response_generator = ResponseGenerator()

    # 1. Input
    req = ChatRequest(session_id=f"sess_{row['id']}", message=row['user_input'])
    processed = await input_processor.run(req)
    
    # 2. Intent
    intent_res = await intent_classifier.run(processed)
    
    # 3. Orchestrator
    plan, _ = await orchestrator.run(processed, intent_res)
    
    # 4. Executor
    tool_res = await executor.run(processed, intent_res, plan)
    
    # 5. Response
    response = await response_generator.run(processed, intent_res, plan, tool_res)
    
    # --- CHECKS ---
    results = {
        "id": row['id'],
        "input": row['user_input'],
        "intent_actual": intent_res.intent,
        "intent_expected": row.get('expected_intent_classifier_intent'),
        "intent_match": intent_res.intent == row.get('expected_intent_classifier_intent'),
        
        "tools_actual": ",".join(plan.tools),
        "tools_expected": row.get('expected_executor_tools_called', ""),
        # Simple check: is expected tool in actual tools?
        "tools_match": str(row.get('expected_executor_tools_called', "")) in ",".join(plan.tools) if pd.notna(row.get('expected_executor_tools_called')) else True,
        
        "response_preview": response.response[:50] + "..."
    }
    return results

# Run Evaluation
results_list = []
print("Starting evaluation...")
for index, row in df.iterrows():
    res = await evaluate_row(row)
    results_list.append(res)
    print(f"Row {row['id']}: Intent Match={res['intent_match']}, Tools Match={res['tools_match']}")

results_df = pd.DataFrame(results_list)

In [ ]:
# --- SUMMARY REPORT ---
print("\n--- EVALUATION SUMMARY ---")
total = len(results_df)
intent_acc = results_df['intent_match'].mean() * 100
tools_acc = results_df['tools_match'].mean() * 100

print(f"Total Test Cases: {total}")
print(f"Intent Accuracy: {intent_acc:.2f}%")
print(f"Tool Selection Accuracy: {tools_acc:.2f}%")

print("\n--- FAILED CASES ---")
failed = results_df[~results_df['intent_match'] | ~results_df['tools_match']]
if not failed.empty:
    display(failed[['id', 'input', 'intent_actual', 'intent_expected', 'tools_actual', 'tools_expected']])
else:
    print("All cases passed!")

In [ ]:
!pip install pydantic openai pandas python-dotenv nest_asyncio pyyaml

In [ ]:
import os
import json
import re
import yaml
import asyncio
import logging
import pandas as pd
import nest_asyncio
from typing import List, Dict, Any, Optional, Tuple
from pydantic import BaseModel, Field
from datetime import datetime

# Apply nest_asyncio to allow async loops in notebook
nest_asyncio.apply()

# Configure Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("Evaluation")

In [ ]:
# --- CONFIGURATION ---
# Replace with your actual API Key or use a dummy one for testing logic only
GOOGLE_API_KEY = "YOUR_API_KEY_HERE" 
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
GEMINI_MODEL = "gemini-1.5-flash"

SYSTEM_PROMPT_VI = """
Bạn là trợ lý ảo AI chuyên nghiệp của cửa hàng JanSport Việt Nam.
Nhiệm vụ của bạn là hỗ trợ khách hàng tìm kiếm sản phẩm, kiểm tra đơn hàng và giải đáp thắc mắc.
Luôn trả lời bằng tiếng Việt, giọng điệu thân thiện, nhiệt tình và chuyên nghiệp.
"""

# Mock Context Config
CONTEXT_CONFIG = {
    "nodes": [
        {
            "id": "guest_root",
            "label": "Guest Root",
            "children": [
                {"label": "Tìm sản phẩm", "value": "action:product_search"},
                {"label": "Tra cứu đơn hàng", "value": "action:order_track"}
            ]
        },
        {
            "id": "customer_root",
            "label": "Customer Root",
            "children": [
                {"label": "Đơn hàng của tôi", "value": "action:order_list"},
                {"label": "Giỏ hàng", "value": "action:view_cart"}
            ]
        }
    ]
}

In [ ]:
# --- DATA MODELS (agent_types.py) ---

class SessionContext(BaseModel):
    last_messages: List[Dict[str, Any]] = Field(default_factory=list)
    cart_id: Optional[str] = None
    last_product_ids: List[str] = Field(default_factory=list)

class ProcessedInput(BaseModel):
    session_id: str
    customer_id: Optional[str] = None
    text: str
    cleaned_text: str
    language: str = "vi"
    user_type: str = "guest"
    tag: Optional[str] = None
    session_ctx: SessionContext = Field(default_factory=SessionContext)

class IntentResult(BaseModel):
    intent: str
    sub_intent: Optional[str] = None
    confidence: float = 0.0
    entities: Dict[str, Any] = Field(default_factory=dict)

class ActionPlan(BaseModel):
    tools: List[str] = Field(default_factory=list)
    required_data: List[str] = Field(default_factory=list)
    next_step: Optional[str] = None
    escalate: bool = False

class ToolResults(BaseModel):
    ok: bool
    data: Any = None
    errors: List[str] = Field(default_factory=list)
    timings_ms: Dict[str, int] = Field(default_factory=dict)

class ProductInfo(BaseModel):
    id: str
    title: str
    handle: str
    thumbnail: Optional[str] = None
    price: Optional[int] = None
    currency_code: Optional[str] = None
    variants: List[Dict[str, Any]] = Field(default_factory=list)

class QuickReply(BaseModel):
    label: str
    value: str
    metadata: Optional[Dict[str, Any]] = None

class ChatResponse(BaseModel):
    response: str
    session_id: str
    products: List[ProductInfo] = Field(default_factory=list)
    quick_replies: List[QuickReply] = Field(default_factory=list)
    metadata: Dict[str, Any] = Field(default_factory=dict)

class ChatRequest(BaseModel):
    session_id: str
    message: str
    customer_id: Optional[str] = None
    language: Optional[str] = None
    metadata: Optional[Dict[str, Any]] = None
    tag: Optional[str] = None

In [ ]:
# --- RESPONSE TEMPLATES (response_templates.py) ---

RESPONSE_TEMPLATES = {
    "greeting": ["Xin chào! Tôi là trợ lý ảo của shop. Tôi có thể giúp gì cho bạn hôm nay? 😊"],
    "product_found": ["Mình tìm thấy {count} sản phẩm phù hợp với '{query}' ạ:"],
    "product_not_found": ["Rất tiếc, shop không tìm thấy sản phẩm '{query}' bạn cần 😔"],
    "product_recommend": ["🌟 Đây là một số sản phẩm hot mà bạn có thể thích:"],
    "cart_added": ["Đã thêm {product} vào giỏ hàng! Bạn muốn tiếp tục mua sắm hay thanh toán luôn?"],
    "cart_view_empty": ["Giỏ hàng của bạn đang trống. Tìm sản phẩm ngay nhé! 🛍️"],
    "cart_view_has_items": ["Giỏ hàng của bạn có {count} sản phẩm:"],
    "order_found": ["Đơn hàng #{order_id} của bạn:\nTrạng thái: {status}\nDự kiến giao: {delivery_date}"],
    "order_not_found": ["Không tìm thấy đơn hàng #{order_id}. Bạn kiểm tra lại mã đơn giúp mình nhé!"],
    "faq_shipping": ["🚚 Thông tin vận chuyển:\n- Nội thành: 2-3 ngày (30k)\n- Ngoại thành: 3-5 ngày (50k)\n- FREE SHIP đơn > 500k"],
    "faq_return": ["🔄 Chính sách đổi trả: Miễn phí trong 7 ngày nếu còn nguyên tem mác."],
    "faq_payment": ["💳 Thanh toán: COD, Chuyển khoản, Thẻ, Ví điện tử."],
    "faq_promo": ["🎁 Khuyến mãi: Giảm 10% đơn đầu tiên."],
    "thank_you": ["Không có gì! Rất vui được hỗ trợ bạn. 😊"],
    "goodbye": ["Tạm biệt! Hẹn gặp lại bạn sớm. 👋"],
    "error_generic": ["Xin lỗi, có lỗi xảy ra. Bạn có thể thử lại không?"]
}

def get_response_template(key: str, context: Dict[str, Any] = None) -> str:
    import random
    templates = RESPONSE_TEMPLATES.get(key, RESPONSE_TEMPLATES["error_generic"])
    template = random.choice(templates)
    if context:
        try:
            return template.format(**context)
        except KeyError:
            return template
    return template

In [ ]:
# --- AGENT 1: INPUT PROCESSOR (input_processor.py) ---

def _normalize_text(text: str) -> str:
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text

def _detect_language_heuristic(text: str) -> str:
    t = text.lower()
    if any(ch in t for ch in "ăâêôơưđáàảãạấầẩẫậắằẳẵặéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ"):
        return "vi"
    if any(word in t for word in [" the ", " and ", " price ", " order ", " product ", " hello "]):
        return "en"
    return "vi"

# MOCK: No DB connection
async def _fetch_session_context_mock(session_id: str) -> SessionContext:
    # Return a dummy context or empty one
    return SessionContext()

class InputProcessor:
    async def run(self, req: ChatRequest) -> ProcessedInput:
        print(f"[InputProcessor] Processing request: session={req.session_id}, msg='{req.message}'")
        cleaned = _normalize_text(req.message)
        lang = req.language or _detect_language_heuristic(cleaned)
        
        user_type = "guest"
        if req.metadata and "user_type" in req.metadata:
            user_type = req.metadata["user_type"]
        elif req.customer_id:
            user_type = "customer"
            
        session_ctx = await _fetch_session_context_mock(req.session_id)

        return ProcessedInput(
            session_id=req.session_id,
            customer_id=req.customer_id,
            text=req.message,
            cleaned_text=cleaned,
            language=lang,
            user_type=user_type,
            tag=req.tag,
            session_ctx=session_ctx,
        )

In [ ]:
# --- AGENT 2: INTENT CLASSIFIER (intent_classifier.py) ---

VI_KEYWORDS: Dict[str, List[str]] = {
    "GREETING": ["xin chào", "chào", "alo", "hello", "hi"],
    "PRODUCT.SEARCH": ["tìm", "mua", "giá", "sản phẩm", "có không", "balo", "túi"],
    "PRODUCT.DETAIL": ["chi tiết", "thông số", "kỹ", "thông tin", "mô tả"],
    "PRODUCT.RECOMMEND": ["gợi ý", "nên mua", "hot trend", "bán chạy", "đề xuất"],
    "ORDER.TRACK": ["tra cứu", "kiểm tra", "đơn hàng", "vận chuyển", "giao hàng"],
    "CART.VIEW": ["xem giỏ", "giỏ hàng", "cart"],
    "CART.ADD": ["thêm vào giỏ", "mua cái này", "đặt hàng", "lấy cái này"],
    "CART.REMOVE": ["xóa khỏi giỏ", "bỏ ra", "không lấy"],
    "FAQ.SHIPPING": ["phí ship", "bao lâu", "giao hàng"],
    "FAQ.PAYMENT": ["thanh toán", "chuyển khoản", "cod"],
    "FAQ.RETURN": ["đổi trả", "hoàn tiền", "bảo hành"],
    "THANK": ["cảm ơn", "thanks"],
    "GOODBYE": ["tạm biệt", "bye"],
}
# (Simplified for brevity, full list in original code)

def _match_score(text: str, phrases: List[str]) -> int:
    t = f" {text.lower()} "
    score = 0
    for p in phrases:
        if f" {p} " in t:
            score += 2 if len(p.split()) > 1 else 1
    return score

def _extract_product_query(cleaned_text: str) -> str:
    s = cleaned_text
    remove = ["tìm", "muốn mua", "cho tôi", "giá", "sản phẩm", "chi tiết"]
    low = s.lower()
    for r in remove:
        low = low.replace(r, " ")
    return re.sub(r"\s+", " ", low).strip()

class IntentClassifier:
    async def run(self, processed: ProcessedInput) -> IntentResult:
        text = processed.cleaned_text
        scores = {}
        for key, phrases in VI_KEYWORDS.items():
            scores[key] = _match_score(text, phrases)
        
        sorted_scores = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
        top_key = sorted_scores[0][0] if sorted_scores and sorted_scores[0][1] > 0 else "UNKNOWN"
        top_score = sorted_scores[0][1] if sorted_scores else 0
        
        intent_map = {
            "GREETING": "general", "PRODUCT.SEARCH": "product_inquiry",
            "PRODUCT.DETAIL": "product_detail", "PRODUCT.RECOMMEND": "product_recommend",
            "ORDER.TRACK": "order_tracking", "CART.VIEW": "cart_view",
            "CART.ADD": "cart_add", "CART.REMOVE": "cart_remove",
            "FAQ.SHIPPING": "faq_shipping", "FAQ.PAYMENT": "faq_payment",
            "FAQ.RETURN": "faq_return", "THANK": "thank_you", "GOODBYE": "goodbye"
        }
        mapped = intent_map.get(top_key, "general")
        
        entities = {}
        if mapped in ["product_inquiry", "cart_add", "product_detail"]:
            q = _extract_product_query(text)
            if q: entities["product_query"] = q
            
        return IntentResult(intent=mapped, confidence=0.9 if top_score > 0 else 0.0, entities=entities)

In [ ]:
# --- AGENT 3: ORCHESTRATOR (orchestrator.py) ---

# Mock Specialized Agents
class MockSpecializedAgent:
    def __init__(self, name): self.name = name
    def can_handle(self, intent): return False # Simplified: always fallback to default routing
    async def process(self, processed, intent): return ActionPlan(tools=[], next_step=f"mock_{self.name}"), None

class Orchestrator:
    def __init__(self):
        self.agents = {
            "sales": MockSpecializedAgent("sales"),
            "order": MockSpecializedAgent("order"),
            "staff": MockSpecializedAgent("staff"),
            "manager": MockSpecializedAgent("manager")
        }

    async def run(self, processed: ProcessedInput, intent: IntentResult) -> Tuple[ActionPlan, Optional[ToolResults]]:
        print(f"[Orchestrator] Routing... Intent={intent.intent}")
        
        # 1. Direct Action Tag
        if processed.tag and processed.tag.startswith("action:"):
            action = processed.tag.split(":", 1)[1]
            if action == "view_cart": return ActionPlan(tools=["cart.view"], next_step="show_cart"), None
            if action == "product_search": return ActionPlan(tools=[], next_step="ask_product"), None
            # ... others
            
        # 2. Implicit Routing (Simplified Logic)
        if intent.intent == "product_inquiry":
            return ActionPlan(tools=["product.search"], next_step="show_products"), None
        elif intent.intent == "product_detail":
            return ActionPlan(tools=["product.detail"], next_step="show_detail"), None
        elif intent.intent == "product_recommend":
            return ActionPlan(tools=["product.recommend"], next_step="show_recommendations"), None
        elif intent.intent == "cart_view":
            return ActionPlan(tools=["cart.view"], next_step="show_cart"), None
        elif intent.intent == "cart_add":
            return ActionPlan(tools=["cart.add"], next_step="confirm_add"), None
        elif intent.intent == "order_tracking":
            return ActionPlan(tools=["order.track"], next_step="show_order"), None
        elif intent.intent == "order_list":
            return ActionPlan(tools=["order.list"], next_step="show_orders"), None
        elif intent.intent.startswith("faq_"):
            return ActionPlan(tools=[], next_step="answer_faq"), None
        elif intent.intent in ["greeting", "thank_you", "goodbye"]:
            return ActionPlan(tools=[], next_step="reply_social"), None
            
        return ActionPlan(tools=[], next_step="fallback"), None

In [ ]:
# --- AGENT 4: EXECUTOR (executor.py) ---

# Mock Tools Service
class MockTools:
    @staticmethod
    async def search_products(query, limit=5):
        return ToolResults(ok=True, data=[
            {"id": "p_1", "title": f"JanSport SuperBreak - {query}", "handle": "superbreak", "price": 500000, "currency_code": "vnd"},
            {"id": "p_2", "title": f"JanSport Big Student - {query}", "handle": "big-student", "price": 800000, "currency_code": "vnd"}
        ])
    
    @staticmethod
    async def get_product_details(product_id):
        return ToolResults(ok=True, data={"id": product_id, "title": "JanSport Detail", "description": "Great bag"})

    @staticmethod
    async def recommend_products(limit=5):
        return ToolResults(ok=True, data=[{"id": "rec_1", "title": "Trending Bag", "handle": "trending"}])

    @staticmethod
    async def view_cart(cart_id):
        return ToolResults(ok=True, data={"id": cart_id, "items": []})

    @staticmethod
    async def add_to_cart(cart_id, variant_id, qty):
        return ToolResults(ok=True, data={"cart_id": cart_id, "added": True})
        
    @staticmethod
    async def add_to_cart_smart(cart_id, query, qty):
        return ToolResults(ok=True, data={"cart_id": cart_id, "added": True, "product": query})

    @staticmethod
    async def lookup_order(order_id):
        if order_id == "123":
            return ToolResults(ok=True, data={"id": "123", "status": "shipped", "delivery_date": "2023-12-25"})
        return ToolResults(ok=False, errors=["Order not found"])

class Executor:
    async def run(self, processed: ProcessedInput, intent: IntentResult, plan: ActionPlan) -> Optional[ToolResults]:
        if not plan.tools: return None
        
        results = ToolResults(ok=True)
        for tool in plan.tools:
            print(f"[Executor] Running tool: {tool}")
            if tool == "product.search":
                q = intent.entities.get("product_query", "balo")
                res = await MockTools.search_products(q)
            elif tool == "product.detail":
                res = await MockTools.get_product_details("p_1")
            elif tool == "product.recommend":
                res = await MockTools.recommend_products()
            elif tool == "cart.view":
                res = await MockTools.view_cart("cart_123")
            elif tool == "cart.add":
                res = await MockTools.add_to_cart_smart("cart_123", intent.entities.get("product_query", "bag"), 1)
            elif tool == "order.track":
                res = await MockTools.lookup_order(intent.entities.get("order_id", "000"))
            else:
                res = ToolResults(ok=False, errors=[f"Tool {tool} not implemented in mock"])
            
            # Merge results
            results.ok = results.ok and res.ok
            results.data = res.data
            results.errors.extend(res.errors)
            
        return results

In [ ]:
# --- AGENT 5: RESPONSE GENERATOR (response_generator.py) ---

class ResponseGenerator:
    def __init__(self):
        if GOOGLE_API_KEY == "YOUR_API_KEY_HERE":
            self.client = None
        else:
            from openai import AsyncOpenAI
            self.client = AsyncOpenAI(api_key=GOOGLE_API_KEY, base_url=GEMINI_BASE_URL)
        self.context_config = CONTEXT_CONFIG

    async def run(self, processed: ProcessedInput, intent: IntentResult, plan: ActionPlan, tool_res: Optional[ToolResults]) -> ChatResponse:
        # 1. Generate Text
        response_text = await self._generate_response(processed, intent, tool_res)
        
        # 2. Quick Replies
        quick_replies = [QuickReply(label=c["label"], value=c["value"]) for c in self.context_config["nodes"][0]["children"]]
        
        # 3. Products
        products = []
        if tool_res and tool_res.ok and isinstance(tool_res.data, list):
            for item in tool_res.data:
                if isinstance(item, dict) and "id" in item:
                    products.append(ProductInfo(**item))

        return ChatResponse(
            response=response_text,
            session_id=processed.session_id,
            products=products,
            quick_replies=quick_replies,
            metadata={"intent": intent.intent}
        )

    async def _generate_response(self, processed, intent, tool_res) -> str:
        # Template Logic
        if intent.intent in ["faq_shipping", "faq_payment", "faq_return", "thank_you", "goodbye"]:
            return get_response_template(intent.intent)
        
        if intent.intent == "product_inquiry":
            count = len(tool_res.data) if tool_res and tool_res.ok and isinstance(tool_res.data, list) else 0
            if count > 0:
                return get_response_template("product_found", {"count": count, "query": intent.entities.get("product_query", "")})
            return get_response_template("product_not_found", {"query": intent.entities.get("product_query", "")})

        # LLM Fallback
        if self.client:
            try:
                messages = [
                    {"role": "system", "content": SYSTEM_PROMPT_VI},
                    {"role": "user", "content": f"Input: {processed.text}\nIntent: {intent.intent}\nData: {tool_res.data if tool_res else 'None'}"}
                ]
                completion = await self.client.chat.completions.create(model=GEMINI_MODEL, messages=messages)
                return completion.choices[0].message.content
            except Exception as e:
                return f"LLM Error: {e}"
        
        return f"[MOCK LLM RESPONSE] Based on {intent.intent} and data: {tool_res.data if tool_res else 'None'}"

In [ ]:
# --- DATA LOADING ---
# Upload 'multi_agent_test_cases_full.csv' to Colab files
try:
    df = pd.read_csv('multi_agent_test_cases_full.csv')
    print(f"Loaded {len(df)} test cases.")
except FileNotFoundError:
    print("File not found. Creating sample data...")
    data = {
        "id": [1, 2, 3],
        "user_input": ["Xin chào shop", "Tìm balo JanSport màu đen", "Kiểm tra đơn hàng #123"],
        "expected_intent_classifier_intent": ["general", "product_inquiry", "order_tracking"],
        "expected_executor_tools_called": ["", "product.search", "order.track"]
    }
    df = pd.DataFrame(data)
    print("Created sample data.")

df.head()

In [ ]:
# --- EVALUATION LOOP ---

async def evaluate_row(row):
    # Initialize Agents
    input_processor = InputProcessor()
    intent_classifier = IntentClassifier()
    orchestrator = Orchestrator()
    executor = Executor()
    response_generator = ResponseGenerator()

    # 1. Input
    req = ChatRequest(session_id=f"sess_{row['id']}", message=row['user_input'])
    processed = await input_processor.run(req)
    
    # 2. Intent
    intent_res = await intent_classifier.run(processed)
    
    # 3. Orchestrator
    plan, _ = await orchestrator.run(processed, intent_res)
    
    # 4. Executor
    tool_res = await executor.run(processed, intent_res, plan)
    
    # 5. Response
    response = await response_generator.run(processed, intent_res, plan, tool_res)
    
    # --- CHECKS ---
    results = {
        "id": row['id'],
        "input": row['user_input'],
        "intent_actual": intent_res.intent,
        "intent_expected": row.get('expected_intent_classifier_intent'),
        "intent_match": intent_res.intent == row.get('expected_intent_classifier_intent'),
        
        "tools_actual": ",".join(plan.tools),
        "tools_expected": row.get('expected_executor_tools_called', ""),
        # Simple check: is expected tool in actual tools?
        "tools_match": str(row.get('expected_executor_tools_called', "")) in ",".join(plan.tools) if pd.notna(row.get('expected_executor_tools_called')) else True,
        
        "response_preview": response.response[:50] + "..."
    }
    return results

# Run Evaluation
results_list = []
print("Starting evaluation...")
for index, row in df.iterrows():
    res = await evaluate_row(row)
    results_list.append(res)
    print(f"Row {row['id']}: Intent Match={res['intent_match']}, Tools Match={res['tools_match']}")

results_df = pd.DataFrame(results_list)

In [ ]:
# --- SUMMARY REPORT ---
print("\n--- EVALUATION SUMMARY ---")
total = len(results_df)
intent_acc = results_df['intent_match'].mean() * 100
tools_acc = results_df['tools_match'].mean() * 100

print(f"Total Test Cases: {total}")
print(f"Intent Accuracy: {intent_acc:.2f}%")
print(f"Tool Selection Accuracy: {tools_acc:.2f}%")

print("\n--- FAILED CASES ---")
failed = results_df[~results_df['intent_match'] | ~results_df['tools_match']]
if not failed.empty:
    display(failed[['id', 'input', 'intent_actual', 'intent_expected', 'tools_actual', 'tools_expected']])
else:
    print("All cases passed!")

In [ ]:
# Import required libraries
import requests
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from typing import Dict, List, Any, Optional, Tuple
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import psycopg2
from psycopg2.extras import RealDictCursor
import os

# Configure display settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Service configuration
CHATBOT_SERVICE_URL = "http://localhost:8000"
MEDUSA_SERVICE_URL = "http://localhost:9000"

# Database configuration
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'postgres',
    'user': 'postgres',
    'password': 'supersecretpassword'
}

print("🤖 Chatbot Multi-Agent System Evaluation")
print("=" * 50)
print(f"📅 Evaluation Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🎯 Target Service: {CHATBOT_SERVICE_URL}")
print(f"🛍️ E-commerce Backend: {MEDUSA_SERVICE_URL}")
print(f"🗄️ Database: {DB_CONFIG['host']}:{DB_CONFIG['port']}")

## 2. Service Health Check and Database Connection

In [ ]:
def check_service_health(url: str, service_name: str) -> bool:
    """Check if a service is healthy and responsive."""
    try:
        response = requests.get(f"{url}/health", timeout=5)
        if response.status_code == 200:
            print(f"✅ {service_name}: Healthy (Status: {response.status_code})")
            return True
        else:
            print(f"⚠️ {service_name}: Unhealthy (Status: {response.status_code})")
            return False
    except requests.exceptions.RequestException as e:
        print(f"❌ {service_name}: Connection failed - {str(e)}")
        return False

def check_database_connection() -> bool:
    """Test database connectivity and check for required tables."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor(cursor_factory=RealDictCursor)
        
        # Check chatbot tables
        cursor.execute("""
            SELECT table_name FROM information_schema.tables 
            WHERE table_schema = 'public' AND table_name LIKE '%chat%'
        """)
        chatbot_tables = [row['table_name'] for row in cursor.fetchall()]
        
        # Check conversation history
        cursor.execute("SELECT COUNT(*) as count FROM chat_conversations LIMIT 1")
        conversation_count = cursor.fetchone()['count']
        
        cursor.close()
        conn.close()
        
        print(f"✅ Database: Connected successfully")
        print(f"📊 Chatbot Tables: {len(chatbot_tables)} found - {chatbot_tables}")
        print(f"💬 Conversation History: {conversation_count} records")
        return True
        
    except Exception as e:
        print(f"❌ Database: Connection failed - {str(e)}")
        return False

# Perform health checks
print("🏥 Service Health Check")
print("-" * 30)

chatbot_healthy = check_service_health(CHATBOT_SERVICE_URL, "Chatbot Service")
medusa_healthy = check_service_health(MEDUSA_SERVICE_URL, "Medusa Backend")
db_healthy = check_database_connection()

print(f"\n📋 System Status Summary:")
print(f"   • Chatbot Service: {'✅ Ready' if chatbot_healthy else '❌ Down'}")
print(f"   • Medusa Backend: {'✅ Ready' if medusa_healthy else '❌ Down'}")
print(f"   • Database: {'✅ Connected' if db_healthy else '❌ Disconnected'}")

system_ready = chatbot_healthy and medusa_healthy and db_healthy
print(f"\n🎯 Overall System: {'✅ READY FOR EVALUATION' if system_ready else '❌ NOT READY - Check failed services'}")

## 3. Test Data Generation and Intent Scenarios

In [ ]:
# Define comprehensive test scenarios for different intents and agents
test_scenarios = {
    "product_search": [
        "I'm looking for a black leather jacket",
        "Do you have any Nike sneakers?",
        "Show me wireless headphones under $100",
        "I need a laptop for gaming",
        "What's the best smartphone you have?",
        "Looking for summer dresses in size M",
        "Show me running shoes for women",
        "I want to buy a watch"
    ],
    "product_details": [
        "Tell me more about product ID prod_01234",
        "What are the specifications of this laptop?",
        "Is this item available in different colors?",
        "What's the warranty on this product?",
        "Can you show me product reviews?",
        "What sizes are available?"
    ],
    "order_inquiry": [
        "What's the status of my order #12345?",
        "When will my order arrive?",
        "Can I track my shipment?",
        "I want to cancel my recent order",
        "Show me my order history",
        "Has my order been shipped yet?",
        "I haven't received my order"
    ],
    "account_management": [
        "I forgot my password",
        "How do I update my profile?",
        "Change my shipping address",
        "I want to delete my account",
        "Show me my account information",
        "Update my payment method"
    ],
    "support_general": [
        "What's your return policy?",
        "How do I contact customer service?",
        "What payment methods do you accept?",
        "Do you offer international shipping?",
        "What are your business hours?",
        "I have a complaint about my experience"
    ],
    "recommendations": [
        "What do you recommend for me?",
        "Show me similar products",
        "What's trending right now?",
        "Suggest something for a birthday gift",
        "What goes well with this item?",
        "Recommend products for my style"
    ],
    "complex_queries": [
        "I want to return an item and buy something else with the refund",
        "Can you help me choose between these three products?",
        "I need to modify my order and change the shipping address",
        "What's the best laptop for video editing under $1500?",
        "I'm having trouble with checkout, can you help?"
    ],
    "edge_cases": [
        "asdfghjkl", # Gibberish
        "", # Empty message
        "Hello" * 100, # Very long message
        "What's the meaning of life?", # Off-topic
        "<script>alert('test')</script>", # Potential injection
        "🤖🎭🎪🎨🎯", # Only emojis
    ]
}

# Expected agent assignments for each intent
expected_agents = {
    "product_search": "product_agent",
    "product_details": "product_agent", 
    "order_inquiry": "order_agent",
    "account_management": "support_agent",
    "support_general": "support_agent",
    "recommendations": "product_agent",
    "complex_queries": "main_pipeline_agent",
    "edge_cases": "main_pipeline_agent"
}

# Generate test dataset
test_data = []
for intent, messages in test_scenarios.items():
    for message in messages:
        test_data.append({
            'message': message,
            'expected_intent': intent,
            'expected_agent': expected_agents[intent],
            'test_category': 'functional' if intent != 'edge_cases' else 'edge_case'
        })

test_df = pd.DataFrame(test_data)

print(f"📝 Test Dataset Generated")
print(f"   • Total Test Cases: {len(test_df)}")
print(f"   • Intent Categories: {len(test_scenarios)}")
print(f"   • Functional Tests: {len(test_df[test_df['test_category'] == 'functional'])}")
print(f"   • Edge Case Tests: {len(test_df[test_df['test_category'] == 'edge_case'])}")

# Display intent distribution
print(f"\n📊 Intent Distribution:")
intent_counts = test_df['expected_intent'].value_counts()
for intent, count in intent_counts.items():
    print(f"   • {intent}: {count} tests")

# Show sample test cases
print(f"\n🔍 Sample Test Cases:")
sample_df = test_df.groupby('expected_intent').head(2)[['message', 'expected_intent', 'expected_agent']]
print(sample_df.to_string(index=False))

## 4. Chat API Testing and Agent Pipeline Evaluation

In [ ]:
def send_chat_message(message: str, conversation_id: str = None, user_id: str = \"test_user\") -> Dict[str, Any]:\n    \"\"\"Send a message to the chatbot and get response with metadata.\"\"\"\n    start_time = time.time()\n    \n    try:\n        payload = {\n            \"message\": message,\n            \"user_id\": user_id,\n            \"conversation_id\": conversation_id\n        }\n        \n        response = requests.post(\n            f\"{CHATBOT_SERVICE_URL}/chat\",\n            json=payload,\n            timeout=30\n        )\n        \n        end_time = time.time()\n        response_time = (end_time - start_time) * 1000  # Convert to milliseconds\n        \n        if response.status_code == 200:\n            data = response.json()\n            return {\n                'success': True,\n                'response_time_ms': response_time,\n                'status_code': response.status_code,\n                'response': data.get('response', ''),\n                'agent_used': data.get('agent_used', 'unknown'),\n                'intent_detected': data.get('intent_detected', 'unknown'),\n                'confidence_score': data.get('confidence_score', 0.0),\n                'conversation_id': data.get('conversation_id'),\n                'tools_used': data.get('tools_used', []),\n                'context_retrieved': data.get('context_retrieved', False),\n                'escalated': data.get('escalated', False),\n                'error': None\n            }\n        else:\n            return {\n                'success': False,\n                'response_time_ms': response_time,\n                'status_code': response.status_code,\n                'response': '',\n                'agent_used': 'error',\n                'intent_detected': 'error',\n                'confidence_score': 0.0,\n                'conversation_id': None,\n                'tools_used': [],\n                'context_retrieved': False,\n                'escalated': False,\n                'error': f\"HTTP {response.status_code}: {response.text}\"\n            }\n            \n    except requests.exceptions.Timeout:\n        return {\n            'success': False,\n            'response_time_ms': 30000,  # Timeout threshold\n            'status_code': 408,\n            'response': '',\n            'agent_used': 'timeout',\n            'intent_detected': 'timeout',\n            'confidence_score': 0.0,\n            'conversation_id': None,\n            'tools_used': [],\n            'context_retrieved': False,\n            'escalated': False,\n            'error': \"Request timeout\"\n        }\n    except Exception as e:\n        end_time = time.time()\n        response_time = (end_time - start_time) * 1000\n        return {\n            'success': False,\n            'response_time_ms': response_time,\n            'status_code': 0,\n            'response': '',\n            'agent_used': 'error',\n            'intent_detected': 'error',\n            'confidence_score': 0.0,\n            'conversation_id': None,\n            'tools_used': [],\n            'context_retrieved': False,\n            'escalated': False,\n            'error': str(e)\n        }\n\ndef evaluate_chat_pipeline(test_cases: List[Dict], max_tests: int = None) -> pd.DataFrame:\n    \"\"\"Evaluate the complete chat pipeline with all test cases.\"\"\"\n    results = []\n    \n    if max_tests:\n        test_cases = test_cases[:max_tests]\n    \n    print(f\"🚀 Starting Chat Pipeline Evaluation\")\n    print(f\"   • Total Test Cases: {len(test_cases)}\")\n    print(f\"   • Target Service: {CHATBOT_SERVICE_URL}\")\n    print(\"\\n📊 Progress:\")\n    \n    for i, test_case in enumerate(test_cases, 1):\n        print(f\"   [{i:3d}/{len(test_cases)}] Testing: '{test_case['message'][:50]}{'...' if len(test_case['message']) > 50 else ''}'\", end=\" \")\n        \n        # Send message and get response\n        result = send_chat_message(test_case['message'])\n        \n        # Combine test case with result\n        combined_result = {**test_case, **result}\n        \n        # Add evaluation metrics\n        combined_result['agent_correct'] = (result['agent_used'] == test_case['expected_agent'])\n        combined_result['intent_correct'] = (result['intent_detected'] == test_case['expected_intent'])\n        combined_result['response_length'] = len(result['response'])\n        combined_result['has_response'] = len(result['response']) > 0\n        combined_result['tools_count'] = len(result['tools_used'])\n        \n        results.append(combined_result)\n        \n        # Display result status\n        if result['success']:\n            status = \"✅\" if combined_result['agent_correct'] else \"⚠️\"\n            print(f\"{status} {result['response_time_ms']:.0f}ms\")\n        else:\n            print(f\"❌ {result['error']}\")\n        \n        # Small delay to prevent overwhelming the service\n        time.sleep(0.1)\n    \n    return pd.DataFrame(results)\n\n# Execute chat pipeline evaluation\nprint(\"🧪 Executing Chat Pipeline Tests...\")\nprint(\"=\" * 50)\n\n# Test with a subset first for quick validation\nquick_test_cases = test_df.sample(n=min(20, len(test_df))).to_dict('records')\nchat_results_df = evaluate_chat_pipeline(quick_test_cases)\n\nprint(f\"\\n📈 Quick Test Results Summary:\")\nprint(f\"   • Tests Completed: {len(chat_results_df)}\")\nprint(f\"   • Success Rate: {chat_results_df['success'].mean():.2%}\")\nprint(f\"   • Agent Accuracy: {chat_results_df['agent_correct'].mean():.2%}\")\nprint(f\"   • Intent Accuracy: {chat_results_df['intent_correct'].mean():.2%}\")\nprint(f\"   • Avg Response Time: {chat_results_df[chat_results_df['success']]['response_time_ms'].mean():.0f}ms\")"

## 5. Performance Analysis and Response Time Evaluation

In [ ]:
# Analyze performance metrics from successful chat interactions\nif len(chat_results_df) > 0 and chat_results_df['success'].any():\n    successful_chats = chat_results_df[chat_results_df['success']]\n    \n    print(\"⚡ Performance Analysis\")\n    print(\"=\" * 40)\n    \n    # Overall performance statistics\n    print(f\"📊 Response Time Analysis:\")\n    print(f\"   • Average Response Time: {successful_chats['response_time_ms'].mean():.2f} ms\")\n    print(f\"   • Median Response Time: {successful_chats['response_time_ms'].median():.2f} ms\")\n    print(f\"   • 95th Percentile: {successful_chats['response_time_ms'].quantile(0.95):.2f} ms\")\n    print(f\"   • 99th Percentile: {successful_chats['response_time_ms'].quantile(0.99):.2f} ms\")\n    print(f\"   • Min Response Time: {successful_chats['response_time_ms'].min():.2f} ms\")\n    print(f\"   • Max Response Time: {successful_chats['response_time_ms'].max():.2f} ms\")\n    \n    # Agent-specific performance\n    print(f\"\\n🤖 Agent Performance Breakdown:\")\n    agent_performance = successful_chats.groupby('agent_used').agg({\n        'response_time_ms': ['mean', 'median', 'count'],\n        'confidence_score': 'mean',\n        'tools_count': 'mean'\n    }).round(2)\n    \n    for agent in agent_performance.index:\n        count = int(agent_performance.loc[agent, ('response_time_ms', 'count')])\n        avg_time = agent_performance.loc[agent, ('response_time_ms', 'mean')]\n        avg_confidence = agent_performance.loc[agent, ('confidence_score', 'mean')]\n        avg_tools = agent_performance.loc[agent, ('tools_count', 'mean')]\n        \n        print(f\"   • {agent}: {avg_time:.0f}ms avg, {avg_confidence:.2f} confidence, {avg_tools:.1f} tools ({count} tests)\")\n    \n    # Intent category performance\n    print(f\"\\n🎯 Intent Category Performance:\")\n    intent_performance = successful_chats.groupby('expected_intent').agg({\n        'response_time_ms': 'mean',\n        'intent_correct': 'mean',\n        'agent_correct': 'mean'\n    }).round(3)\n    \n    for intent in intent_performance.index:\n        avg_time = intent_performance.loc[intent, 'response_time_ms']\n        intent_acc = intent_performance.loc[intent, 'intent_correct']\n        agent_acc = intent_performance.loc[intent, 'agent_correct']\n        \n        print(f\"   • {intent}: {avg_time:.0f}ms, Intent Acc: {intent_acc:.1%}, Agent Acc: {agent_acc:.1%}\")\n    \n    # Create performance visualization\n    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))\n    \n    # Response time distribution\n    ax1.hist(successful_chats['response_time_ms'], bins=20, alpha=0.7, edgecolor='black')\n    ax1.axvline(successful_chats['response_time_ms'].mean(), color='red', linestyle='--', label='Mean')\n    ax1.axvline(successful_chats['response_time_ms'].median(), color='green', linestyle='--', label='Median')\n    ax1.set_xlabel('Response Time (ms)')\n    ax1.set_ylabel('Frequency')\n    ax1.set_title('Response Time Distribution')\n    ax1.legend()\n    ax1.grid(True, alpha=0.3)\n    \n    # Agent performance comparison\n    agent_stats = successful_chats.groupby('agent_used')['response_time_ms'].agg(['mean', 'count']).reset_index()\n    agent_stats = agent_stats[agent_stats['count'] >= 2]  # Only show agents with 2+ tests\n    \n    if len(agent_stats) > 0:\n        bars = ax2.bar(range(len(agent_stats)), agent_stats['mean'])\n        ax2.set_xlabel('Agent')\n        ax2.set_ylabel('Average Response Time (ms)')\n        ax2.set_title('Agent Performance Comparison')\n        ax2.set_xticks(range(len(agent_stats)))\n        ax2.set_xticklabels([agent.replace('_agent', '') for agent in agent_stats['agent_used']], rotation=45)\n        ax2.grid(True, alpha=0.3)\n        \n        # Add count labels on bars\n        for i, (bar, count) in enumerate(zip(bars, agent_stats['count'])):\n            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10,\n                    f'n={int(count)}', ha='center', va='bottom', fontsize=10)\n    \n    # Confidence score vs response time scatter\n    ax3.scatter(successful_chats['confidence_score'], successful_chats['response_time_ms'], alpha=0.6)\n    ax3.set_xlabel('Confidence Score')\n    ax3.set_ylabel('Response Time (ms)')\n    ax3.set_title('Confidence vs Response Time')\n    ax3.grid(True, alpha=0.3)\n    \n    # Success rate by intent category\n    intent_success = chat_results_df.groupby('expected_intent').agg({\n        'success': ['mean', 'count']\n    }).reset_index()\n    intent_success.columns = ['intent', 'success_rate', 'count']\n    intent_success = intent_success[intent_success['count'] >= 2]\n    \n    if len(intent_success) > 0:\n        colors = ['green' if sr >= 0.8 else 'orange' if sr >= 0.5 else 'red' for sr in intent_success['success_rate']]\n        bars = ax4.bar(range(len(intent_success)), intent_success['success_rate'], color=colors, alpha=0.7)\n        ax4.set_xlabel('Intent Category')\n        ax4.set_ylabel('Success Rate')\n        ax4.set_title('Success Rate by Intent Category')\n        ax4.set_xticks(range(len(intent_success)))\n        ax4.set_xticklabels(intent_success['intent'], rotation=45)\n        ax4.set_ylim(0, 1.1)\n        ax4.grid(True, alpha=0.3)\n        \n        # Add percentage labels on bars\n        for i, (bar, rate, count) in enumerate(zip(bars, intent_success['success_rate'], intent_success['count'])):\n            ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,\n                    f'{rate:.1%}\\n(n={int(count)})', ha='center', va='bottom', fontsize=9)\n    \n    plt.tight_layout()\n    plt.show()\n    \nelse:\n    print(\"❌ No successful chat interactions to analyze\")\n    print(\"   Check service connectivity and try again\")"

## 6. Intent Classification and Agent Routing Accuracy

In [ ]:
# Detailed accuracy analysis for intent classification and agent routing\nif len(chat_results_df) > 0:\n    print(\"🎯 Intent Classification & Agent Routing Analysis\")\n    print(\"=\" * 60)\n    \n    # Overall accuracy metrics\n    successful_tests = chat_results_df[chat_results_df['success']]\n    \n    if len(successful_tests) > 0:\n        intent_accuracy = successful_tests['intent_correct'].mean()\n        agent_accuracy = successful_tests['agent_correct'].mean()\n        \n        print(f\"📈 Overall Accuracy Metrics:\")\n        print(f\"   • Intent Classification Accuracy: {intent_accuracy:.2%}\")\n        print(f\"   • Agent Routing Accuracy: {agent_accuracy:.2%}\")\n        print(f\"   • System Success Rate: {chat_results_df['success'].mean():.2%}\")\n        print(f\"   • Tests with Responses: {successful_tests['has_response'].mean():.2%}\")\n        \n        # Confusion matrix for intent classification\n        if len(successful_tests) >= 5:  # Need minimum samples for meaningful analysis\n            print(f\"\\n🧩 Intent Classification Confusion Analysis:\")\n            \n            # Get unique intents for classification analysis\n            actual_intents = successful_tests['expected_intent'].values\n            predicted_intents = successful_tests['intent_detected'].values\n            \n            # Calculate per-intent accuracy\n            intent_accuracy_detail = successful_tests.groupby('expected_intent').agg({\n                'intent_correct': ['mean', 'count', 'sum'],\n                'agent_correct': 'mean',\n                'confidence_score': 'mean'\n            }).round(3)\n            \n            print(f\"\\n📊 Per-Intent Performance:\")\n            for intent in intent_accuracy_detail.index:\n                accuracy = intent_accuracy_detail.loc[intent, ('intent_correct', 'mean')]\n                correct_count = int(intent_accuracy_detail.loc[intent, ('intent_correct', 'sum')])\n                total_count = int(intent_accuracy_detail.loc[intent, ('intent_correct', 'count')])\n                agent_acc = intent_accuracy_detail.loc[intent, ('agent_correct', 'mean')]\n                avg_confidence = intent_accuracy_detail.loc[intent, ('confidence_score', 'mean')]\n                \n                status = \"✅\" if accuracy >= 0.8 else \"⚠️\" if accuracy >= 0.6 else \"❌\"\n                print(f\"   {status} {intent}: {accuracy:.1%} ({correct_count}/{total_count}), Agent: {agent_acc:.1%}, Conf: {avg_confidence:.2f}\")\n        \n        # Agent routing analysis\n        print(f\"\\n🤖 Agent Routing Analysis:\")\n        \n        # Expected vs actual agent mapping\n        agent_routing = successful_tests.groupby('expected_agent').agg({\n            'agent_correct': ['mean', 'count'],\n            'response_time_ms': 'mean',\n            'confidence_score': 'mean'\n        }).round(3)\n        \n        for expected_agent in agent_routing.index:\n            accuracy = agent_routing.loc[expected_agent, ('agent_correct', 'mean')]\n            count = int(agent_routing.loc[expected_agent, ('agent_correct', 'count')])\n            avg_time = agent_routing.loc[expected_agent, ('response_time_ms', 'mean')]\n            avg_conf = agent_routing.loc[expected_agent, ('confidence_score', 'mean')]\n            \n            status = \"✅\" if accuracy >= 0.8 else \"⚠️\" if accuracy >= 0.6 else \"❌\"\n            print(f\"   {status} {expected_agent}: {accuracy:.1%} accuracy ({count} tests), {avg_time:.0f}ms avg, {avg_conf:.2f} conf\")\n        \n        # Misclassification analysis\n        print(f\"\\n🔍 Misclassification Analysis:\")\n        misclassified_intents = successful_tests[~successful_tests['intent_correct']]\n        misrouted_agents = successful_tests[~successful_tests['agent_correct']]\n        \n        if len(misclassified_intents) > 0:\n            print(f\"   • Intent Misclassifications: {len(misclassified_intents)} cases\")\n            intent_errors = misclassified_intents.groupby(['expected_intent', 'intent_detected']).size().reset_index(name='count')\n            for _, row in intent_errors.head(10).iterrows():  # Show top 10 error patterns\n                print(f\"     - Expected '{row['expected_intent']}' → Got '{row['intent_detected']}' ({row['count']} times)\")\n        \n        if len(misrouted_agents) > 0:\n            print(f\"   • Agent Misrouting: {len(misrouted_agents)} cases\")\n            agent_errors = misrouted_agents.groupby(['expected_agent', 'agent_used']).size().reset_index(name='count')\n            for _, row in agent_errors.head(10).iterrows():  # Show top 10 error patterns\n                print(f\"     - Expected '{row['expected_agent']}' → Got '{row['agent_used']}' ({row['count']} times)\")\n        \n        # Edge case handling\n        edge_cases = chat_results_df[chat_results_df['test_category'] == 'edge_case']\n        if len(edge_cases) > 0:\n            print(f\"\\n🚨 Edge Case Handling:\")\n            edge_success_rate = edge_cases['success'].mean()\n            print(f\"   • Edge Case Success Rate: {edge_success_rate:.2%}\")\n            print(f\"   • Edge Cases Handled: {edge_cases['success'].sum()}/{len(edge_cases)}\")\n            \n            if edge_cases['success'].any():\n                successful_edges = edge_cases[edge_cases['success']]\n                print(f\"   • Avg Response Time for Edge Cases: {successful_edges['response_time_ms'].mean():.0f}ms\")\n                print(f\"   • Escalation Rate: {successful_edges['escalated'].mean():.2%}\")\n        \n        # Create accuracy visualization\n        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))\n        \n        # Intent accuracy by category\n        if len(intent_accuracy_detail) > 1:\n            intent_acc_data = intent_accuracy_detail[('intent_correct', 'mean')].sort_values(ascending=True)\n            colors = ['red' if acc < 0.6 else 'orange' if acc < 0.8 else 'green' for acc in intent_acc_data.values]\n            \n            bars = ax1.barh(range(len(intent_acc_data)), intent_acc_data.values, color=colors, alpha=0.7)\n            ax1.set_xlabel('Intent Classification Accuracy')\n            ax1.set_ylabel('Intent Category')\n            ax1.set_title('Intent Classification Accuracy by Category')\n            ax1.set_yticks(range(len(intent_acc_data)))\n            ax1.set_yticklabels(intent_acc_data.index)\n            ax1.set_xlim(0, 1.1)\n            ax1.grid(True, alpha=0.3)\n            \n            # Add percentage labels\n            for i, (bar, acc) in enumerate(zip(bars, intent_acc_data.values)):\n                ax1.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,\n                        f'{acc:.1%}', va='center', fontsize=10)\n        \n        # Agent routing accuracy\n        if len(agent_routing) > 1:\n            agent_acc_data = agent_routing[('agent_correct', 'mean')].sort_values(ascending=True)\n            colors = ['red' if acc < 0.6 else 'orange' if acc < 0.8 else 'green' for acc in agent_acc_data.values]\n            \n            bars = ax2.barh(range(len(agent_acc_data)), agent_acc_data.values, color=colors, alpha=0.7)\n            ax2.set_xlabel('Agent Routing Accuracy')\n            ax2.set_ylabel('Expected Agent')\n            ax2.set_title('Agent Routing Accuracy by Agent Type')\n            ax2.set_yticks(range(len(agent_acc_data)))\n            ax2.set_yticklabels([agent.replace('_agent', '') for agent in agent_acc_data.index])\n            ax2.set_xlim(0, 1.1)\n            ax2.grid(True, alpha=0.3)\n            \n            # Add percentage labels\n            for i, (bar, acc) in enumerate(zip(bars, agent_acc_data.values)):\n                ax2.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,\n                        f'{acc:.1%}', va='center', fontsize=10)\n        \n        # Confidence score distribution for correct vs incorrect classifications\n        correct_confidences = successful_tests[successful_tests['intent_correct']]['confidence_score']\n        incorrect_confidences = successful_tests[~successful_tests['intent_correct']]['confidence_score']\n        \n        if len(correct_confidences) > 0:\n            ax3.hist(correct_confidences, bins=15, alpha=0.7, label='Correct', color='green', density=True)\n        if len(incorrect_confidences) > 0:\n            ax3.hist(incorrect_confidences, bins=15, alpha=0.7, label='Incorrect', color='red', density=True)\n        \n        ax3.set_xlabel('Confidence Score')\n        ax3.set_ylabel('Density')\n        ax3.set_title('Confidence Score Distribution: Correct vs Incorrect')\n        ax3.legend()\n        ax3.grid(True, alpha=0.3)\n        \n        # Response time vs accuracy correlation\n        ax4.scatter(successful_tests[successful_tests['intent_correct']]['response_time_ms'], \n                   successful_tests[successful_tests['intent_correct']]['confidence_score'], \n                   alpha=0.6, color='green', label='Correct Intent', s=50)\n        ax4.scatter(successful_tests[~successful_tests['intent_correct']]['response_time_ms'], \n                   successful_tests[~successful_tests['intent_correct']]['confidence_score'], \n                   alpha=0.6, color='red', label='Wrong Intent', s=50)\n        \n        ax4.set_xlabel('Response Time (ms)')\n        ax4.set_ylabel('Confidence Score')\n        ax4.set_title('Response Time vs Confidence (by Accuracy)')\n        ax4.legend()\n        ax4.grid(True, alpha=0.3)\n        \n        plt.tight_layout()\n        plt.show()\n        \n    else:\n        print(\"❌ No successful chat responses to analyze accuracy\")\n        print(\"   • Check service health and test data\")\nelse:\n    print(\"❌ No test results available for accuracy analysis\")"

## 7. Conversation Flow and Context Management

In [ ]:
# Test multi-turn conversations and context retention\ndef test_conversation_flow() -> List[Dict[str, Any]]:\n    \"\"\"Test multi-turn conversations to evaluate context management.\"\"\"\n    \n    conversation_scenarios = [\n        {\n            \"name\": \"Product Search + Details\",\n            \"turns\": [\n                \"I'm looking for a laptop\",\n                \"What about gaming laptops?\",\n                \"Show me the specifications of the first one\",\n                \"What's the price?\"\n            ],\n            \"expected_context\": True\n        },\n        {\n            \"name\": \"Order Tracking Flow\",\n            \"turns\": [\n                \"Check my order status\",\n                \"My order number is ORD-12345\",\n                \"When will it arrive?\",\n                \"Can I change the delivery address?\"\n            ],\n            \"expected_context\": True\n        },\n        {\n            \"name\": \"Support Issue Resolution\",\n            \"turns\": [\n                \"I have a problem with my recent purchase\",\n                \"The item doesn't match the description\",\n                \"I want to return it\",\n                \"How long does the return process take?\"\n            ],\n            \"expected_context\": True\n        },\n        {\n            \"name\": \"Mixed Intent Conversation\",\n            \"turns\": [\n                \"Hi, I need help with my account\",\n                \"Actually, I want to buy a gift\",\n                \"What do you recommend for a teenager?\",\n                \"Can I get it wrapped?\"\n            ],\n            \"expected_context\": False  # Context switching\n        }\n    ]\n    \n    print(\"💬 Testing Conversation Flow & Context Management\")\n    print(\"=\" * 55)\n    \n    conversation_results = []\n    \n    for scenario_idx, scenario in enumerate(conversation_scenarios, 1):\n        print(f\"\\n🗣️ Scenario {scenario_idx}: {scenario['name']}\")\n        print(f\"   Expected Context Retention: {'✅ Yes' if scenario['expected_context'] else '❌ No'}\")\n        \n        conversation_id = None\n        scenario_results = []\n        \n        for turn_idx, message in enumerate(scenario['turns'], 1):\n            print(f\"   Turn {turn_idx}: '{message[:40]}{'...' if len(message) > 40 else ''}'\", end=\" \")\n            \n            # Send message with conversation context\n            result = send_chat_message(message, conversation_id=conversation_id)\n            \n            # Update conversation ID from first response\n            if turn_idx == 1 and result['conversation_id']:\n                conversation_id = result['conversation_id']\n            \n            # Add scenario metadata\n            result.update({\n                'scenario_name': scenario['name'],\n                'turn_number': turn_idx,\n                'total_turns': len(scenario['turns']),\n                'expected_context_retention': scenario['expected_context'],\n                'message_text': message\n            })\n            \n            scenario_results.append(result)\n            \n            # Display turn result\n            if result['success']:\n                context_status = \"🔗\" if result['context_retrieved'] else \"🔸\"\n                print(f\"{context_status} {result['response_time_ms']:.0f}ms\")\n            else:\n                print(f\"❌ {result['error']}\")\n        \n        conversation_results.extend(scenario_results)\n        \n        # Analyze scenario completion\n        scenario_success_rate = sum(1 for r in scenario_results if r['success']) / len(scenario_results)\n        avg_response_time = np.mean([r['response_time_ms'] for r in scenario_results if r['success']])\n        context_retention_rate = sum(1 for r in scenario_results if r.get('context_retrieved', False)) / len(scenario_results)\n        \n        print(f\"   📊 Scenario Summary: {scenario_success_rate:.1%} success, {avg_response_time:.0f}ms avg, {context_retention_rate:.1%} context\")\n    \n    return conversation_results\n\n# Execute conversation flow tests\nprint(\"🧪 Running Conversation Flow Tests...\")\nconversation_results = test_conversation_flow()\nconversation_df = pd.DataFrame(conversation_results)\n\n# Analyze conversation flow results\nif len(conversation_df) > 0:\n    print(f\"\\n📈 Conversation Flow Analysis\")\n    print(\"=\" * 40)\n    \n    successful_conversations = conversation_df[conversation_df['success']]\n    \n    if len(successful_conversations) > 0:\n        print(f\"📊 Overall Conversation Metrics:\")\n        print(f\"   • Conversation Success Rate: {conversation_df['success'].mean():.2%}\")\n        print(f\"   • Context Retrieval Rate: {successful_conversations['context_retrieved'].mean():.2%}\")\n        print(f\"   • Avg Response Time per Turn: {successful_conversations['response_time_ms'].mean():.0f}ms\")\n        print(f\"   • Total Conversation Turns: {len(conversation_df)}\")\n        \n        # Per-scenario analysis\n        print(f\"\\n🎭 Per-Scenario Performance:\")\n        scenario_stats = conversation_df.groupby('scenario_name').agg({\n            'success': 'mean',\n            'context_retrieved': 'mean',\n            'response_time_ms': 'mean',\n            'turn_number': 'count'\n        }).round(3)\n        \n        for scenario in scenario_stats.index:\n            success_rate = scenario_stats.loc[scenario, 'success']\n            context_rate = scenario_stats.loc[scenario, 'context_retrieved']\n            avg_time = scenario_stats.loc[scenario, 'response_time_ms']\n            turn_count = int(scenario_stats.loc[scenario, 'turn_number'])\n            \n            status = \"✅\" if success_rate >= 0.8 else \"⚠️\" if success_rate >= 0.6 else \"❌\"\n            print(f\"   {status} {scenario}: {success_rate:.1%} success, {context_rate:.1%} context, {avg_time:.0f}ms ({turn_count} turns)\")\n        \n        # Turn position analysis\n        print(f\"\\n🔄 Turn Position Analysis:\")\n        turn_analysis = successful_conversations.groupby('turn_number').agg({\n            'response_time_ms': ['mean', 'count'],\n            'context_retrieved': 'mean',\n            'confidence_score': 'mean'\n        }).round(2)\n        \n        for turn_num in sorted(turn_analysis.index):\n            avg_time = turn_analysis.loc[turn_num, ('response_time_ms', 'mean')]\n            count = int(turn_analysis.loc[turn_num, ('response_time_ms', 'count')])\n            context_rate = turn_analysis.loc[turn_num, ('context_retrieved', 'mean')]\n            confidence = turn_analysis.loc[turn_num, ('confidence_score', 'mean')]\n            \n            print(f\"   Turn {turn_num}: {avg_time:.0f}ms avg, {context_rate:.1%} context, {confidence:.2f} conf ({count} samples)\")\n        \n        # Context retention visualization\n        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))\n        \n        # Response time by turn number\n        turn_times = successful_conversations.groupby('turn_number')['response_time_ms'].agg(['mean', 'std']).reset_index()\n        ax1.errorbar(turn_times['turn_number'], turn_times['mean'], yerr=turn_times['std'], \n                    marker='o', capsize=5, capthick=2, linewidth=2)\n        ax1.set_xlabel('Turn Number')\n        ax1.set_ylabel('Response Time (ms)')\n        ax1.set_title('Response Time by Conversation Turn')\n        ax1.grid(True, alpha=0.3)\n        \n        # Context retrieval by scenario\n        if len(scenario_stats) > 1:\n            scenario_context = scenario_stats['context_retrieved'].sort_values(ascending=True)\n            colors = ['red' if rate < 0.3 else 'orange' if rate < 0.7 else 'green' for rate in scenario_context.values]\n            \n            bars = ax2.barh(range(len(scenario_context)), scenario_context.values, color=colors, alpha=0.7)\n            ax2.set_xlabel('Context Retrieval Rate')\n            ax2.set_ylabel('Scenario')\n            ax2.set_title('Context Retrieval by Scenario')\n            ax2.set_yticks(range(len(scenario_context)))\n            ax2.set_yticklabels([name.replace(' ', '\\n') for name in scenario_context.index], fontsize=9)\n            ax2.set_xlim(0, 1.1)\n            ax2.grid(True, alpha=0.3)\n            \n            # Add percentage labels\n            for i, (bar, rate) in enumerate(zip(bars, scenario_context.values)):\n                ax2.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,\n                        f'{rate:.1%}', va='center', fontsize=10)\n        \n        # Confidence vs turn number\n        ax3.scatter(successful_conversations['turn_number'], successful_conversations['confidence_score'], \n                   alpha=0.6, s=60)\n        \n        # Add trendline\n        if len(successful_conversations) > 5:\n            z = np.polyfit(successful_conversations['turn_number'], successful_conversations['confidence_score'], 1)\n            p = np.poly1d(z)\n            ax3.plot(successful_conversations['turn_number'].unique(), \n                    p(successful_conversations['turn_number'].unique()), \"r--\", alpha=0.8)\n        \n        ax3.set_xlabel('Turn Number')\n        ax3.set_ylabel('Confidence Score')\n        ax3.set_title('Confidence Score by Turn Number')\n        ax3.grid(True, alpha=0.3)\n        \n        # Success rate by scenario\n        if len(scenario_stats) > 1:\n            scenario_success = scenario_stats['success'].sort_values(ascending=True)\n            colors = ['red' if rate < 0.6 else 'orange' if rate < 0.8 else 'green' for rate in scenario_success.values]\n            \n            bars = ax4.barh(range(len(scenario_success)), scenario_success.values, color=colors, alpha=0.7)\n            ax4.set_xlabel('Success Rate')\n            ax4.set_ylabel('Scenario')\n            ax4.set_title('Success Rate by Scenario')\n            ax4.set_yticks(range(len(scenario_success)))\n            ax4.set_yticklabels([name.replace(' ', '\\n') for name in scenario_success.index], fontsize=9)\n            ax4.set_xlim(0, 1.1)\n            ax4.grid(True, alpha=0.3)\n            \n            # Add percentage labels\n            for i, (bar, rate) in enumerate(zip(bars, scenario_success.values)):\n                ax4.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,\n                        f'{rate:.1%}', va='center', fontsize=10)\n        \n        plt.tight_layout()\n        plt.show()\n        \n    else:\n        print(\"❌ No successful conversation turns to analyze\")\nelse:\n    print(\"❌ No conversation test results available\")"

## 8. Generate Comprehensive Evaluation Report

In [ ]:
# Generate comprehensive chatbot evaluation report\nprint(\"📊 Generating Comprehensive Chatbot Evaluation Report...\")\nprint(\"=\" * 80)\n\n# Report timestamp\nreport_time = datetime.now()\nprint(f\"📅 Report Generated: {report_time.strftime('%Y-%m-%d %H:%M:%S')}\")\nprint(f\"🤖 System Evaluated: Multi-Agent Chatbot Service (Port 8000)\")\nprint(f\"⏱️ Evaluation Duration: {(report_time - datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds() / 3600:.2f} hours\")\n\nprint(\"\\n\" + \"=\"*80)\nprint(\"📈 EXECUTIVE SUMMARY\")\nprint(\"=\"*80)\n\n# Calculate overall metrics\nall_results = []\nif len(chat_results_df) > 0:\n    all_results.extend(chat_results_df.to_dict('records'))\nif len(conversation_df) > 0:\n    all_results.extend(conversation_df.to_dict('records'))\n\nif len(all_results) > 0:\n    all_df = pd.DataFrame(all_results)\n    successful_tests = all_df[all_df['success']]\n    \n    if len(successful_tests) > 0:\n        overall_success_rate = all_df['success'].mean()\n        avg_response_time = successful_tests['response_time_ms'].mean()\n        p95_response_time = successful_tests['response_time_ms'].quantile(0.95)\n        \n        # Calculate intent and agent accuracy for available data\n        intent_accuracy = successful_tests['intent_correct'].mean() if 'intent_correct' in successful_tests.columns else 0\n        agent_accuracy = successful_tests['agent_correct'].mean() if 'agent_correct' in successful_tests.columns else 0\n        context_retrieval_rate = successful_tests['context_retrieved'].mean() if 'context_retrieved' in successful_tests.columns else 0\n        \n        print(f\"✅ Overall System Health: {'EXCELLENT' if overall_success_rate > 0.95 else 'GOOD' if overall_success_rate > 0.85 else 'NEEDS IMPROVEMENT'}\")\n        print(f\"🎯 Chatbot Intelligence: {'HIGH' if intent_accuracy > 0.8 else 'MEDIUM' if intent_accuracy > 0.6 else 'LOW'}\")\n        print(f\"⚡ Performance Grade: {'A' if p95_response_time < 2000 else 'B' if p95_response_time < 3500 else 'C'}\")\n        print(f\"💬 Context Management: {'EXCELLENT' if context_retrieval_rate > 0.8 else 'GOOD' if context_retrieval_rate > 0.6 else 'BASIC'}\")\n\nprint(\"\\n\" + \"=\"*80)\nprint(\"🎯 KEY PERFORMANCE INDICATORS\")\nprint(\"=\"*80)\n\nif len(successful_tests) > 0:\n    print(f\"🤖 Chatbot Intelligence Metrics:\")\n    print(f\"   • Intent Classification Accuracy: {intent_accuracy:.2%}\")\n    print(f\"   • Agent Routing Accuracy: {agent_accuracy:.2%}\")\n    print(f\"   • Overall System Success Rate: {overall_success_rate:.2%}\")\n    \n    if 'confidence_score' in successful_tests.columns:\n        avg_confidence = successful_tests['confidence_score'].mean()\n        print(f\"   • Average Confidence Score: {avg_confidence:.3f}\")\n    \n    print(f\"\\n⚡ Performance Metrics:\")\n    print(f\"   • Average Response Time: {avg_response_time:.2f} ms\")\n    print(f\"   • Median Response Time: {successful_tests['response_time_ms'].median():.2f} ms\")\n    print(f\"   • 95th Percentile Response Time: {p95_response_time:.2f} ms\")\n    print(f\"   • 99th Percentile Response Time: {successful_tests['response_time_ms'].quantile(0.99):.2f} ms\")\n    \n    print(f\"\\n💬 Conversation Management:\")\n    if context_retrieval_rate > 0:\n        print(f\"   • Context Retrieval Rate: {context_retrieval_rate:.2%}\")\n    \n    if 'escalated' in successful_tests.columns:\n        escalation_rate = successful_tests['escalated'].mean()\n        print(f\"   • Escalation Rate: {escalation_rate:.2%}\")\n    \n    print(f\"\\n🔧 System Reliability:\")\n    print(f\"   • Total Test Cases: {len(all_df)}\")\n    print(f\"   • Successful Interactions: {len(successful_tests)}\")\n    print(f\"   • Failed Interactions: {len(all_df) - len(successful_tests)}\")\n\n# Agent-specific performance\nif len(successful_tests) > 0 and 'agent_used' in successful_tests.columns:\n    print(\"\\n\" + \"=\"*80)\n    print(\"🤖 AGENT PERFORMANCE BREAKDOWN\")\n    print(\"=\"*80)\n    \n    agent_performance = successful_tests.groupby('agent_used').agg({\n        'response_time_ms': ['mean', 'median', 'count'],\n        'confidence_score': 'mean' if 'confidence_score' in successful_tests.columns else lambda x: 0,\n    }).round(2)\n    \n    for agent in agent_performance.index:\n        count = int(agent_performance.loc[agent, ('response_time_ms', 'count')])\n        avg_time = agent_performance.loc[agent, ('response_time_ms', 'mean')]\n        median_time = agent_performance.loc[agent, ('response_time_ms', 'median')]\n        \n        print(f\"🔸 {agent.replace('_', ' ').title()}:\")\n        print(f\"   • Interactions Handled: {count}\")\n        print(f\"   • Average Response Time: {avg_time:.0f}ms\")\n        print(f\"   • Median Response Time: {median_time:.0f}ms\")\n        \n        if 'confidence_score' in successful_tests.columns:\n            avg_confidence = agent_performance.loc[agent, ('confidence_score', 'mean')]\n            print(f\"   • Average Confidence: {avg_confidence:.3f}\")\n\n# Intent category analysis\nif len(successful_tests) > 0 and 'expected_intent' in successful_tests.columns:\n    print(\"\\n\" + \"=\"*80)\n    print(\"🎯 INTENT CLASSIFICATION ANALYSIS\")\n    print(\"=\"*80)\n    \n    intent_stats = successful_tests.groupby('expected_intent').agg({\n        'intent_correct': ['mean', 'count'] if 'intent_correct' in successful_tests.columns else lambda x: [0, len(x)],\n        'response_time_ms': 'mean',\n        'agent_correct': 'mean' if 'agent_correct' in successful_tests.columns else lambda x: 0\n    }).round(3)\n    \n    print(\"Intent Performance Summary:\")\n    for intent in intent_stats.index:\n        if 'intent_correct' in successful_tests.columns:\n            accuracy = intent_stats.loc[intent, ('intent_correct', 'mean')]\n            count = int(intent_stats.loc[intent, ('intent_correct', 'count')])\n            status = \"✅\" if accuracy >= 0.8 else \"⚠️\" if accuracy >= 0.6 else \"❌\"\n            print(f\"   {status} {intent}: {accuracy:.1%} accuracy ({count} tests)\")\n\n# Edge case handling\nif len(all_df) > 0 and 'test_category' in all_df.columns:\n    edge_cases = all_df[all_df['test_category'] == 'edge_case']\n    if len(edge_cases) > 0:\n        print(\"\\n\" + \"=\"*80)\n        print(\"🚨 EDGE CASE HANDLING\")\n        print(\"=\"*80)\n        \n        edge_success_rate = edge_cases['success'].mean()\n        edge_response_time = edge_cases[edge_cases['success']]['response_time_ms'].mean()\n        \n        print(f\"   • Edge Cases Tested: {len(edge_cases)}\")\n        print(f\"   • Edge Case Success Rate: {edge_success_rate:.2%}\")\n        print(f\"   • Average Response Time: {edge_response_time:.0f}ms\")\n        \n        if edge_success_rate < 0.5:\n            print(f\"   ⚠️ Warning: Low edge case handling capability\")\n        elif edge_success_rate > 0.8:\n            print(f\"   ✅ Excellent edge case resilience\")\n\nprint(\"\\n\" + \"=\"*80)\nprint(\"✅ GOALS ACHIEVEMENT ASSESSMENT\")\nprint(\"=\"*80)\n\n# Define chatbot-specific targets\nchatbot_targets = {\n    'system_success_rate': {'target': 0.90, 'actual': overall_success_rate if 'overall_success_rate' in locals() else 0, 'unit': '%'},\n    'intent_accuracy': {'target': 0.85, 'actual': intent_accuracy if 'intent_accuracy' in locals() else 0, 'unit': '%'},\n    'agent_accuracy': {'target': 0.85, 'actual': agent_accuracy if 'agent_accuracy' in locals() else 0, 'unit': '%'},\n    'response_time_p95': {'target': 3000, 'actual': p95_response_time if 'p95_response_time' in locals() else 0, 'unit': 'ms'},\n    'context_retention': {'target': 0.70, 'actual': context_retrieval_rate if 'context_retrieval_rate' in locals() else 0, 'unit': '%'}\n}\n\nfor metric, data in chatbot_targets.items():\n    target = data['target']\n    actual = data['actual']\n    unit = data['unit']\n    \n    if unit == '%':\n        if metric == 'response_time_p95':\n            status = '✅ PASS' if actual <= target else '❌ FAIL'\n            print(f\"{metric.replace('_', ' ').title()}: {actual:.2f}{unit} (Target: <{target}{unit}) {status}\")\n        else:\n            status = '✅ PASS' if actual >= target else '❌ FAIL'\n            print(f\"{metric.replace('_', ' ').title()}: {actual:.2%} (Target: ≥{target:.0%}) {status}\")\n    else:\n        status = '✅ PASS' if actual <= target else '❌ FAIL'\n        print(f\"{metric.replace('_', ' ').title()}: {actual:.2f}{unit} (Target: ≤{target}{unit}) {status}\")\n\nprint(\"\\n\" + \"=\"*80)\nprint(\"🔧 RECOMMENDATIONS FOR IMPROVEMENT\")\nprint(\"=\"*80)\n\nrecommendations = []\n\n# Performance recommendations\nif 'p95_response_time' in locals() and p95_response_time > 3000:\n    recommendations.append(\"⚡ Optimize response time: P95 exceeds 3000ms target\")\nif 'intent_accuracy' in locals() and intent_accuracy < 0.85:\n    recommendations.append(\"🎯 Improve NLU training: Intent accuracy below 85%\")\nif 'agent_accuracy' in locals() and agent_accuracy < 0.85:\n    recommendations.append(\"🤖 Enhance agent routing logic: Agent accuracy below 85%\")\nif 'overall_success_rate' in locals() and overall_success_rate < 0.90:\n    recommendations.append(\"🛠️ Improve error handling: Success rate below 90%\")\nif 'context_retrieval_rate' in locals() and context_retrieval_rate < 0.70:\n    recommendations.append(\"💬 Enhance context management: Context retrieval below 70%\")\n\n# General chatbot improvements\nrecommendations.extend([\n    \"📚 Expand training data with more diverse conversation patterns\",\n    \"🔄 Implement active learning from production conversations\",\n    \"📊 Add conversation satisfaction scoring\",\n    \"🎭 Implement personality consistency across agents\",\n    \"🔍 Add conversation analytics and user behavior tracking\",\n    \"⚙️ Set up automated A/B testing for conversation flows\"\n])\n\nfor i, rec in enumerate(recommendations, 1):\n    print(f\"{i}. {rec}\")\n\nprint(\"\\n\" + \"=\"*80)\nprint(\"💾 DATA EXPORT AND RESULTS\")\nprint(\"=\"*80)\n\n# Save comprehensive evaluation results\nresults_data = {\n    'single_turn_tests': chat_results_df.to_dict('records') if len(chat_results_df) > 0 else [],\n    'conversation_tests': conversation_df.to_dict('records') if len(conversation_df) > 0 else [],\n    'summary_metrics': {\n        'overall_success_rate': overall_success_rate if 'overall_success_rate' in locals() else 0,\n        'intent_accuracy': intent_accuracy if 'intent_accuracy' in locals() else 0,\n        'agent_accuracy': agent_accuracy if 'agent_accuracy' in locals() else 0,\n        'avg_response_time': avg_response_time if 'avg_response_time' in locals() else 0,\n        'p95_response_time': p95_response_time if 'p95_response_time' in locals() else 0,\n        'context_retrieval_rate': context_retrieval_rate if 'context_retrieval_rate' in locals() else 0,\n        'total_tests': len(all_df) if len(all_df) > 0 else 0,\n        'evaluation_timestamp': report_time.isoformat()\n    },\n    'test_scenarios': {\n        'single_turn_scenarios': len(test_df) if 'test_df' in locals() else 0,\n        'conversation_scenarios': len(conversation_df.groupby('scenario_name')) if len(conversation_df) > 0 else 0,\n        'edge_cases_tested': len(edge_cases) if 'edge_cases' in locals() else 0\n    }\n}\n\n# Save results to JSON file\nimport os\nos.makedirs('d:/Edu/graduation-project/report/evaluation/results', exist_ok=True)\nresults_file = f'd:/Edu/graduation-project/report/evaluation/results/chatbot_evaluation_{report_time.strftime(\"%Y%m%d_%H%M%S\")}.json'\n\nwith open(results_file, 'w', encoding='utf-8') as f:\n    json.dump(results_data, f, indent=2, default=str, ensure_ascii=False)\n\nprint(f\"📁 Evaluation results saved to: {results_file}\")\n\n# Generate summary statistics\nif len(all_df) > 0:\n    print(f\"\\n📊 Test Coverage Summary:\")\n    print(f\"   • Single-turn tests: {len(chat_results_df) if len(chat_results_df) > 0 else 0}\")\n    print(f\"   • Multi-turn conversations: {len(conversation_df.groupby('scenario_name')) if len(conversation_df) > 0 else 0} scenarios\")\n    print(f\"   • Total interactions tested: {len(all_df)}\")\n    print(f\"   • Test categories covered: {len(all_df['test_category'].unique()) if 'test_category' in all_df.columns else 'N/A'}\")\n\nprint(\"\\n\" + \"=\"*80)\nprint(\"🎉 CHATBOT EVALUATION COMPLETED SUCCESSFULLY!\")\nprint(\"=\"*80)\nprint(f\"🤖 Multi-Agent System: {'✅ Production Ready' if overall_success_rate > 0.85 and intent_accuracy > 0.8 else '⚠️ Needs Optimization'}\")\nprint(f\"📊 Total Test Cases: {len(all_df) if len(all_df) > 0 else 0}\")\nprint(f\"⏱️ Evaluation Time: {report_time.strftime('%H:%M:%S')}\")\nprint(f\"💾 Results Saved: {results_file}\")\nprint(\"✅ Ready for deployment assessment and user acceptance testing!\")"